In [2]:
import requests
import pandas as pd
import json
import time
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import threading
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import re
from tkinter import filedialog
import configparser
import os
import concurrent.futures
import queue
import webbrowser  # 이 코드를 다른 import문 근처에 추가
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment

class ComplexSelectionDialog:
    def __init__(self, parent, title, complex_list):
        self.result = None
        
        # 대화상자 생성
        self.dialog = tk.Toplevel(parent)
        self.dialog.title(title)
        self.dialog.geometry("700x500")  # 크기를 더 크게 설정 (원래 600x400)
        self.dialog.minsize(700, 500)    # 최소 크기 설정
        self.dialog.transient(parent)
        self.dialog.grab_set()
    
        # 프레임 생성
        frame = ttk.Frame(self.dialog, padding="10")
        frame.pack(fill=tk.BOTH, expand=True)
        
        # 라벨 생성
        ttk.Label(frame, text="검색된 단지 목록. 선택하세요:").pack(anchor="w", pady=(0, 5))
        
        # 트리뷰 생성
        columns = ("name", "address")
        self.tree = ttk.Treeview(frame, columns=columns, show="headings", height=15)
        self.tree.heading("name", text="단지명")
        self.tree.heading("address", text="주소")
        
        self.tree.column("name", width=250)
        self.tree.column("address", width=400)
        
        # 스크롤바 생성
        scrollbar = ttk.Scrollbar(frame, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        
        self.tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # 데이터 추가
        for item in complex_list:
            self.tree.insert("", "end", values=(item["name"], item["address"]), tags=(item["id"],))
        
        # 버튼 프레임 - 패딩 늘림
        button_frame = ttk.Frame(self.dialog, padding="20")  # 패딩 값 증가 (원래 10)
        button_frame.pack(fill="x", pady=10)  # 상하 여백 추가
        
        # 확인 버튼
        select_button = ttk.Button(button_frame, text="선택", command=self.on_select, width=10)  # 버튼 너비 명시
        select_button.pack(side="right", padx=10)  # 여백 증가
        
        # 취소 버튼
        cancel_button = ttk.Button(button_frame, text="취소", command=self.on_cancel, width=10)  # 버튼 너비 명시
        cancel_button.pack(side="right", padx=10)  # 여백 증가
        
        # 더블 클릭 이벤트 바인딩
        self.tree.bind("<Double-1>", lambda e: self.on_select())
        
        # 창 중앙 배치
        self.center_dialog()
        
        # 대화상자가 닫힐 때까지 대기
        parent.wait_window(self.dialog)
    
    def center_dialog(self):
        """대화상자를 화면 중앙에 배치"""
        self.dialog.update_idletasks()
        
        # 화면 크기 가져오기
        screen_width = self.dialog.winfo_screenwidth()
        screen_height = self.dialog.winfo_screenheight()
        
        # 창 크기 가져오기
        dialog_width = self.dialog.winfo_width()
        dialog_height = self.dialog.winfo_height()
        
        # 화면 중앙 위치 계산
        x = (screen_width - dialog_width) // 2
        y = (screen_height - dialog_height) // 2
        
        # 위치 설정
        self.dialog.geometry(f"+{x}+{y}")
    
    def on_select(self):
        selected_items = self.tree.selection()
        if selected_items:
            item_id = selected_items[0]
            item_values = self.tree.item(item_id)['values']
            item_tags = self.tree.item(item_id)['tags']
            
            self.result = {
                "name": item_values[0],
                "address": item_values[1],
                "id": item_tags[0]
            }
            
        self.dialog.destroy()
    
    def on_cancel(self):
        self.dialog.destroy()

class NaverRealEstateApp:
# NaverRealEstateApp 클래스의 __init__ 메서드에 log_queue 초기화 코드 추가
    def __init__(self, root):
        self.root = root
        self.root.title("네이버 부동산 매물 수집기")
        self.root.geometry("600x450")  # 창 크기 확장
        self.root.resizable(True, True)
        
        # 변수 초기화
        self.complex_data = None
        self.driver = None
        
        # 로그 큐 초기화 추가
        self.log_queue = queue.Queue()
        
        # 설정 관리
        self.config = configparser.ConfigParser()
        self.save_path = os.path.expanduser("~/Documents")  # 기본 저장 경로
        self.load_config()
        
        # 탭 컨트롤 생성
        self.tab_control = ttk.Notebook(self.root)
        
        # 탭 1: 단일 단지 검색 (기존 기능)
        self.tab1 = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab1, text='단일 단지 검색')
        
        # 탭 2: 다중 단지 검색 (새 기능)
        self.tab2 = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab2, text='다중 단지 검색')
        
        self.tab_control.pack(expand=1, fill="both")
        
        # 단일 단지 검색 UI 설정
        self.setup_single_search_ui()
        
        # 다중 단지 검색 UI 설정
        self.setup_multi_search_ui()
        
        # 상태 표시 레이블
        self.status_label = tk.Label(
            self.root, 
            text="단지명을 입력하고 검색하세요", 
            font=("맑은 고딕", 9),
            pady=15
        )
        self.status_label.pack(fill="x")
        
        # 제작자 정보 프레임
        self.setup_author_info()
        
        # 로그 큐 처리 시작
        self.root.after(100, self.process_log_queue)


    def setup_single_search_ui(self):
        """단일 단지 검색 UI 설정 (기존 기능)"""
        # 프레임 구성
        search_frame = ttk.LabelFrame(self.tab1, text="아파트 단지 검색")
        search_frame.pack(fill="x", padx=10, pady=5)
        
        # 검색 프레임 구성
        ttk.Label(search_frame, text="단지명:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.search_entry = ttk.Entry(search_frame, width=40)
        self.search_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.search_entry.bind("<Return>", lambda event: self.start_search())
        
        self.search_button = ttk.Button(search_frame, text="검색 및 매물 수집", command=self.start_search)
        self.search_button.grid(row=0, column=2, padx=5, pady=5)
        
        # 설정 버튼 추가
        self.settings_button = ttk.Button(search_frame, text="⚙", width=3, command=self.open_settings)
        self.settings_button.grid(row=0, column=3, padx=5, pady=5)
        
        search_frame.columnconfigure(1, weight=1)


    def setup_author_info(self):
        """제작자 정보 프레임 설정"""
        author_frame = ttk.Frame(self.root)
        author_frame.pack(side="bottom", fill="x", padx=10, pady=10)
        
        # 제작자 이름 (클릭 가능한 링크)
        author_label = tk.Label(author_frame, text="만든이 부태리", font=("맑은 고딕", 9, "bold"), fg="black", cursor="hand2")
        author_label.pack(anchor="center")
        author_label.bind("<Button-1>", lambda e: self.open_blog())
        
        # 블로그 주소
        blog_label = tk.Label(author_frame, text="https://blog.naver.com/landlover333", font=("맑은 고딕", 8))
        blog_label.pack(anchor="center")
        blog_label.bind("<Button-1>", lambda e: self.open_blog())

    def setup_multi_search_ui(self):
        """다중 단지 검색 UI 설정 (새 기능)"""
        # 메인 프레임
        main_frame = ttk.Frame(self.tab2, padding=10)
        main_frame.pack(fill="both", expand=True)
        
        # 왼쪽 영역 (단지 입력)
        left_frame = ttk.LabelFrame(main_frame, text="단지 목록 입력 (최대 25개)")
        left_frame.pack(side="left", fill="both", expand=True, padx=(0, 5))
        
        # 단지 입력용 Text 위젯
        ttk.Label(left_frame, text="단지명을 한 줄에 하나씩 입력하세요:").pack(anchor="w", pady=(5, 0))
        self.complex_text = scrolledtext.ScrolledText(left_frame, width=30, height=15)
        self.complex_text.pack(fill="both", expand=True, pady=5)
        
        # 오른쪽 영역 (옵션 설정)
        right_frame = ttk.LabelFrame(main_frame, text="검색 옵션")
        right_frame.pack(side="right", fill="both", padx=(5, 0))
        
        # 전용면적 범위 설정
        ttk.Label(right_frame, text="전용면적 범위:").grid(row=0, column=0, sticky="w", pady=5)
        
        area_frame = ttk.Frame(right_frame)
        area_frame.grid(row=0, column=1, sticky="ew", pady=5)
        
        self.min_area_var = tk.StringVar()
        self.max_area_var = tk.StringVar()
        
        ttk.Entry(area_frame, textvariable=self.min_area_var, width=6).pack(side="left")
        ttk.Label(area_frame, text="㎡ ~").pack(side="left", padx=2)
        ttk.Entry(area_frame, textvariable=self.max_area_var, width=6).pack(side="left")
        ttk.Label(area_frame, text="㎡").pack(side="left", padx=2)
        
        # 층수 조건 정보 (정보만 표시)
        ttk.Label(right_frame, text="층수 조건:").grid(row=1, column=0, sticky="w", pady=5)
        ttk.Label(right_frame, text="중층/고층/5층 이상 매물만 검색").grid(row=1, column=1, sticky="w", pady=5)
        
        # 거래 유형 정보 (정보만 표시)
        ttk.Label(right_frame, text="거래 유형:").grid(row=2, column=0, sticky="w", pady=5)
        ttk.Label(right_frame, text="매매 및 전세 최저가 조사").grid(row=2, column=1, sticky="w", pady=5)
        
        # 버튼 영역
        button_frame = ttk.Frame(right_frame)
        button_frame.grid(row=3, column=0, columnspan=2, pady=10)
        
        self.multi_search_button = ttk.Button(button_frame, text="다중 검색 시작", command=self.start_multi_search, width=20)
        self.multi_search_button.pack(pady=5)
        
        # 진행상황 표시
        self.progress_text = scrolledtext.ScrolledText(right_frame, width=30, height=10, state='disabled')
        self.progress_text.grid(row=4, column=0, columnspan=2, sticky="ew", pady=5)

        
    def start_multi_search(self):
        """다중 단지 검색 시작"""
        # 입력 텍스트에서 단지명 리스트 추출
        complex_text = self.complex_text.get('1.0', tk.END).strip()
        complex_list = [name.strip() for name in complex_text.split('\n') if name.strip()]
        
        # 최대 25개로 제한
        if len(complex_list) > 25:
            messagebox.showwarning("경고", "최대 25개까지만 검색 가능합니다. 처음 25개만 처리합니다.")
            complex_list = complex_list[:25]
        
        if not complex_list:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        # 전용면적 범위 확인
        min_area = self.min_area_var.get().strip()
        max_area = self.max_area_var.get().strip()
        
        try:
            min_area = float(min_area) if min_area else None
            max_area = float(max_area) if max_area else None
        except ValueError:
            messagebox.showwarning("경고", "전용면적은 숫자로 입력하세요.")
            return
        
        # 검색 옵션 저장
        search_options = {
            'min_area': min_area,
            'max_area': max_area
        }
        
        # 진행 상황 초기화
        self.clear_progress_text()
        self.append_progress_text(f"총 {len(complex_list)}개 단지 검색을 시작합니다.\n")
        
        # 검색 버튼 비활성화
        self.multi_search_button.config(state=tk.DISABLED)
        
        # 별도 스레드로 검색 시작
        threading.Thread(target=self.process_multi_search, 
                         args=(complex_list, search_options), 
                         daemon=True).start()
    
    def process_multi_search(self, complex_list, search_options):
        """다중 단지 검색 처리 (별도 스레드에서 실행)"""
        try:
            # 결과 저장용 리스트
            all_results = []        # 모든 단지의 결과
            summary_results = []    # 최저가 요약 결과
            
            # 각 단지별로 처리
            for idx, complex_name in enumerate(complex_list):
                self.append_progress_text(f"[{idx+1}/{len(complex_list)}] '{complex_name}' 검색 중...")
                
                # 단지 정보 검색
                complex_id = self.find_complex_id(complex_name)
                
                if not complex_id:
                    self.append_progress_text(f"  - 단지를 찾을 수 없습니다.\n")
                    continue
                
                self.append_progress_text(f"  - 단지번호: {complex_id}")
                
                # 매물 정보 수집
                property_data = self.collect_property_data(complex_id, complex_name, search_options)
                
                if not property_data:
                    self.append_progress_text(f"  - 매물 정보가 없습니다.\n")
                    continue
                
                # 데이터프레임으로 변환
                df = pd.DataFrame(property_data)
                
                # 전체 매물 저장 (각 단지별 전체 결과 저장용)
                all_results.append({
                    'complex_name': complex_name,
                    'complex_id': complex_id,
                    'properties': df
                })
                
                # === 최저가 찾기 (pandas 활용) ===
                
                # 1. 전용면적 필터링
                if search_options['min_area'] or search_options['max_area']:
                    area_conditions = []
                    if search_options['min_area']:
                        area_conditions.append(f"전용면적 >= {search_options['min_area']}")
                    if search_options['max_area']:
                        area_conditions.append(f"전용면적 <= {search_options['max_area']}")
                    
                    area_query = " and ".join(area_conditions)
                    try:
                        df = df.query(area_query)
                        self.append_progress_text(f"  - 전용면적 필터링: {area_query}")
                    except Exception as e:
                        self.append_progress_text(f"  - 전용면적 필터링 오류: {str(e)}")
                
                # 2. 층수 필터링 (저층 제외, 5층 이상 포함)
                # 2. 층수 필터링 (저층 및 1~4층 제외)
                try:
                    # copy()를 사용하여 명시적 복사본 생성
                    df = df.copy()
                    df_original = df.copy()
                    
                    def is_high_floor(floor_info):
                        # 기존과 같은 고층 판단 함수
                        floor_str = str(floor_info)
                        if pd.isna(floor_info) or floor_str.strip() == '':
                            return False
                        if '저' in floor_str:
                            return False
                        if '중' in floor_str or '고' in floor_str:
                            return True
                        parts = floor_str.split('/')
                        if len(parts) >= 1:
                            try:
                                floor_nums = re.findall(r'\d+', parts[0])
                                if floor_nums:
                                    floor_num = int(floor_nums[0])
                                    return floor_num >= 5
                            except:
                                pass
                        return False
                    
                    df['is_high_floor'] = df['층/전체층'].apply(is_high_floor)
                    df_high = df[df['is_high_floor']]
                    df_low = df[~df['is_high_floor']]
                    
                    if not df_high.empty:
                        df = df_high
                        self.append_progress_text("  - 고층(중층/5층 이상) 매물이 선택됨")
                    elif not df_low.empty:
                        df = df_low
                        self.append_progress_text("  - 고층 매물이 없어 저층/1~4층 매물 중 최저가 매물이 선택됨")
                    else:
                        self.append_progress_text("  - 조건에 맞는 매물이 없습니다.\n")
                        continue

                except Exception as e:
                    self.append_progress_text(f"  - 층수 필터링 오류: {str(e)}")
                
                # 3. 매매/전세 최저가 찾기
                # 매매 최저가
                deal_min = None
                try:
                    # 거래유형이 'A1'(매매)인 행 필터링
                    df_deal = df[df['거래유형'] == 'A1']
                    if not df_deal.empty:
                        # 매물 정보 출력 (디버깅용)
                        self.append_progress_text(f"  - [디버그] 매매 매물 수: {len(df_deal)}")
                        
                        # 매매가를 숫자형으로 변환 (문자열인 경우를 대비)
                        df_deal = df_deal.copy()  # 여기서 복사본 만들기
                        df_deal.loc[:, '매매가_숫자'] = pd.to_numeric(df_deal['매매가'], errors='coerce')
                        df_deal = df_deal.dropna(subset=['매매가_숫자'])
                        
                        if not df_deal.empty:
                            # 매매가를 기준으로 정렬
                            df_deal = df_deal.sort_values('매매가_숫자')
                            # 최저가 매물 선택
                            deal_min = df_deal.iloc[0].to_dict()
                            
                            # 가격 표시 (안전한 형변환 포함)
                            try:
                                price_val = float(deal_min['매매가'])
                                self.append_progress_text(f"  - 매매 최저가: {deal_min['동']} {deal_min['층/전체층']} {price_val/10000:.1f}만원")
                            except (ValueError, TypeError):
                                self.append_progress_text(f"  - 매매 최저가: {deal_min['동']} {deal_min['층/전체층']} (가격 변환 오류)")
                    else:
                        self.append_progress_text("  - 조건에 맞는 매매 매물이 없습니다.")
                except Exception as e:
                    self.append_progress_text(f"  - 매매 최저가 계산 오류: {str(e)}")
                
                # 전세 최저가
                jeonse_min = None
                try:
                    # 거래유형이 'B1'(전세)인 행 필터링
                    df_jeonse = df[df['거래유형'] == 'B1']
                    if not df_jeonse.empty:
                        # 매물 정보 출력 (디버깅용)
                        self.append_progress_text(f"  - [디버그] 전세 매물 수: {len(df_jeonse)}")
                        
                        # 보증금을 숫자형으로 변환 (문자열인 경우를 대비)
                        df_jeonse = df_jeonse.copy()  # 복사본 만들기
                        df_jeonse.loc[:, '보증금_숫자'] = pd.to_numeric(df_jeonse['보증금'], errors='coerce')
                        df_jeonse = df_jeonse.dropna(subset=['보증금_숫자'])
               
                        
                        if not df_jeonse.empty:
                            # 보증금을 기준으로 정렬
                            df_jeonse = df_jeonse.sort_values('보증금_숫자')
                            # 최저가 매물 선택
                            jeonse_min = df_jeonse.iloc[0].to_dict()
                            
                            # 가격 표시 (안전한 형변환 포함)
                            try:
                                price_val = float(jeonse_min['보증금'])
                                self.append_progress_text(f"  - 전세 최저가: {jeonse_min['동']} {jeonse_min['층/전체층']} {price_val/10000:.1f}만원")
                            except (ValueError, TypeError):
                                self.append_progress_text(f"  - 전세 최저가: {jeonse_min['동']} {jeonse_min['층/전체층']} (가격 변환 오류)")
                    else:
                        self.append_progress_text("  - 조건에 맞는 전세 매물이 없습니다.")
                except Exception as e:
                    self.append_progress_text(f"  - 전세 최저가 계산 오류: {str(e)}")
                
                # 4. 요약 결과에 추가
                summary = {
                    '단지명': complex_name,
                    '단지ID': complex_id,
                    '전용면적_Min': search_options.get('min_area', ''),
                    '전용면적_Max': search_options.get('max_area', '')
                }
                
                # 매매 정보 추가
                if deal_min is not None:
                    summary['매매가'] = float(deal_min['매매가']) / 10000
                    summary['매매_동'] = deal_min['동']
                    summary['매매_층'] = deal_min['층/전체층']
                    summary['매매_면적'] = deal_min['전용면적']
                else:
                    summary['매매가'] = ''
                    summary['매매_동'] = ''
                    summary['매매_층'] = ''
                    summary['매매_면적'] = ''
                
                # 전세 정보 추가
                if jeonse_min is not None:
                    summary['전세가'] = float(jeonse_min['보증금']) / 10000
                    summary['전세_동'] = jeonse_min['동']
                    summary['전세_층'] = jeonse_min['층/전체층']
                    summary['전세_면적'] = jeonse_min['전용면적']
                else:
                    summary['전세가'] = ''
                    summary['전세_동'] = ''
                    summary['전세_층'] = ''
                    summary['전세_면적'] = ''
                
                summary_results.append(summary)
                self.append_progress_text("")
            
            # 결과를 엑셀로 저장
            if summary_results:
                self.save_multi_search_results(summary_results, all_results)
            else:
                self.append_progress_text("검색 완료: 조건에 맞는 매물이 없습니다.")
                messagebox.showinfo("검색 완료", "조건에 맞는 매물이 없습니다.")
        
        except Exception as e:
            self.append_progress_text(f"오류 발생: {str(e)}")
            messagebox.showerror("오류", f"다중 검색 중 오류가 발생했습니다.\n{str(e)}")
        
        finally:
            # 검색 버튼 활성화
            self.root.after(0, lambda: self.multi_search_button.config(state=tk.NORMAL))
                
    def find_cheapest_properties(self, properties, search_options):
        """조건에 맞는 매매/전세 최저가 매물 찾기 (저층 제외)"""
        try:
            # 원본 데이터 백업
            all_properties = properties.copy()
            
            # 전용면적 필터링 (모든 매물에 적용)
            if search_options['min_area'] or search_options['max_area']:
                filtered_properties = []
                for prop in all_properties:
                    try:
                        area = float(prop.get('전용면적', 0))
                        if (search_options['min_area'] is None or area >= search_options['min_area']) and \
                           (search_options['max_area'] is None or area <= search_options['max_area']):
                            filtered_properties.append(prop)
                    except (ValueError, TypeError):
                        self.append_progress_text(f"  - 경고: 전용면적 값 '{prop.get('전용면적')}' 형식 오류")
                all_properties = filtered_properties
            
            # 거래유형 필터링 (매매와 전세만 포함)
            all_properties = [prop for prop in all_properties if prop.get('거래유형') in ['A1', 'B1']]
            
            if not all_properties:
                self.append_progress_text("  - 조건에 맞는 매물이 없습니다.")
                return None
            
            # 층수 필터링 (저층 및 1~4층 제외)
            preferred_properties = []
            excluded_properties = []
            
            for prop in all_properties:
                floor_info = str(prop.get('층/전체층', ''))
                
                # 가격 정보 표시용
                if prop.get('거래유형') == 'A1':
                    try:
                        price_val = float(prop.get('매매가', 0))
                        price_info = f"매매 {price_val/10000:.1f}만원"
                    except (ValueError, TypeError):
                        price_info = f"매매 가격 정보 오류"
                else:
                    try:
                        price_val = float(prop.get('보증금', 0))
                        price_info = f"전세 {price_val/10000:.1f}만원"
                    except (ValueError, TypeError):
                        price_info = f"전세 가격 정보 오류"
                
                # 층수 정보 없는 경우
                if not floor_info or floor_info.strip() == "":
                    excluded_properties.append(prop)
                    self.append_progress_text(f"  - 제외: 층수 정보 없음, {price_info}")
                    continue
                    
                # 강화된 저층 필터링: '저'가 포함된 모든 경우 제외 (가장 중요한 수정 부분)
                if '저' in floor_info:
                    excluded_properties.append(prop)
                    self.append_progress_text(f"  - 제외: {floor_info} (저층 포함), {price_info}")
                    continue
                
                # 중/고층 포함
                if '중' in floor_info or '고' in floor_info:
                    preferred_properties.append(prop)
                    self.append_progress_text(f"  - 포함: {floor_info} (중층/고층), {price_info}")
                    continue
                
                # 숫자로 된 층수 확인 (예: "15/30")
                try:
                    # 슬래시로 분리
                    parts = floor_info.split('/')
                    if len(parts) >= 1:
                        current_floor = parts[0].strip()
                        
                        # 현재 층이 순수 숫자인지 확인
                        if current_floor.isdigit():
                            floor_num = int(current_floor)
                            
                            # 5층 이상만 포함
                            if floor_num >= 5:
                                preferred_properties.append(prop)
                                self.append_progress_text(f"  - 포함: {floor_info} ({floor_num}층), {price_info}")
                            else:
                                excluded_properties.append(prop)
                                self.append_progress_text(f"  - 제외: {floor_info} ({floor_num}층 < 5층), {price_info}")
                        else:
                            # 숫자가 아닌 경우 제외
                            excluded_properties.append(prop)
                            self.append_progress_text(f"  - 제외: {floor_info} (층수 숫자 아님), {price_info}")
                    else:
                        # 슬래시로 분리 불가능한 경우 제외
                        excluded_properties.append(prop)
                        self.append_progress_text(f"  - 제외: {floor_info} (형식 인식 불가), {price_info}")
                except Exception as e:
                    # 파싱 오류 시 제외
                    excluded_properties.append(prop)
                    self.append_progress_text(f"  - 제외: {floor_info} (파싱 오류: {str(e)}), {price_info}")
            
            # 필터링 결과 요약
            self.append_progress_text(f"  - 총 {len(all_properties)}개 매물 중 {len(preferred_properties)}개 포함 (중층/고층/5층이상), {len(excluded_properties)}개 제외")
            
            # 필터링된 매물이 없는 경우 안내
            if not preferred_properties:
                self.append_progress_text("  - 중층/고층/5층 이상 매물이 없습니다.")
                return {}
            
            # 매매/전세 구분
            cheapest_properties = {}
            
            # 매매 최저가 찾기
            deal_properties = [p for p in preferred_properties if p.get('거래유형') == 'A1']
            if deal_properties:
                try:
                    # 추가 안전 장치: 저층 매물 다시 한번 필터링 (최종 확인)
                    safe_deal_properties = [p for p in deal_properties if '저' not in str(p.get('층/전체층', ''))]
                    
                    if not safe_deal_properties:
                        self.append_progress_text("  - 조건에 맞는 매매 매물이 없습니다.")
                    else:
                        # 디버깅용 출력: 모든 매매 매물의 층수와 가격 로깅
                        self.append_progress_text("  - [디버그] 포함된 매매 매물 목록:")
                        for idx, p in enumerate(safe_deal_properties):
                            try:
                                price_val = float(p.get('매매가', 0))
                                self.append_progress_text(f"    {idx+1}. 층: {p.get('층/전체층', '')}, 가격: {price_val/10000:.1f}만원")
                            except:
                                pass
                        
                        # 숫자 타입 확인하여 최저가 찾기
                        def get_price(prop):
                            try:
                                return float(prop.get('매매가', float('inf')))
                            except (ValueError, TypeError):
                                return float('inf')
                        
                        cheapest_deal = min(safe_deal_properties, key=get_price)
                        cheapest_properties['A1'] = cheapest_deal
                        price_val = float(cheapest_deal.get('매매가', 0))
                        self.append_progress_text(f"  - 매매 최저가: {cheapest_deal.get('층/전체층', '')} {price_val/10000:.1f}만원")
                except Exception as e:
                    self.append_progress_text(f"  - 매매 최저가 계산 중 오류: {str(e)}")
            else:
                self.append_progress_text("  - 조건에 맞는 매매 매물이 없습니다.")
            
            # 전세 최저가 찾기
            jeonse_properties = [p for p in preferred_properties if p.get('거래유형') == 'B1']
            if jeonse_properties:
                try:
                    # 추가 안전 장치: 저층 매물 다시 한번 필터링 (최종 확인)
                    safe_jeonse_properties = [p for p in jeonse_properties if '저' not in str(p.get('층/전체층', ''))]
                    
                    if not safe_jeonse_properties:
                        self.append_progress_text("  - 조건에 맞는 전세 매물이 없습니다.")
                    else:
                        # 디버깅용 출력: 모든 전세 매물의 층수와 가격 로깅
                        self.append_progress_text("  - [디버그] 포함된 전세 매물 목록:")
                        for idx, p in enumerate(safe_jeonse_properties):
                            try:
                                price_val = float(p.get('보증금', 0))
                                self.append_progress_text(f"    {idx+1}. 층: {p.get('층/전체층', '')}, 가격: {price_val/10000:.1f}만원")
                            except:
                                pass
                        
                        # 숫자 타입 확인하여 최저가 찾기
                        def get_deposit(prop):
                            try:
                                return float(prop.get('보증금', float('inf')))
                            except (ValueError, TypeError):
                                return float('inf')
                        
                        cheapest_jeonse = min(safe_jeonse_properties, key=get_deposit)
                        cheapest_properties['B1'] = cheapest_jeonse
                        price_val = float(cheapest_jeonse.get('보증금', 0))
                        self.append_progress_text(f"  - 전세 최저가: {cheapest_jeonse.get('층/전체층', '')} {price_val/10000:.1f}만원")
                except Exception as e:
                    self.append_progress_text(f"  - 전세 최저가 계산 중 오류: {str(e)}")
            else:
                self.append_progress_text("  - 조건에 맞는 전세 매물이 없습니다.")
            
            return cheapest_properties
        
        except Exception as e:
            self.append_progress_text(f"  - 최저가 매물 검색 중 오류: {str(e)}")
            import traceback
            self.append_progress_text(f"  - 상세 오류: {traceback.format_exc()}")
            return None
    def find_complex_id(self, complex_name):
        """단지명으로 단지번호 검색"""
        try:
            # Chrome 옵션 설정
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
            
            service = Service(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service, options=chrome_options)
            
            # 네이버 부동산 접속
            self.append_progress_text(f"  - '{complex_name}' 단지 검색 중...")
            driver.get("https://fin.land.naver.com/")
            time.sleep(2)
            
            # 검색 버튼 클릭
            search_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "svg[viewBox='0 0 24 24']"))
            )
            search_button.click()
            time.sleep(1)
            
            # 검색창에 단지명 입력
            search_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "query"))
            )
            search_input.clear()
            search_input.send_keys(complex_name)
            search_input.send_keys(Keys.ENTER)
            time.sleep(3)
            
            # 검색 결과 확인
            complex_id = None
            
            # 복합 단지 목록 찾기
            complex_items = driver.find_elements(By.CSS_SELECTOR, "#complex_list_ul .result_item")
            
            if complex_items and len(complex_items) > 0:
                # 첫 번째 결과의 단지 ID 추출
                item = complex_items[0]
                link_element = item.find_element(By.CSS_SELECTOR, "a.inner")
                href = link_element.get_attribute("href")
                
                # href에서 단지번호 추출
                id_match = re.search(r"/complex/info/(\d+)", href)
                if id_match:
                    complex_id = id_match.group(1)
            else:
                # 단일 단지 페이지로 이동한 경우 URL에서 ID 추출
                current_url = driver.current_url
                
                # URL에서 단지 ID 추출
                pattern1 = r"complexes/(\d+)"
                pattern2 = r"complexNumber=(\d+)"
                pattern3 = r"/complex/info/(\d+)"
                
                match1 = re.search(pattern1, current_url)
                match2 = re.search(pattern2, current_url)
                match3 = re.search(pattern3, current_url)
                
                if match1:
                    complex_id = match1.group(1)
                elif match2:
                    complex_id = match2.group(1)
                elif match3:
                    complex_id = match3.group(1)
            
            driver.quit()
            return complex_id
        
        except Exception as e:
            self.append_progress_text(f"  - 단지번호 검색 중 오류: {str(e)}")
            try:
                if driver:
                    driver.quit()
            except:
                pass
            return None


    
    def collect_property_data(self, complex_number, complex_name, search_options):
        """단지의 매물 정보 수집"""
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
            }
            
            all_properties = []
            
            # 첫 페이지 요청
            # url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&userChannelType=PC&page=0"
            url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page=0"
                    
            
            try:
                response = requests.get(url, headers=headers)
                
                if response.status_code != 200:
                    self.append_progress_text(f"  - API 요청 실패: 상태 코드 {response.status_code}")
                    return None
                
                data = response.json()
                
                if 'result' not in data or 'list' not in data['result']:
                    self.append_progress_text("  - 데이터 구조가 예상과 다릅니다.")
                    return None
                
                property_list = data['result']['list']
                
                if not property_list:
                    self.append_progress_text("  - 매물 정보가 없습니다.")
                    return None
                
                # 첫 페이지 데이터 처리
                self.append_progress_text(f"  - 첫 페이지에서 {len(property_list)}개의 매물 발견")
                for item in property_list:
                    property_data = self.extract_property_data(item, 0)
                    all_properties.append(property_data)
                
                # 다음 페이지 여부 확인
                has_next_page = data['result'].get('hasNextPage', False)
                
                # 모든 페이지 처리 (최대 50페이지로 제한 - 필요시 조정)
                max_pages = 50  # 최대 페이지 수 증가
                page = 1
                consecutive_empty_pages = 0  # 연속된 빈 페이지 수 
                
                while has_next_page and page < max_pages:
                    url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
               
                    
                    # 최대 3번 재시도
                    max_retries = 3
                    retry_count = 0
                    success = False
                    
                    while retry_count < max_retries and not success:
                        try:
                            response = requests.get(url, headers=headers, timeout=10)
                            if response.status_code == 200:
                                success = True
                            else:
                                retry_count += 1
                                time.sleep(1)  # 1초 대기 후 재시도
                        except Exception as e:
                            self.append_progress_text(f"  - 페이지 {page} 요청 오류 (시도 {retry_count+1}/{max_retries}): {str(e)}")
                            retry_count += 1
                            time.sleep(1)
                    
                    if not success:
                        self.append_progress_text(f"  - 페이지 {page} 요청 실패")
                        break
                    
                    try:
                        data = response.json()
                        if 'result' not in data or 'list' not in data['result']:
                            self.append_progress_text(f"  - 페이지 {page} 데이터 구조 오류")
                            consecutive_empty_pages += 1
                            if consecutive_empty_pages >= 3:  # 3개 연속 오류면 중단
                                break
                            page += 1
                            continue
                        
                        property_list = data['result']['list']
                        
                        if not property_list:
                            self.append_progress_text(f"  - 페이지 {page}에 매물 없음")
                            consecutive_empty_pages += 1
                            if consecutive_empty_pages >= 3:  # 3개 연속 빈 페이지면 중단
                                break
                        else:
                            consecutive_empty_pages = 0  # 데이터가 있으면 카운터 초기화
                            
                            self.append_progress_text(f"  - 페이지 {page}에서 {len(property_list)}개의 매물 발견")
                            for item in property_list:
                                property_data = self.extract_property_data(item, page)
                                all_properties.append(property_data)
                        
                        # 다음 페이지 확인
                        has_next_page = data['result'].get('hasNextPage', False)
                        if not has_next_page:
                            break
                        
                        page += 1
                        
                    except Exception as e:
                        self.append_progress_text(f"  - 페이지 {page} 처리 중 오류: {str(e)}")
                        consecutive_empty_pages += 1
                        if consecutive_empty_pages >= 3:
                            break
                        page += 1
                
                self.append_progress_text(f"  - 총 {len(all_properties)}개의 매물 정보 수집 완료")
                return all_properties
            
            except Exception as e:
                self.append_progress_text(f"  - 데이터 요청 중 오류: {str(e)}")
                return None
        
        except Exception as e:
            self.append_progress_text(f"  - 매물 수집 중 오류: {str(e)}")
            return None



    def save_multi_search_results(self, summary_results, all_results=None):
        """다중 검색 결과를 엑셀로 저장"""
        try:
            # 날짜 형식화
            today = datetime.now().strftime('%Y%m%d')
            
            # 저장 파일명
            filename = f'단지별_매물정보_{today}.xlsx'
            excel_filename = os.path.join(self.save_path, filename)
            
            # 데이터프레임 생성
            summary_df = pd.DataFrame(summary_results)
            
            # 엑셀 저장
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                # 최저가 매물 시트
                summary_df.to_excel(writer, sheet_name='단지별_최저가', index=False)
                
                # 엑셀 시트 서식 설정 (최저가 시트)
                workbook = writer.book
                worksheet = writer.sheets['단지별_최저가']
                
                # 헤더 스타일
                header_font = Font(bold=True, size=11)
                header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
                
                for cell in worksheet[1]:
                    cell.font = header_font
                    cell.fill = header_fill
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                
                # 열 너비 자동 조정
                for column in worksheet.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    
                    for cell in column:
                        if cell.value:
                            try:
                                cell_length = len(str(cell.value))
                                max_length = max(max_length, cell_length)
                            except:
                                pass
                    
                    max_length = max(max_length, 10)
                    max_length = min(max_length, 50)
                    worksheet.column_dimensions[column_letter].width = max_length + 2
                
                # 필터 적용
                worksheet.auto_filter.ref = f"A1:{chr(64 + len(summary_df.columns))}1"
                
                # 각 단지별 모든 매물 시트 추가 (새로 추가된 부분)
                if all_results:
                    for result in all_results:
                        complex_name = result['complex_name']
                        properties_df = result['properties']
                        
                        # 시트명 (Excel 시트명 제약: 31자 이내)
                        sheet_name = complex_name[:30]
                        
                        # 거래유형 변환
                        if '거래유형' in properties_df.columns:
                            trade_type_map = {
                                'A1': '매매',
                                'B1': '전세',
                                'B2': '월세',
                                'B3': '단기임대'
                            }
                            properties_df['거래유형'] = properties_df['거래유형'].map(lambda x: trade_type_map.get(x, x))
                        
                        # 방향 변환
                        if '방향' in properties_df.columns:
                            direction_map = {
                                'SS': '남향',
                                'SE': '남동향',
                                'SW': '남서향',
                                'EE': '동향',
                                'WW': '서향',
                                'NN': '북향',
                                'NE': '북동향',
                                'NW': '북서향'
                            }
                            properties_df['방향'] = properties_df['방향'].map(lambda x: direction_map.get(x, x))
                        
                        # DataFrame을 엑셀 시트로 저장
                        properties_df.to_excel(writer, sheet_name=sheet_name, index=False)
                        
                        # 단지별 시트 서식 설정
                        sheet = writer.sheets[sheet_name]
                        
                        # 헤더 스타일 적용
                        for cell in sheet[1]:
                            cell.font = header_font
                            cell.fill = header_fill
                            cell.alignment = Alignment(horizontal='center', vertical='center')
                        
                        # 열 너비 자동 조정
                        for column in sheet.columns:
                            max_length = 0
                            column_letter = column[0].column_letter
                            
                            for cell in column:
                                if cell.value:
                                    try:
                                        cell_length = len(str(cell.value))
                                        max_length = max(max_length, cell_length)
                                    except:
                                        pass
                            
                            max_length = max(max_length, 10)
                            max_length = min(max_length, 50)
                            sheet.column_dimensions[column_letter].width = max_length + 2
                        
                        # 필터 적용
                        sheet.auto_filter.ref = f"A1:{chr(64 + len(properties_df.columns))}1"
            
            self.append_progress_text(f"검색 완료: 총 {len(summary_df)}개 단지의 매물 정보가 저장되었습니다.")
            self.append_progress_text(f"파일 저장 경로: {excel_filename}")
            messagebox.showinfo("검색 완료", f"총 {len(summary_df)}개 단지의 매물 정보가 저장되었습니다.\n\n파일 저장 경로: {excel_filename}")
            
        except Exception as e:
            self.append_progress_text(f"결과 저장 중 오류: {str(e)}")
            messagebox.showerror("오류", f"결과 저장 중 오류가 발생했습니다.\n{str(e)}")
    # NaverRealEstateApp 클래스에 아래 메서드를 추가해주세요
    def process_log_queue(self):
        """로그 큐의 메시지를 처리"""
        try:
            while True:
                message = self.log_queue.get_nowait()
                
                # 콘솔에 출력 (디버깅용)
                print(message)
                
                # 상태 표시 업데이트
                if "단지 검색을 시작합니다" in message:
                    self.status_label.config(text="검색 중...")
                elif "매물 정보 수집을 시작합니다" in message:
                    self.status_label.config(text="매물 수집 중...")
                elif "총 " in message and "개의 매물 정보가 저장되었습니다" in message:
                    match = re.search(r'총 (\d+)개', message)
                    if match:
                        count = match.group(1)
                        self.status_label.config(text=f"{count}건 매물 검색 완료")
                elif "엑셀 파일" in message and "생성 완료" in message:
                    self.status_label.config(text="파일 저장 완료")
                
                self.log_queue.task_done()
        except queue.Empty:
            pass
        
        # 100ms 후 다시 확인
        self.root.after(100, self.process_log_queue)
    
    def format_price(self, property_data):
        """매물 가격 표시 형식화"""
        trade_type = property_data.get('거래유형')
        
        if trade_type == 'A1':  # 매매
            price = property_data.get('매매가', 0)
            return f"매매 {price/10000:.0f}만원"
        elif trade_type == 'B1':  # 전세
            price = property_data.get('보증금', 0)
            return f"전세 {price/10000:.0f}만원"
        elif trade_type == 'B2':  # 월세
            deposit = property_data.get('보증금', 0)
            monthly = property_data.get('월세', 0)
            return f"월세 {deposit/10000:.0f}/{monthly}만원"
        else:
            return "가격 정보 없음"

    def append_progress_text(self, text):
        """진행 상황 텍스트 추가 (스레드 안전)"""
        def _update():
            self.progress_text.config(state='normal')
            self.progress_text.insert(tk.END, text + "\n")
            self.progress_text.see(tk.END)
            self.progress_text.config(state='disabled')
        
        if threading.current_thread() is threading.main_thread():
            _update()
        else:
            self.root.after(0, _update)
    
    def clear_progress_text(self):
        """진행 상황 텍스트 초기화"""
        self.progress_text.config(state='normal')
        self.progress_text.delete('1.0', tk.END)
        self.progress_text.config(state='disabled')    




            
    def setup_ui(self):
        # 프레임 구성
        search_frame = ttk.LabelFrame(self.root, text="아파트 단지 검색")
        search_frame.pack(fill="x", padx=10, pady=5)
        
        # 검색 프레임 구성
        ttk.Label(search_frame, text="단지명:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.search_entry = ttk.Entry(search_frame, width=40)
        self.search_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.search_entry.bind("<Return>", lambda event: self.start_search())
        
        self.search_button = ttk.Button(search_frame, text="검색 및 매물 수집", command=self.start_search)
        self.search_button.grid(row=0, column=2, padx=5, pady=5)
        
        # 설정 버튼 추가
        self.settings_button = ttk.Button(search_frame, text="⚙", width=3, command=self.open_settings)
        self.settings_button.grid(row=0, column=3, padx=5, pady=5)
        
        search_frame.columnconfigure(1, weight=1)
        
        # 상태 표시 레이블 - 중앙에 크게 표시
        self.status_label = tk.Label(
            self.root, 
            text="단지명을 입력하고 검색하세요", 
            font=("맑은 고딕", 9),
            pady=15
        )
        self.status_label.pack(fill="x")
        
        # 제작자 정보 프레임
        author_frame = ttk.Frame(self.root)
        author_frame.pack(side="bottom", fill="x", padx=10, pady=10)
        
        # 제작자 이름 (클릭 가능한 링크)
        author_label = tk.Label(author_frame, text="만든이 부태리", font=("맑은 고딕", 9, "bold"), fg="black", cursor="hand2")
        author_label.pack(anchor="center")
        author_label.bind("<Button-1>", lambda e: self.open_blog())
        
        # 블로그 주소
        blog_label = tk.Label(author_frame, text="https://blog.naver.com/landlover333", font=("맑은 고딕", 8))
        blog_label.pack(anchor="center")
        blog_label.bind("<Button-1>", lambda e: self.open_blog())
    def open_blog(self):
        """블로그 링크 열기"""
        webbrowser.open("https://blog.naver.com/landlover333")        
        
    def load_config(self):
        """설정 파일 로드"""
        try:
            # 설정 파일 경로
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if os.path.exists(config_path):
                self.config.read(config_path)
                if 'Settings' in self.config and 'save_path' in self.config['Settings']:
                    path = self.config['Settings']['save_path']
                    if os.path.exists(path):
                        self.save_path = path
        except Exception as e:
            print(f"설정 로드 오류: {str(e)}")
            
    def save_config(self):
        """설정 파일 저장"""
        try:
            # 설정 파일 경로
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if 'Settings' not in self.config:
                self.config['Settings'] = {}
            
            self.config['Settings']['save_path'] = self.save_path
            
            with open(config_path, 'w') as f:
                self.config.write(f)
        except Exception as e:
            print(f"설정 저장 오류: {str(e)}")
    
    def open_settings(self):
        """설정 창 열기"""
        # 폴더 선택 대화상자
        path = filedialog.askdirectory(initialdir=self.save_path, title="저장할 폴더 선택")
        
        if path:
            self.save_path = path
            self.save_config()
            self.log(f"저장 경로가 변경되었습니다: {path}")
    
    def log(self, message):
        """간단한 로깅"""
        # 콘솔에만 출력
        print(message)
        
        # 상태 라벨 업데이트 (있는 경우)
        if hasattr(self, 'status_label'):
            if "단지 검색을 시작합니다" in message:
                self.status_label.config(text="검색 중...")
            elif "매물 정보 수집을 시작합니다" in message:
                self.status_label.config(text="매물 수집 중...")
            elif "총 " in message and "개의 매물 정보가 저장되었습니다" in message:
                match = re.search(r'총 (\d+)개', message)
                if match:
                    count = match.group(1)
                    self.status_label.config(text=f"{count}건 매물 검색 완료")
            elif "엑셀 파일" in message and "생성 완료" in message:
                self.status_label.config(text="파일 저장 완료")
        
    def open_settings(self):
        """설정 창 열기"""
        # 폴더 선택 대화상자
        path = filedialog.askdirectory(initialdir=self.save_path, title="저장할 폴더 선택")
        
        if path:
            self.save_path = path
            self.save_config()
            self.log(f"저장 경로가 변경되었습니다: {path}")

    
    # log 메서드를 스레드 안전하게 변경
    def log(self, message):
        """스레드 안전한 로깅"""
        # 로그 큐에 메시지 추가
        self.log_queue.put(message)
        # 나머지 코드 제거

    
    def start_search(self):
        """단지명으로 검색 시작 (스레드로 실행)"""
        search_keyword = self.search_entry.get().strip()
        if not search_keyword:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        self.search_button.config(state=tk.DISABLED)
        threading.Thread(target=self.search_complex, args=(search_keyword,), daemon=True).start()
    
    def search_complex(self, search_keyword):
        """단지명으로 검색 및 단지번호 찾기"""
        try:
            self.log(f"'{search_keyword}' 단지 검색을 시작합니다...")
            
            # Chrome 옵션 설정
            chrome_options = Options()
            chrome_options.add_argument("--headless")  # 헤드리스 모드 (화면 표시 없음)
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
            
            self.log("웹 브라우저를 초기화하는 중...")

            # driver 초기화 부분을 다음과 같이 수정 (search_complex 메서드와 search_complex_detail 메서드 모두)
            # 기존 코드:
            # self.driver = webdriver.Chrome(options=chrome_options)
            
            # 수정할 코드:
            service = Service(ChromeDriverManager().install())
            self.driver = webdriver.Chrome(service=service, options=chrome_options)
            
            # 네이버 부동산 접속
            self.log("네이버 부동산에 접속 중...")
            self.driver.get("https://fin.land.naver.com/")
            time.sleep(2)
            
            # 검색 버튼 클릭
            self.log("검색 버튼 클릭 중...")
            search_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "svg[viewBox='0 0 24 24']"))
            )
            search_button.click()
            time.sleep(1)
            
            # 검색창에 단지명 입력
            self.log(f"검색창에 '{search_keyword}' 입력 중...")
            search_input = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "query"))
            )
            search_input.clear()
            search_input.send_keys(search_keyword)
            search_input.send_keys(Keys.ENTER)
            time.sleep(3)
            
            # 검색 결과 확인
            complex_list = []

            # 페이지 소스 확인 (디버깅용)
            self.log("현재 페이지 HTML 구조 확인 중...")
            page_source = self.driver.page_source
            if "keyword_auto" in page_source:
                self.log("자동완성 결과 영역을 감지했습니다.")
            if "_keywordAutoCompleteUl" in page_source:
                self.log("자동완성 리스트를 감지했습니다.")
            
            # 자동완성 영역 찾기 시도
            try:
                auto_complete = self.driver.find_element(By.ID, "keyword_auto")
                self.log(f"자동완성 영역 표시 상태: {auto_complete.get_attribute('style')}")
            except:
                self.log("자동완성 영역을 찾을 수 없습니다.")
            
      
           
            try:
                # 기존 자동완성 코드 모두 제거 후, 아래 코드로 대체
                
                # 검색 결과 목록 찾기
                self.log("검색 결과 목록을 찾는 중...")
                time.sleep(2)  # 검색 결과가 로드될 시간 확보
                
                # 복합 단지 목록 찾기
                complex_items = self.driver.find_elements(By.CSS_SELECTOR, "#complex_list_ul .result_item")
                
                if complex_items and len(complex_items) > 0:
                    self.log(f"{len(complex_items)}개의 단지 검색 결과를 찾았습니다.")
                    
                    # 검색 결과 목록에서 단지 정보 추출
                    for item in complex_items:
                        try:
                            # a 태그에서 href 속성으로 단지번호 추출
                            link_element = item.find_element(By.CSS_SELECTOR, "a.inner")
                            href = link_element.get_attribute("href")
                            
                            # href에서 단지번호 추출 (/complex/info/22746 형식)
                            complex_id = ""
                            id_match = re.search(r"/complex/info/(\d+)", href)
                            if id_match:
                                complex_id = id_match.group(1)
                            
                            # 단지명 추출
                            name_element = item.find_element(By.CSS_SELECTOR, "span.keyword")
                            complex_name = name_element.text.strip()
                            
                            # 주소 추출
                            address_element = item.find_element(By.CSS_SELECTOR, "span.address")
                            address = address_element.text.strip()
                            
                            self.log(f"단지 발견: {complex_name} (위치: {address}, ID: {complex_id})")
                            
                            # 단지 정보 추가
                            complex_list.append({
                                "name": complex_name,
                                "address": address,
                                "id": complex_id
                            })
                            
                        except Exception as e:
                            self.log(f"검색 결과 항목 처리 중 오류: {str(e)}")
                            continue
                    
                    # 검색 결과가 여러 개면 선택 팝업 표시
                    if len(complex_list) > 1:
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 선택 팝업 표시
                        dialog = ComplexSelectionDialog(self.root, "단지 선택", complex_list)
                        selected_complex = dialog.result
                        
                        if selected_complex and selected_complex["id"]:
                            # ID가 있는 경우 바로 매물 정보 수집
                            self.complex_data = selected_complex
                            self.log(f"단지번호 {selected_complex['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                            self.download_data(selected_complex['id'], selected_complex['name'])
                        else:
                            self.log("단지 선택이 취소되었습니다.")
                        return
                    
                    elif len(complex_list) == 1 and complex_list[0]["id"]:
                        # 단일 결과이고 ID가 있는 경우
                        complex_data = complex_list[0]
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        self.complex_data = complex_data
                        self.log(f"단지번호 {complex_data['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                        self.download_data(complex_data['id'], complex_data['name'])
                        return
                
                # 검색 결과 목록이 없는 경우 (단일 단지 페이지로 이동한 경우)
                current_url = self.driver.current_url
                self.log(f"현재 URL: {current_url}")
                
                # URL에서 단지 ID 추출
                pattern1 = r"complexes/(\d+)"
                pattern2 = r"complexNumber=(\d+)"
                pattern3 = r"/complex/info/(\d+)"
                
                match1 = re.search(pattern1, current_url)
                match2 = re.search(pattern2, current_url)
                match3 = re.search(pattern3, current_url)
                
                if match1:
                    complex_id = match1.group(1)
                elif match2:
                    complex_id = match2.group(1)
                elif match3:
                    complex_id = match3.group(1)
                else:
                    complex_id = None
                
                if complex_id:
                    # 단지명 추출 시도
                    try:
                        complex_name_element = self.driver.find_element(By.CSS_SELECTOR, ".complex_title, h2.text_title, .complex_name")
                        complex_name = complex_name_element.text.strip()
                    except:
                        complex_name = search_keyword
                    
                    self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                    
                    # 단지 정보 저장
                    self.complex_data = {
                        'name': complex_name,
                        'id': complex_id
                    }
                    
                    # 브라우저 종료
                    self.driver.quit()
                    self.driver = None
                    
                    # 매물 정보 수집 시작
                    self.download_data(complex_id, complex_name)
                    return
                        
            except Exception as e:
                self.log(f"검색 결과 처리 중 오류: {str(e)}")
                
                # 단일 단지 페이지로 이동한 경우 URL에서 ID 추출
                if not complex_list:
                    current_url = self.driver.current_url
                    self.log(f"단일 단지 페이지로 이동됨: {current_url}")
                    
                    # 단지명 추출
                    try:
                        complex_name = self.driver.find_element(By.CSS_SELECTOR, ".complex_title, h2.text_title").text
                    except:
                        complex_name = search_keyword
                    
                    # URL에서 단지 ID 추출
                    pattern1 = r"complexes/(\d+)"
                    pattern2 = r"complexNumber=(\d+)"
                    
                    match1 = re.search(pattern1, current_url)
                    match2 = re.search(pattern2, current_url)
                    
                    if match1:
                        complex_id = match1.group(1)
                        self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                        
                        # 단지 정보 저장
                        self.complex_data = {
                            'name': complex_name,
                            'id': complex_id
                        }
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 매물 정보 수집 시작
                        self.download_data(complex_id, complex_name)
                        return
                    elif match2:
                        complex_id = match2.group(1)
                        self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                        
                        # 단지 정보 저장
                        self.complex_data = {
                            'name': complex_name,
                            'id': complex_id
                        }
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 매물 정보 수집 시작
                        self.download_data(complex_id, complex_name)
                        return
            except Exception as e:
                self.log(f"검색 결과 처리 중 오류: {str(e)}")
            
            # 브라우저 종료
            self.driver.quit()
            self.driver = None
            
            # 검색 결과가 있으면 팝업창 표시
            if complex_list:
                # 검색 결과가 하나만 있고 ID가 있는 경우 바로 처리
                if len(complex_list) == 1 and complex_list[0]["id"]:
                    complex_data = complex_list[0]
                    self.complex_data = complex_data
                    self.log(f"단지번호 {complex_data['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                    self.download_data(complex_data['id'], complex_data['name'])
                    return
                
                # 다중 검색 결과인 경우 선택 팝업창 표시
                dialog = ComplexSelectionDialog(self.root, "단지 선택", complex_list)
                selected_complex = dialog.result
                
                if selected_complex and selected_complex["id"]:
                    # ID가 있는 경우 바로 매물 정보 수집
                    self.complex_data = selected_complex
                    self.log(f"단지번호 {selected_complex['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                    self.download_data(selected_complex['id'], selected_complex['name'])
                elif selected_complex:
                    # ID가 없는 경우 클릭하여 상세 페이지 접속 후 ID 추출
                    self.log(f"'{selected_complex['name']}' 단지를 선택했습니다. 단지번호를 찾는 중...")
                    self.search_complex_detail(selected_complex["name"])
                else:
                    self.log("단지 선택이 취소되었습니다.")
            else:
                self.log("검색 결과가 없습니다. 다른 검색어를 입력해보세요.")
                messagebox.showwarning("경고", "검색 결과가 없습니다. 다른 검색어를 입력해보세요.")
        
        except Exception as e:
            self.log(f"오류 발생: {str(e)}")
            messagebox.showerror("오류", f"검색 중 오류가 발생했습니다.\n{str(e)}")
            if self.driver:
                self.driver.quit()
                self.driver = None
        
        finally:
            self.search_button.config(state=tk.NORMAL)

    def search_complex_detail(self, complex_name):
     
        """선택한 단지의 상세 정보를 검색"""
        try:
            # Chrome 옵션 설정
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
            
            self.log("웹 브라우저를 초기화하는 중...")
            self.driver = webdriver.Chrome(options=chrome_options)
            
            # 네이버 부동산 접속
            self.log("네이버 부동산에 접속 중...")
            self.driver.get("https://fin.land.naver.com/")
            time.sleep(2)
            
            # 검색 버튼 클릭
            self.log("검색 버튼 클릭 중...")
            search_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "svg[viewBox='0 0 24 24']"))
            )
            search_button.click()
            time.sleep(1)
            
            # 검색창에 단지명 입력
            self.log(f"검색창에 '{complex_name}' 입력 중...")
            search_input = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "query"))
            )
            search_input.clear()
            search_input.send_keys(complex_name)
            search_input.send_keys(Keys.ENTER)
            time.sleep(3)
            
            # 검색 결과 목록에서 일치하는 단지 찾기
            try:
                # 검색 결과 목록 찾기
                complex_items = self.driver.find_elements(By.CSS_SELECTOR, "#complex_list_ul .result_item")
                
                if complex_items and len(complex_items) > 0:
                    # 첫 번째 결과 클릭
                    first_item = complex_items[0]
                    link = first_item.find_element(By.CSS_SELECTOR, "a.inner")
                    
                    self.log(f"검색 결과 클릭: {link.text}")
                    link.click()
                    time.sleep(3)
                    
                    # 현재 URL에서 단지번호 추출
                    current_url = self.driver.current_url
                    self.log(f"단지 상세 페이지 URL: {current_url}")
                    
                    # URL에서 단지 ID 추출
                    pattern1 = r"complexes/(\d+)"
                    pattern2 = r"complexNumber=(\d+)"
                    pattern3 = r"/complex/info/(\d+)"
                    
                    match1 = re.search(pattern1, current_url)
                    match2 = re.search(pattern2, current_url)
                    match3 = re.search(pattern3, current_url)
                    
                    complex_id = None
                    if match1:
                        complex_id = match1.group(1)
                    elif match2:
                        complex_id = match2.group(1)
                    elif match3:
                        complex_id = match3.group(1)
                    
                    if complex_id:
                        self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                        
                        # 단지 정보 저장
                        self.complex_data = {
                            'name': complex_name,
                            'id': complex_id
                        }
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 매물 정보 수집 시작
                        self.download_data(complex_id, complex_name)
                        return
                else:
                    self.log("검색 결과 목록을 찾을 수 없습니다.")
            except Exception as e:
                self.log(f"검색 결과 클릭 중 오류: {str(e)}")
            
            # 브라우저 종료
            if self.driver:
                self.driver.quit()
                self.driver = None
            
            self.log("단지번호를 찾을 수 없습니다.")
            messagebox.showwarning("경고", "단지번호를 찾을 수 없습니다. 다른 검색어를 입력해보세요.")
        
        except Exception as e:
            self.log(f"상세 검색 중 오류 발생: {str(e)}")
            messagebox.showerror("오류", f"상세 검색 중 오류가 발생했습니다.\n{str(e)}")
            if self.driver:
                self.driver.quit()
                self.driver = None
            
            self.log("단지번호를 찾을 수 없습니다.")
            messagebox.showwarning("경고", "단지번호를 찾을 수 없습니다. 다른 검색어를 입력해보세요.")
        
        except Exception as e:
            self.log(f"상세 검색 중 오류 발생: {str(e)}")
            messagebox.showerror("오류", f"상세 검색 중 오류가 발생했습니다.\n{str(e)}")
            if self.driver:
                self.driver.quit()
                self.driver = None
    
    def download_data(self, complex_number, complex_name):
        """선택한 단지의 매물 정보 다운로드 실행"""
        self.log(f"'{complex_name}' 단지(번호: {complex_number})의 매물 정보 수집을 시작합니다.")
        
        # # 프로그레스 바 초기화
        # self.setup_progress_bar()
        
        # 스레드 작업 함수
        def download_worker():
            try:
                headers = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
                }
                
                all_properties = []
                
                # 첫 페이지 요청
                url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page=0"
                        
                self.log("첫 번째 페이지 데이터 요청 중...")
                
                try:
                    response = requests.get(url, headers=headers)
                    
                    if response.status_code != 200:
                        self.log(f"API 요청 실패: 상태 코드 {response.status_code}")
                        self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
                        return
                    
                    data = response.json()
                    
                    if 'result' not in data or 'list' not in data['result']:
                        self.log("데이터 구조가 예상과 다릅니다.")
                        self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
                        return
                    
                    property_list = data['result']['list']
                    
                    if not property_list:
                        self.log("매물 정보가 없습니다.")
                        self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
                        return
                    
                    # 첫 페이지 데이터 처리
                    self.log(f"페이지 0에서 {len(property_list)}개의 매물 정보를 찾았습니다.")
                    for item in property_list:
                        property_data = self.extract_property_data(item, 0)
                        all_properties.append(property_data)
                    
                    # 다음 페이지 여부 확인
                    has_next_page = data['result'].get('hasNextPage', False)
                    
                    if has_next_page:
                        # 스레드 수를 5로 고정 (self.thread_var.get() 대신 5 사용)
                        max_threads = 5
                        self.log(f"다음 페이지 존재. {max_threads}개의 스레드로 병렬 처리를 시작합니다.")
                                      
                        # 향상된 배치 처리 방식 적용
                        # 작은 배치로 나누어 실행하고, 빈 결과가 있으면 중단
                        batch_size = 5  # 한 번에 처리할 페이지 수
                        page = 1
                        stop_processing = False
                        
                        while not stop_processing:
                            batch_end = page + batch_size
                            self.log(f"페이지 {page}~{batch_end-1} 배치 처리 중...")
                            
                            # 배치 내 페이지 병렬 처리
                            with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
                                future_to_page = {executor.submit(self.fetch_page, complex_number, p, headers): p 
                                                 for p in range(page, batch_end)}
                                
                                # 배치 내 페이지 결과 확인
                                empty_pages = 0
                                processed_pages = []
                                
                                for future in concurrent.futures.as_completed(future_to_page):
                                    page_num = future_to_page[future]
                                    processed_pages.append(page_num)
                                    
                                    try:
                                        page_properties = future.result()
                                        
                                        if page_properties:
                                            self.log(f"페이지 {page_num}에서 {len(page_properties)}개의 매물 정보를 찾았습니다.")
                                            all_properties.extend(page_properties)
                                            self.update_progress(len(all_properties))
                                        else:
                                            self.log(f"페이지 {page_num}에서 데이터가 없습니다.")
                                            empty_pages += 1
                                    except Exception as exc:
                                        self.log(f"페이지 {page_num} 처리 중 오류: {str(exc)}")
                                        empty_pages += 1
                                
                                # 배치의 절반 이상이 빈 페이지면 중단
                                if empty_pages >= batch_size * 0.5:
                                    self.log(f"빈 페이지가 많아 처리를 중단합니다.")
                                    stop_processing = True
                                    break
                                
                                # 다음 배치 시작 페이지 설정
                                if processed_pages:
                                    page = max(processed_pages) + 1
                                else:
                                    page = batch_end
                    
                    self.log(f"총 {len(all_properties)}개의 매물 정보를 수집했습니다.")
                    
                except Exception as e:
                    self.log(f"데이터 요청 중 오류: {str(e)}")
                
                # 결과 처리 (메인 스레드에서 안전하게 처리하기 위해 after 사용)
                self.root.after(0, lambda: self.process_results(all_properties, complex_name))
                
            except Exception as e:
                self.log(f"데이터 수집 중 오류 발생: {str(e)}")
                # 에러 메시지는 메인 스레드에서 표시
                self.root.after(0, lambda: messagebox.showerror("오류", f"데이터 수집 중 오류가 발생했습니다.\n{str(e)}"))
                
            finally:
                # 버튼 상태 복원은 메인 스레드에서 처리
                self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
        
        # 작업을 별도 스레드로 실행
        worker_thread = threading.Thread(target=download_worker, daemon=True)
        worker_thread.start()

    
    def process_results(self, all_properties, complex_name):
        """수집된 데이터 처리 (메인 스레드에서 실행)"""
        try:
            # self.complete_progress()
            
            if not all_properties:
                self.log("수집된 매물 정보가 없습니다.")
                messagebox.showinfo("알림", "수집된 매물 정보가 없습니다.")
                return
            
            # 데이터프레임 생성
            df = pd.DataFrame(all_properties)
            
            # 거래유형 변환
            trade_type_map = {
                'A1': '매매',
                'B1': '전세',
                'B2': '월세',
                'B3': '단기임대'
            }
            df['거래유형'] = df['거래유형'].map(lambda x: trade_type_map.get(x, x))
            
            # 방향 변환
            direction_map = {
                'SS': '남향',
                'SE': '남동향',
                'SW': '남서향',
                'EE': '동향',
                'WW': '서향',
                'NN': '북향',
                'NE': '북동향',
                'NW': '북서향'
            }
            df['방향'] = df['방향'].map(lambda x: direction_map.get(x, x))
            
            # 가격변동상태 변환 추가
            price_change_map = {
                0: '변동없음',
                1: '가격상승',
                -1: '가격하락'
            }
            df['가격변동상태'] = df['가격변동상태'].map(lambda x: price_change_map.get(x, '알수없음'))
            
            # 금액 데이터 형식 변환 (프리미엄 추가)
            money_columns = ['보증금', '월세', '매매가', '프리미엄', '최소매매가', '최대매매가', 
                             '최소보증금', '최대보증금', '최소월세', '최대월세', '최소프리미엄', '최대프리미엄']
            
            for col in money_columns:
                if col in df.columns:
                    df[col] = df[col].apply(lambda x: f"{x/10000:.0f}" if x > 0 else '')
            
            # 날짜 형식화
            today = datetime.now().strftime('%Y%m%d')
            
            # 엑셀 파일로 저장
            excel_filename = os.path.join(self.save_path, f'{complex_name}_매물정보_{today}.xlsx')
            
            # 여러 시트로 저장 (오픈파이썬 엔진 사용하여 필터 기능 적용)
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                # 전체 매물 시트
                df.to_excel(writer, sheet_name='전체매물', index=False)
                self.apply_filter_to_sheet(writer.sheets['전체매물'], len(df.columns))
                
                # 매매 매물만 필터링 (query 사용)
                deal_properties = df.query('매매가 != "" and 매매가.notnull()')
                if not deal_properties.empty:
                    deal_properties.to_excel(writer, sheet_name='매매매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['매매매물'], len(deal_properties.columns))
                
                # 전세 매물만 필터링 (query 사용)
                full_deposit = df.query('보증금 != "" and 보증금.notnull() and (월세 == "" or 월세.isnull())')
                if not full_deposit.empty:
                    full_deposit.to_excel(writer, sheet_name='전세매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['전세매물'], len(full_deposit.columns))
                
                # 월세 매물만 필터링 (query 사용)
                monthly_rent = df.query('월세 != "" and 월세.notnull()')
                if not monthly_rent.empty:
                    monthly_rent.to_excel(writer, sheet_name='월세매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['월세매물'], len(monthly_rent.columns))
                
                # 중개사 수별 매물 필터링 (query 사용)
                multi_realtor = df.query('`중개사 수` > 1')
                if not multi_realtor.empty:
                    multi_realtor.to_excel(writer, sheet_name='중복매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['중복매물'], len(multi_realtor.columns))
                
                # 열 너비 자동 조정 (모든 시트)
                for sheet_name in writer.sheets:
                    sheet = writer.sheets[sheet_name]
                    self.adjust_column_width(sheet)
            
            self.log(f"엑셀 파일 '{excel_filename}' 생성 완료!")
            self.log(f"총 {len(df)}개의 매물 정보가 저장되었습니다.")
            
            # 저장 완료 메시지 표시
            messagebox.showinfo("완료", f"'{complex_name}' 단지의 매물 정보가 '{excel_filename}' 파일로 저장되었습니다.")
        
        except Exception as e:
            self.log(f"결과 처리 중 오류: {str(e)}")
            messagebox.showerror("오류", f"결과 처리 중 오류가 발생했습니다.\n{str(e)}")
    
    def apply_filter_to_sheet(self, sheet, columns_count):
        """시트의 첫 행에 필터 적용"""
        try:
            # openpyxl 라이브러리가 설치되어 있어야 함
            # 첫 행에 필터 적용
            sheet.auto_filter.ref = f"A1:{chr(64 + columns_count)}1"
            
            # 첫 행(헤더) 스타일 설정
            from openpyxl.styles import Font, PatternFill, Alignment
            
            header_font = Font(bold=True, size=11)
            header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
            
            for cell in sheet[1]:
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = Alignment(horizontal='center', vertical='center')
        except Exception as e:
            self.log(f"필터 적용 중 오류: {str(e)}")
    
    def adjust_column_width(self, sheet):
        """열 너비 자동 조정"""
        try:
            for column in sheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                ㄴ
                for cell in column:
                    if cell.value:
                        # 셀 값의 길이 계산
                        try:
                            cell_length = len(str(cell.value))
                            max_length = max(max_length, cell_length)
                        except:
                            pass
                
                # 최소 너비 설정
                max_length = max(max_length, 10)
                # 최대 너비 제한
                max_length = min(max_length, 50)
                
                # 열 너비 설정 (약간의 여유 공간 추가)
                sheet.column_dimensions[column_letter].width = max_length + 2
        except Exception as e:
            self.log(f"열 너비 조정 중 오류: {str(e)}")

        

    def fetch_page(self, complex_number, page, headers):
        """페이지별 데이터 요청 (병렬 처리용)"""
        url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
                
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                return None
            
            data = response.json()
            
            if 'result' in data and 'list' in data['result']:
                property_list = data['result']['list']
                
                if not property_list:
                    return None
                
                # 매물 정보 추출
                page_properties = []
                for item in property_list:
                    property_data = self.extract_property_data(item, page)
                    page_properties.append(property_data)
                
                return page_properties
        except Exception as e:
            self.log(f"페이지 {page} 요청 오류: {str(e)}")
            return None
        
        return None
    
    def extract_property_data(self, item, page):
        """매물 항목에서 필요한 정보 추출"""
        rep_info = item['representativeArticleInfo']
        
        # 기존 매물 정보 추출 코드에 프리미엄 가격 추가
        property_data = {
            '단지명': rep_info.get('complexName', ''),
            '동': rep_info.get('dongName', ''),
            '거래유형': rep_info.get('tradeType', ''),
            '전용면적': rep_info['spaceInfo'].get('exclusiveSpace', ''),
            '타입구분': rep_info['spaceInfo'].get('nameType', ''),
            '층/전체층': rep_info['articleDetail'].get('floorInfo', ''),
            '방향': rep_info['articleDetail'].get('direction', ''),
            '매물특징': rep_info['articleDetail'].get('articleFeatureDescription', ''),
            '보증금': rep_info['priceInfo'].get('warrantyPrice', 0),
            '월세': rep_info['priceInfo'].get('rentPrice', 0),
            '매매가': rep_info['priceInfo'].get('dealPrice', 0),
            '프리미엄': rep_info['priceInfo'].get('premiumPrice', 0),  # 프리미엄 가격 추가
            '가격변동상태': rep_info['priceInfo'].get('priceChangeStatus', 0),
            '중개사명': rep_info['brokerInfo'].get('brokerageName', ''),
            'VR노출여부': rep_info['articleMediaDto'].get('isVrExposed', False) if rep_info.get('articleMediaDto') else False,
            '중개사 수': 0,
            '최소매매가': 0,
            '최대매매가': 0,
            '최소보증금': 0,
            '최대보증금': 0,
            '최소월세': 0,
            '최대월세': 0,
            '최소프리미엄': 0,  # 최소 프리미엄 추가
            '최대프리미엄': 0,  # 최대 프리미엄 추가
            '페이지번호': page
        }
        
        # 중복 매물 정보 처리
        if 'duplicatedArticlesInfo' in item and item['duplicatedArticlesInfo'] is not None:
            # 중개사 수 추가
            property_data['중개사 수'] = item['duplicatedArticlesInfo'].get('realtorCount', 0)
            
            # 중복 매물 가격 정보
            if 'representativePriceInfo' in item['duplicatedArticlesInfo']:
                price_info = item['duplicatedArticlesInfo']['representativePriceInfo']
                
                if 'dealPrice' in price_info:
                    property_data['최소매매가'] = price_info['dealPrice'].get('minPrice', 0)
                    property_data['최대매매가'] = price_info['dealPrice'].get('maxPrice', 0)
                
                if 'warrantyPrice' in price_info:
                    property_data['최소보증금'] = price_info['warrantyPrice'].get('minPrice', 0)
                    property_data['최대보증금'] = price_info['warrantyPrice'].get('maxPrice', 0)
                
                if 'rentPrice' in price_info:
                    property_data['최소월세'] = price_info['rentPrice'].get('minPrice', 0)
                    property_data['최대월세'] = price_info['rentPrice'].get('maxPrice', 0)
                
                # 프리미엄 최소/최대 가격 추가
                if 'premiumPrice' in price_info:
                    property_data['최소프리미엄'] = price_info['premiumPrice'].get('minPrice', 0)
                    property_data['최대프리미엄'] = price_info['premiumPrice'].get('maxPrice', 0)
        
        return property_data

    def setup_progress_bar(self):
        """프로그레스 바 초기화"""
        # # 프로그레스 바가 이미 존재하면 제거
        # if hasattr(self, 'progress_frame'):
        #     self.progress_frame.destroy()
        
        # # 프레임 생성
        # self.progress_frame = ttk.Frame(self.root)
        # self.progress_frame.pack(fill="x", padx=10, pady=5)
        
        # # 레이블
        # self.progress_label = ttk.Label(self.progress_frame, text="데이터 수집 중...")
        # self.progress_label.pack(anchor="w", pady=(0, 5))
        
        # # 프로그레스 바
        # self.progress_bar = ttk.Progressbar(self.progress_frame, orient="horizontal", mode="indeterminate", length=300)
        # self.progress_bar.pack(fill="x")
        # self.progress_bar.start(10)
        
        # # 항목 카운터 레이블
        # self.count_label = ttk.Label(self.progress_frame, text="0개 매물 수집됨")
        # self.count_label.pack(anchor="e", pady=(5, 0))
        
        # # UI 업데이트
        # self.root.update_idletasks()
        pass
    
    def update_progress(self, count):
        """프로그레스 상태 업데이트 (스레드 안전)"""
        # def _update_progress():
        #     if hasattr(self, 'count_label'):
        #         self.count_label.config(text=f"{count}개 매물 수집됨")
        #         self.root.update_idletasks()
        
        # # 메인 스레드에서 실행
        # if threading.current_thread() is threading.main_thread():
        #     _update_progress()
        # else:
        #     # 다른 스레드에서는 after 메서드를 사용
        #     self.root.after(0, _update_progress)
        # 로그에만 정보 출력
        self.log(f"현재까지 {count}개 매물 수집됨")
    
    def complete_progress(self):
        """프로그레스 바 완료 처리 (스레드 안전)"""
        # def _complete_progress():
        #     if hasattr(self, 'progress_bar'):
        #         self.progress_bar.stop()
        #         self.progress_bar.configure(mode="determinate", value=100)
        #         self.progress_label.config(text="데이터 수집 완료")
        #         self.root.update_idletasks()
        
        # # 메인 스레드에서 실행
        # if threading.current_thread() is threading.main_thread():
        #     _complete_progress()
        # else:
        #     # 다른 스레드에서는 after 메서드를 사용
        #     self.root.after(0, _complete_progress)
        pass

# 메인 함수
def main():
    
    # 아직 유효한 경우 프로그램 실행
    root = tk.Tk()
    app = NaverRealEstateApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

'사가정센트럴아이파크' 단지 검색을 시작합니다...
웹 브라우저를 초기화하는 중...
네이버 부동산에 접속 중...
검색 버튼 클릭 중...
검색창에 '사가정센트럴아이파크' 입력 중...
현재 페이지 HTML 구조 확인 중...
자동완성 영역을 찾을 수 없습니다.
검색 결과 목록을 찾는 중...
현재 URL: https://fin.land.naver.com/complexes/119588?tab=transaction&transactionTradeType=A1&transactionPyeongTypeNumber=1
URL에서 단지번호를 찾았습니다: 119588
'사가정센트럴아이파크' 단지(번호: 119588)의 매물 정보 수집을 시작합니다.
첫 번째 페이지 데이터 요청 중...
페이지 0에서 30개의 매물 정보를 찾았습니다.
다음 페이지 존재. 5개의 스레드로 병렬 처리를 시작합니다.
페이지 1~5 배치 처리 중...
페이지 5에서 데이터가 없습니다.
페이지 3에서 30개의 매물 정보를 찾았습니다.
현재까지 60개 매물 수집됨
페이지 4에서 24개의 매물 정보를 찾았습니다.
현재까지 84개 매물 수집됨
페이지 2에서 30개의 매물 정보를 찾았습니다.
현재까지 114개 매물 수집됨
페이지 1에서 30개의 매물 정보를 찾았습니다.
현재까지 144개 매물 수집됨
페이지 6~10 배치 처리 중...
페이지 10에서 데이터가 없습니다.
페이지 8에서 데이터가 없습니다.
페이지 9에서 데이터가 없습니다.
페이지 6에서 데이터가 없습니다.
페이지 7에서 데이터가 없습니다.
빈 페이지가 많아 처리를 중단합니다.
총 144개의 매물 정보를 수집했습니다.
열 너비 조정 중 오류: name 'ᄂ' is not defined
열 너비 조정 중 오류: name 'ᄂ' is not defined
열 너비 조정 중 오류: name 'ᄂ' is not defined
열 너비 조정 중 오류: name 'ᄂ' is not defined
열 너비 조정 중 오류: name 'ᄂ' is not defi

In [2]:
import requests
import pandas as pd
import json
import time
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import threading
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import re
from tkinter import filedialog
import configparser
import os
import concurrent.futures
import queue
import webbrowser  # 이 코드를 다른 import문 근처에 추가
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment

class ComplexSelectionDialog:
    def __init__(self, parent, title, complex_list):
        self.result = None
        
        # 대화상자 생성
        self.dialog = tk.Toplevel(parent)
        self.dialog.title(title)
        self.dialog.geometry("700x500")  # 크기를 더 크게 설정 (원래 600x400)
        self.dialog.minsize(700, 500)    # 최소 크기 설정
        self.dialog.transient(parent)
        self.dialog.grab_set()
    
        # 프레임 생성
        frame = ttk.Frame(self.dialog, padding="10")
        frame.pack(fill=tk.BOTH, expand=True)
        
        # 라벨 생성
        ttk.Label(frame, text="검색된 단지 목록. 선택하세요:").pack(anchor="w", pady=(0, 5))
        
        # 트리뷰 생성
        columns = ("name", "address")
        self.tree = ttk.Treeview(frame, columns=columns, show="headings", height=15)
        self.tree.heading("name", text="단지명")
        self.tree.heading("address", text="주소")
        
        self.tree.column("name", width=250)
        self.tree.column("address", width=400)
        
        # 스크롤바 생성
        scrollbar = ttk.Scrollbar(frame, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        
        self.tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # 데이터 추가
        for item in complex_list:
            self.tree.insert("", "end", values=(item["name"], item["address"]), tags=(item["id"],))
        
        # 버튼 프레임 - 패딩 늘림
        button_frame = ttk.Frame(self.dialog, padding="20")  # 패딩 값 증가 (원래 10)
        button_frame.pack(fill="x", pady=10)  # 상하 여백 추가
        
        # 확인 버튼
        select_button = ttk.Button(button_frame, text="선택", command=self.on_select, width=10)  # 버튼 너비 명시
        select_button.pack(side="right", padx=10)  # 여백 증가
        
        # 취소 버튼
        cancel_button = ttk.Button(button_frame, text="취소", command=self.on_cancel, width=10)  # 버튼 너비 명시
        cancel_button.pack(side="right", padx=10)  # 여백 증가
        
        # 더블 클릭 이벤트 바인딩
        self.tree.bind("<Double-1>", lambda e: self.on_select())
        
        # 창 중앙 배치
        self.center_dialog()
        
        # 대화상자가 닫힐 때까지 대기
        parent.wait_window(self.dialog)
    
    def center_dialog(self):
        """대화상자를 화면 중앙에 배치"""
        self.dialog.update_idletasks()
        
        # 화면 크기 가져오기
        screen_width = self.dialog.winfo_screenwidth()
        screen_height = self.dialog.winfo_screenheight()
        
        # 창 크기 가져오기
        dialog_width = self.dialog.winfo_width()
        dialog_height = self.dialog.winfo_height()
        
        # 화면 중앙 위치 계산
        x = (screen_width - dialog_width) // 2
        y = (screen_height - dialog_height) // 2
        
        # 위치 설정
        self.dialog.geometry(f"+{x}+{y}")
    
    def on_select(self):
        selected_items = self.tree.selection()
        if selected_items:
            item_id = selected_items[0]
            item_values = self.tree.item(item_id)['values']
            item_tags = self.tree.item(item_id)['tags']
            
            self.result = {
                "name": item_values[0],
                "address": item_values[1],
                "id": item_tags[0]
            }
            
        self.dialog.destroy()
    
    def on_cancel(self):
        self.dialog.destroy()

class NaverRealEstateApp:
# NaverRealEstateApp 클래스의 __init__ 메서드에 log_queue 초기화 코드 추가
    def __init__(self, root):
        self.root = root
        self.root.title("네이버 부동산 매물 수집기")
        self.root.geometry("600x450")  # 창 크기 확장
        self.root.resizable(True, True)
        
        # 변수 초기화
        self.complex_data = None
        self.driver = None
        
        # 로그 큐 초기화 추가
        self.log_queue = queue.Queue()
        
        # 설정 관리
        self.config = configparser.ConfigParser()
        self.save_path = os.path.expanduser("~/Documents")  # 기본 저장 경로
        self.load_config()
        
        # 탭 컨트롤 생성
        self.tab_control = ttk.Notebook(self.root)
        
        # 탭 1: 단일 단지 검색 (기존 기능)
        self.tab1 = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab1, text='단일 단지 검색')
        
        # 탭 2: 다중 단지 검색 (새 기능)
        self.tab2 = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab2, text='다중 단지 검색')
        
        self.tab_control.pack(expand=1, fill="both")
        
        # 단일 단지 검색 UI 설정
        self.setup_single_search_ui()
        
        # 다중 단지 검색 UI 설정
        self.setup_multi_search_ui()
        
        # 상태 표시 레이블
        self.status_label = tk.Label(
            self.root, 
            text="단지명을 입력하고 검색하세요", 
            font=("맑은 고딕", 9),
            pady=15
        )
        self.status_label.pack(fill="x")
        
        # 제작자 정보 프레임
        self.setup_author_info()
        
        # 로그 큐 처리 시작
        self.root.after(100, self.process_log_queue)


    def setup_single_search_ui(self):
        """단일 단지 검색 UI 설정 (기존 기능)"""
        # 프레임 구성
        search_frame = ttk.LabelFrame(self.tab1, text="아파트 단지 검색")
        search_frame.pack(fill="x", padx=10, pady=5)
        
        # 검색 프레임 구성
        ttk.Label(search_frame, text="단지명:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.search_entry = ttk.Entry(search_frame, width=40)
        self.search_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.search_entry.bind("<Return>", lambda event: self.start_search())
        
        self.search_button = ttk.Button(search_frame, text="검색 및 매물 수집", command=self.start_search)
        self.search_button.grid(row=0, column=2, padx=5, pady=5)
        
        # 설정 버튼 추가
        self.settings_button = ttk.Button(search_frame, text="⚙", width=3, command=self.open_settings)
        self.settings_button.grid(row=0, column=3, padx=5, pady=5)
        
        search_frame.columnconfigure(1, weight=1)


    def setup_author_info(self):
        """제작자 정보 프레임 설정"""
        author_frame = ttk.Frame(self.root)
        author_frame.pack(side="bottom", fill="x", padx=10, pady=10)
        
        # 제작자 이름 (클릭 가능한 링크)
        author_label = tk.Label(author_frame, text="만든이 부태리", font=("맑은 고딕", 9, "bold"), fg="black", cursor="hand2")
        author_label.pack(anchor="center")
        author_label.bind("<Button-1>", lambda e: self.open_blog())
        
        # 블로그 주소
        blog_label = tk.Label(author_frame, text="https://blog.naver.com/landlover333", font=("맑은 고딕", 8))
        blog_label.pack(anchor="center")
        blog_label.bind("<Button-1>", lambda e: self.open_blog())

    def setup_multi_search_ui(self):
        """다중 단지 검색 UI 설정 (새 기능)"""
        # 메인 프레임
        main_frame = ttk.Frame(self.tab2, padding=10)
        main_frame.pack(fill="both", expand=True)
        
        # 왼쪽 영역 (단지 입력)
        left_frame = ttk.LabelFrame(main_frame, text="단지 목록 입력 (최대 50개)")  # 25개 → 50개로 변경
        left_frame.pack(side="left", fill="both", expand=True, padx=(0, 5))
        
        # 단지 입력용 Text 위젯
        ttk.Label(left_frame, text="단지명을 한 줄에 하나씩 입력하세요:").pack(anchor="w", pady=(5, 0))
        self.complex_text = scrolledtext.ScrolledText(left_frame, width=30, height=15)
        self.complex_text.pack(fill="both", expand=True, pady=5)
        
        # 오른쪽 영역 (옵션 설정)
        right_frame = ttk.LabelFrame(main_frame, text="검색 옵션")
        right_frame.pack(side="right", fill="both", padx=(5, 0))
        
        # 전용면적 범위 설정
        ttk.Label(right_frame, text="전용면적 범위:").grid(row=0, column=0, sticky="w", pady=5)
        
        area_frame = ttk.Frame(right_frame)
        area_frame.grid(row=0, column=1, sticky="ew", pady=5)
        
        self.min_area_var = tk.StringVar()
        self.max_area_var = tk.StringVar()
        
        ttk.Entry(area_frame, textvariable=self.min_area_var, width=6).pack(side="left")
        ttk.Label(area_frame, text="㎡ ~").pack(side="left", padx=2)
        ttk.Entry(area_frame, textvariable=self.max_area_var, width=6).pack(side="left")
        ttk.Label(area_frame, text="㎡").pack(side="left", padx=2)
        
        # 층수 조건 정보 (정보만 표시)
        ttk.Label(right_frame, text="층수 조건:").grid(row=1, column=0, sticky="w", pady=5)
        ttk.Label(right_frame, text="중층/고층/5층 이상 매물만 검색").grid(row=1, column=1, sticky="w", pady=5)
        
        # 거래 유형 정보 (정보만 표시)
        ttk.Label(right_frame, text="거래 유형:").grid(row=2, column=0, sticky="w", pady=5)
        ttk.Label(right_frame, text="매매 및 전세 최저가 조사").grid(row=2, column=1, sticky="w", pady=5)
        
        # 버튼 영역
        button_frame = ttk.Frame(right_frame)
        button_frame.grid(row=3, column=0, columnspan=2, pady=10)
        
        self.multi_search_button = ttk.Button(button_frame, text="다중 검색 시작", command=self.start_multi_search, width=20)
        self.multi_search_button.pack(pady=5)
        
        # 진행상황 표시
        self.progress_text = scrolledtext.ScrolledText(right_frame, width=30, height=10, state='disabled')
        self.progress_text.grid(row=4, column=0, columnspan=2, sticky="ew", pady=5)

        
    # 2. start_multi_search 메서드의 제한 및 메시지 수정
    def start_multi_search(self):
        """다중 단지 검색 시작"""
        # 입력 텍스트에서 단지명 리스트 추출
        complex_text = self.complex_text.get('1.0', tk.END).strip()
        complex_list = [name.strip() for name in complex_text.split('\n') if name.strip()]
        
        # 최대 50개로 제한 (25개 → 50개로 변경)
        if len(complex_list) > 50:
            messagebox.showwarning("경고", "최대 50개까지만 검색 가능합니다. 처음 50개만 처리합니다.")  # 메시지도 수정
            complex_list = complex_list[:50]  # 25 → 50으로 변경
        
        if not complex_list:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        # 전용면적 범위 확인
        min_area = self.min_area_var.get().strip()
        max_area = self.max_area_var.get().strip()
        
        try:
            min_area = float(min_area) if min_area else None
            max_area = float(max_area) if max_area else None
        except ValueError:
            messagebox.showwarning("경고", "전용면적은 숫자로 입력하세요.")
            return
        
        # 검색 옵션 저장
        search_options = {
            'min_area': min_area,
            'max_area': max_area
        }
        
        # 진행 상황 초기화
        self.clear_progress_text()
        self.append_progress_text(f"총 {len(complex_list)}개 단지 검색을 시작합니다.\n")
        
        # 검색 버튼 비활성화
        self.multi_search_button.config(state=tk.DISABLED)
        
        # 별도 스레드로 검색 시작
        threading.Thread(target=self.process_multi_search, 
                         args=(complex_list, search_options), 
                         daemon=True).start()
    
    def process_multi_search(self, complex_list, search_options):
        """다중 단지 검색 처리 (별도 스레드에서 실행)"""
        try:
            # 결과 저장용 리스트
            all_results = []        # 모든 단지의 결과
            summary_results = []    # 최저가 요약 결과
            
            # 각 단지별로 처리
            for idx, complex_name in enumerate(complex_list):
                self.append_progress_text(f"[{idx+1}/{len(complex_list)}] '{complex_name}' 검색 중...")
                
                # 단지 정보 검색
                complex_id = self.find_complex_id(complex_name)
                
                if not complex_id:
                    self.append_progress_text(f"  - 단지를 찾을 수 없습니다.\n")
                    continue
                
                self.append_progress_text(f"  - 단지번호: {complex_id}")
                
                # 매물 정보 수집
                property_data = self.collect_property_data(complex_id, complex_name, search_options)
                
                if not property_data:
                    self.append_progress_text(f"  - 매물 정보가 없습니다.\n")
                    continue
                
                # 데이터프레임으로 변환
                df = pd.DataFrame(property_data)
                
                # 전체 매물 저장 (각 단지별 전체 결과 저장용)
                all_results.append({
                    'complex_name': complex_name,
                    'complex_id': complex_id,
                    'properties': df
                })
                
                # === 최저가 찾기 (pandas 활용) ===
                
                # 1. 전용면적 필터링
                if search_options['min_area'] or search_options['max_area']:
                    area_conditions = []
                    if search_options['min_area']:
                        area_conditions.append(f"전용면적 >= {search_options['min_area']}")
                    if search_options['max_area']:
                        area_conditions.append(f"전용면적 <= {search_options['max_area']}")
                    
                    area_query = " and ".join(area_conditions)
                    try:
                        df = df.query(area_query)
                        self.append_progress_text(f"  - 전용면적 필터링: {area_query}")
                    except Exception as e:
                        self.append_progress_text(f"  - 전용면적 필터링 오류: {str(e)}")
                
                # 2. 층수 필터링 (저층 제외, 5층 이상 포함)
                # 2. 층수 필터링 (저층 및 1~4층 제외)
                try:
                    # copy()를 사용하여 명시적 복사본 생성
                    df = df.copy()
                    df_original = df.copy()
                    
                    def is_high_floor(floor_info):
                        # 기존과 같은 고층 판단 함수
                        floor_str = str(floor_info)
                        if pd.isna(floor_info) or floor_str.strip() == '':
                            return False
                        if '저' in floor_str:
                            return False
                        if '중' in floor_str or '고' in floor_str:
                            return True
                        parts = floor_str.split('/')
                        if len(parts) >= 1:
                            try:
                                floor_nums = re.findall(r'\d+', parts[0])
                                if floor_nums:
                                    floor_num = int(floor_nums[0])
                                    return floor_num >= 5
                            except:
                                pass
                        return False
                    
                    df['is_high_floor'] = df['층/전체층'].apply(is_high_floor)
                    df_high = df[df['is_high_floor']]
                    df_low = df[~df['is_high_floor']]
                    
                    if not df_high.empty:
                        df = df_high
                        self.append_progress_text("  - 고층(중층/5층 이상) 매물이 선택됨")
                    elif not df_low.empty:
                        df = df_low
                        self.append_progress_text("  - 고층 매물이 없어 저층/1~4층 매물 중 최저가 매물이 선택됨")
                    else:
                        self.append_progress_text("  - 조건에 맞는 매물이 없습니다.\n")
                        continue

                except Exception as e:
                    self.append_progress_text(f"  - 층수 필터링 오류: {str(e)}")
                
                # 3. 매매/전세 최저가 찾기
                # 매매 최저가
                deal_min = None
                try:
                    # 거래유형이 'A1'(매매)인 행 필터링
                    df_deal = df[df['거래유형'] == 'A1']
                    if not df_deal.empty:
                        # 매물 정보 출력 (디버깅용)
                        self.append_progress_text(f"  - [디버그] 매매 매물 수: {len(df_deal)}")
                        
                        # 매매가를 숫자형으로 변환 (문자열인 경우를 대비)
                        df_deal = df_deal.copy()  # 여기서 복사본 만들기
                        df_deal.loc[:, '매매가_숫자'] = pd.to_numeric(df_deal['매매가'], errors='coerce')
                        df_deal = df_deal.dropna(subset=['매매가_숫자'])
                        
                        if not df_deal.empty:
                            # 매매가를 기준으로 정렬
                            df_deal = df_deal.sort_values('매매가_숫자')
                            # 최저가 매물 선택
                            deal_min = df_deal.iloc[0].to_dict()
                            
                            # 가격 표시 (안전한 형변환 포함)
                            try:
                                price_val = float(deal_min['매매가'])
                                self.append_progress_text(f"  - 매매 최저가: {deal_min['동']} {deal_min['층/전체층']} {price_val/10000:.1f}만원")
                            except (ValueError, TypeError):
                                self.append_progress_text(f"  - 매매 최저가: {deal_min['동']} {deal_min['층/전체층']} (가격 변환 오류)")
                    else:
                        self.append_progress_text("  - 조건에 맞는 매매 매물이 없습니다.")
                except Exception as e:
                    self.append_progress_text(f"  - 매매 최저가 계산 오류: {str(e)}")
                
                # 전세 최저가
                jeonse_min = None
                try:
                    # 거래유형이 'B1'(전세)인 행 필터링
                    df_jeonse = df[df['거래유형'] == 'B1']
                    if not df_jeonse.empty:
                        # 매물 정보 출력 (디버깅용)
                        self.append_progress_text(f"  - [디버그] 전세 매물 수: {len(df_jeonse)}")
                        
                        # 보증금을 숫자형으로 변환 (문자열인 경우를 대비)
                        df_jeonse = df_jeonse.copy()  # 복사본 만들기
                        df_jeonse.loc[:, '보증금_숫자'] = pd.to_numeric(df_jeonse['보증금'], errors='coerce')
                        df_jeonse = df_jeonse.dropna(subset=['보증금_숫자'])
               
                        
                        if not df_jeonse.empty:
                            # 보증금을 기준으로 정렬
                            df_jeonse = df_jeonse.sort_values('보증금_숫자')
                            # 최저가 매물 선택
                            jeonse_min = df_jeonse.iloc[0].to_dict()
                            
                            # 가격 표시 (안전한 형변환 포함)
                            try:
                                price_val = float(jeonse_min['보증금'])
                                self.append_progress_text(f"  - 전세 최저가: {jeonse_min['동']} {jeonse_min['층/전체층']} {price_val/10000:.1f}만원")
                            except (ValueError, TypeError):
                                self.append_progress_text(f"  - 전세 최저가: {jeonse_min['동']} {jeonse_min['층/전체층']} (가격 변환 오류)")
                    else:
                        self.append_progress_text("  - 조건에 맞는 전세 매물이 없습니다.")
                except Exception as e:
                    self.append_progress_text(f"  - 전세 최저가 계산 오류: {str(e)}")
                
                # 4. 요약 결과에 추가
                summary = {
                    '단지명': complex_name,
                    '단지ID': complex_id,
                    '전용면적_Min': search_options.get('min_area', ''),
                    '전용면적_Max': search_options.get('max_area', '')
                }
                
                # 매매 정보 추가
                if deal_min is not None:
                    summary['매매가'] = float(deal_min['매매가']) / 10000
                    summary['매매_동'] = deal_min['동']
                    summary['매매_층'] = deal_min['층/전체층']
                    summary['매매_면적'] = deal_min['전용면적']
                else:
                    summary['매매가'] = ''
                    summary['매매_동'] = ''
                    summary['매매_층'] = ''
                    summary['매매_면적'] = ''
                
                # 전세 정보 추가
                if jeonse_min is not None:
                    summary['전세가'] = float(jeonse_min['보증금']) / 10000
                    summary['전세_동'] = jeonse_min['동']
                    summary['전세_층'] = jeonse_min['층/전체층']
                    summary['전세_면적'] = jeonse_min['전용면적']
                else:
                    summary['전세가'] = ''
                    summary['전세_동'] = ''
                    summary['전세_층'] = ''
                    summary['전세_면적'] = ''
                
                summary_results.append(summary)
                self.append_progress_text("")
            
            # 결과를 엑셀로 저장
            if summary_results:
                self.save_multi_search_results(summary_results, all_results)
            else:
                self.append_progress_text("검색 완료: 조건에 맞는 매물이 없습니다.")
                messagebox.showinfo("검색 완료", "조건에 맞는 매물이 없습니다.")
        
        except Exception as e:
            self.append_progress_text(f"오류 발생: {str(e)}")
            messagebox.showerror("오류", f"다중 검색 중 오류가 발생했습니다.\n{str(e)}")
        
        finally:
            # 검색 버튼 활성화
            self.root.after(0, lambda: self.multi_search_button.config(state=tk.NORMAL))
                
    def find_cheapest_properties(self, properties, search_options):
        """조건에 맞는 매매/전세 최저가 매물 찾기 (저층 제외)"""
        try:
            # 원본 데이터 백업
            all_properties = properties.copy()
            
            # 전용면적 필터링 (모든 매물에 적용)
            if search_options['min_area'] or search_options['max_area']:
                filtered_properties = []
                for prop in all_properties:
                    try:
                        area = float(prop.get('전용면적', 0))
                        if (search_options['min_area'] is None or area >= search_options['min_area']) and \
                           (search_options['max_area'] is None or area <= search_options['max_area']):
                            filtered_properties.append(prop)
                    except (ValueError, TypeError):
                        self.append_progress_text(f"  - 경고: 전용면적 값 '{prop.get('전용면적')}' 형식 오류")
                all_properties = filtered_properties
            
            # 거래유형 필터링 (매매와 전세만 포함)
            all_properties = [prop for prop in all_properties if prop.get('거래유형') in ['A1', 'B1']]
            
            if not all_properties:
                self.append_progress_text("  - 조건에 맞는 매물이 없습니다.")
                return None
            
            # 층수 필터링 (저층 및 1~4층 제외)
            preferred_properties = []
            excluded_properties = []
            
            for prop in all_properties:
                floor_info = str(prop.get('층/전체층', ''))
                
                # 가격 정보 표시용
                if prop.get('거래유형') == 'A1':
                    try:
                        price_val = float(prop.get('매매가', 0))
                        price_info = f"매매 {price_val/10000:.1f}만원"
                    except (ValueError, TypeError):
                        price_info = f"매매 가격 정보 오류"
                else:
                    try:
                        price_val = float(prop.get('보증금', 0))
                        price_info = f"전세 {price_val/10000:.1f}만원"
                    except (ValueError, TypeError):
                        price_info = f"전세 가격 정보 오류"
                
                # 층수 정보 없는 경우
                if not floor_info or floor_info.strip() == "":
                    excluded_properties.append(prop)
                    self.append_progress_text(f"  - 제외: 층수 정보 없음, {price_info}")
                    continue
                    
                # 강화된 저층 필터링: '저'가 포함된 모든 경우 제외 (가장 중요한 수정 부분)
                if '저' in floor_info:
                    excluded_properties.append(prop)
                    self.append_progress_text(f"  - 제외: {floor_info} (저층 포함), {price_info}")
                    continue
                
                # 중/고층 포함
                if '중' in floor_info or '고' in floor_info:
                    preferred_properties.append(prop)
                    self.append_progress_text(f"  - 포함: {floor_info} (중층/고층), {price_info}")
                    continue
                
                # 숫자로 된 층수 확인 (예: "15/30")
                try:
                    # 슬래시로 분리
                    parts = floor_info.split('/')
                    if len(parts) >= 1:
                        current_floor = parts[0].strip()
                        
                        # 현재 층이 순수 숫자인지 확인
                        if current_floor.isdigit():
                            floor_num = int(current_floor)
                            
                            # 5층 이상만 포함
                            if floor_num >= 5:
                                preferred_properties.append(prop)
                                self.append_progress_text(f"  - 포함: {floor_info} ({floor_num}층), {price_info}")
                            else:
                                excluded_properties.append(prop)
                                self.append_progress_text(f"  - 제외: {floor_info} ({floor_num}층 < 5층), {price_info}")
                        else:
                            # 숫자가 아닌 경우 제외
                            excluded_properties.append(prop)
                            self.append_progress_text(f"  - 제외: {floor_info} (층수 숫자 아님), {price_info}")
                    else:
                        # 슬래시로 분리 불가능한 경우 제외
                        excluded_properties.append(prop)
                        self.append_progress_text(f"  - 제외: {floor_info} (형식 인식 불가), {price_info}")
                except Exception as e:
                    # 파싱 오류 시 제외
                    excluded_properties.append(prop)
                    self.append_progress_text(f"  - 제외: {floor_info} (파싱 오류: {str(e)}), {price_info}")
            
            # 필터링 결과 요약
            self.append_progress_text(f"  - 총 {len(all_properties)}개 매물 중 {len(preferred_properties)}개 포함 (중층/고층/5층이상), {len(excluded_properties)}개 제외")
            
            # 필터링된 매물이 없는 경우 안내
            if not preferred_properties:
                self.append_progress_text("  - 중층/고층/5층 이상 매물이 없습니다.")
                return {}
            
            # 매매/전세 구분
            cheapest_properties = {}
            
            # 매매 최저가 찾기
            deal_properties = [p for p in preferred_properties if p.get('거래유형') == 'A1']
            if deal_properties:
                try:
                    # 추가 안전 장치: 저층 매물 다시 한번 필터링 (최종 확인)
                    safe_deal_properties = [p for p in deal_properties if '저' not in str(p.get('층/전체층', ''))]
                    
                    if not safe_deal_properties:
                        self.append_progress_text("  - 조건에 맞는 매매 매물이 없습니다.")
                    else:
                        # 디버깅용 출력: 모든 매매 매물의 층수와 가격 로깅
                        self.append_progress_text("  - [디버그] 포함된 매매 매물 목록:")
                        for idx, p in enumerate(safe_deal_properties):
                            try:
                                price_val = float(p.get('매매가', 0))
                                self.append_progress_text(f"    {idx+1}. 층: {p.get('층/전체층', '')}, 가격: {price_val/10000:.1f}만원")
                            except:
                                pass
                        
                        # 숫자 타입 확인하여 최저가 찾기
                        def get_price(prop):
                            try:
                                return float(prop.get('매매가', float('inf')))
                            except (ValueError, TypeError):
                                return float('inf')
                        
                        cheapest_deal = min(safe_deal_properties, key=get_price)
                        cheapest_properties['A1'] = cheapest_deal
                        price_val = float(cheapest_deal.get('매매가', 0))
                        self.append_progress_text(f"  - 매매 최저가: {cheapest_deal.get('층/전체층', '')} {price_val/10000:.1f}만원")
                except Exception as e:
                    self.append_progress_text(f"  - 매매 최저가 계산 중 오류: {str(e)}")
            else:
                self.append_progress_text("  - 조건에 맞는 매매 매물이 없습니다.")
            
            # 전세 최저가 찾기
            jeonse_properties = [p for p in preferred_properties if p.get('거래유형') == 'B1']
            if jeonse_properties:
                try:
                    # 추가 안전 장치: 저층 매물 다시 한번 필터링 (최종 확인)
                    safe_jeonse_properties = [p for p in jeonse_properties if '저' not in str(p.get('층/전체층', ''))]
                    
                    if not safe_jeonse_properties:
                        self.append_progress_text("  - 조건에 맞는 전세 매물이 없습니다.")
                    else:
                        # 디버깅용 출력: 모든 전세 매물의 층수와 가격 로깅
                        self.append_progress_text("  - [디버그] 포함된 전세 매물 목록:")
                        for idx, p in enumerate(safe_jeonse_properties):
                            try:
                                price_val = float(p.get('보증금', 0))
                                self.append_progress_text(f"    {idx+1}. 층: {p.get('층/전체층', '')}, 가격: {price_val/10000:.1f}만원")
                            except:
                                pass
                        
                        # 숫자 타입 확인하여 최저가 찾기
                        def get_deposit(prop):
                            try:
                                return float(prop.get('보증금', float('inf')))
                            except (ValueError, TypeError):
                                return float('inf')
                        
                        cheapest_jeonse = min(safe_jeonse_properties, key=get_deposit)
                        cheapest_properties['B1'] = cheapest_jeonse
                        price_val = float(cheapest_jeonse.get('보증금', 0))
                        self.append_progress_text(f"  - 전세 최저가: {cheapest_jeonse.get('층/전체층', '')} {price_val/10000:.1f}만원")
                except Exception as e:
                    self.append_progress_text(f"  - 전세 최저가 계산 중 오류: {str(e)}")
            else:
                self.append_progress_text("  - 조건에 맞는 전세 매물이 없습니다.")
            
            return cheapest_properties
        
        except Exception as e:
            self.append_progress_text(f"  - 최저가 매물 검색 중 오류: {str(e)}")
            import traceback
            self.append_progress_text(f"  - 상세 오류: {traceback.format_exc()}")
            return None
    def find_complex_id(self, complex_name):
        """단지명으로 단지번호 검색"""
        try:
            # Chrome 옵션 설정
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
            
            service = Service(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service, options=chrome_options)
            
            # 네이버 부동산 접속
            self.append_progress_text(f"  - '{complex_name}' 단지 검색 중...")
            driver.get("https://fin.land.naver.com/")
            time.sleep(2)
            
            # 검색 버튼 클릭
            search_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "svg[viewBox='0 0 24 24']"))
            )
            search_button.click()
            time.sleep(1)
            
            # 검색창에 단지명 입력
            search_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "query"))
            )
            search_input.clear()
            search_input.send_keys(complex_name)
            search_input.send_keys(Keys.ENTER)
            time.sleep(3)
            
            # 검색 결과 확인
            complex_id = None
            
            # 복합 단지 목록 찾기
            complex_items = driver.find_elements(By.CSS_SELECTOR, "#complex_list_ul .result_item")
            
            if complex_items and len(complex_items) > 0:
                # 첫 번째 결과의 단지 ID 추출
                item = complex_items[0]
                link_element = item.find_element(By.CSS_SELECTOR, "a.inner")
                href = link_element.get_attribute("href")
                
                # href에서 단지번호 추출
                id_match = re.search(r"/complex/info/(\d+)", href)
                if id_match:
                    complex_id = id_match.group(1)
            else:
                # 단일 단지 페이지로 이동한 경우 URL에서 ID 추출
                current_url = driver.current_url
                
                # URL에서 단지 ID 추출
                pattern1 = r"complexes/(\d+)"
                pattern2 = r"complexNumber=(\d+)"
                pattern3 = r"/complex/info/(\d+)"
                
                match1 = re.search(pattern1, current_url)
                match2 = re.search(pattern2, current_url)
                match3 = re.search(pattern3, current_url)
                
                if match1:
                    complex_id = match1.group(1)
                elif match2:
                    complex_id = match2.group(1)
                elif match3:
                    complex_id = match3.group(1)
            
            driver.quit()
            return complex_id
        
        except Exception as e:
            self.append_progress_text(f"  - 단지번호 검색 중 오류: {str(e)}")
            try:
                if driver:
                    driver.quit()
            except:
                pass
            return None


    
    def collect_property_data(self, complex_number, complex_name, search_options):
        """단지의 매물 정보 수집"""
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
            }
            
            all_properties = []
            
            # 첫 페이지 요청
            # url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&userChannelType=PC&page=0"
            url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page=0"
                    
            
            try:
                response = requests.get(url, headers=headers)
                
                if response.status_code != 200:
                    self.append_progress_text(f"  - API 요청 실패: 상태 코드 {response.status_code}")
                    return None
                
                data = response.json()
                
                if 'result' not in data or 'list' not in data['result']:
                    self.append_progress_text("  - 데이터 구조가 예상과 다릅니다.")
                    return None
                
                property_list = data['result']['list']
                
                if not property_list:
                    self.append_progress_text("  - 매물 정보가 없습니다.")
                    return None
                
                # 첫 페이지 데이터 처리
                self.append_progress_text(f"  - 첫 페이지에서 {len(property_list)}개의 매물 발견")
                for item in property_list:
                    property_data = self.extract_property_data(item, 0)
                    all_properties.append(property_data)
                
                # 다음 페이지 여부 확인
                has_next_page = data['result'].get('hasNextPage', False)
                
                # 모든 페이지 처리 (최대 50페이지로 제한 - 필요시 조정)
                max_pages = 50  # 최대 페이지 수 증가
                page = 1
                consecutive_empty_pages = 0  # 연속된 빈 페이지 수 
                
                while has_next_page and page < max_pages:
                    url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
               
                    
                    # 최대 3번 재시도
                    max_retries = 3
                    retry_count = 0
                    success = False
                    
                    while retry_count < max_retries and not success:
                        try:
                            response = requests.get(url, headers=headers, timeout=10)
                            if response.status_code == 200:
                                success = True
                            else:
                                retry_count += 1
                                time.sleep(1)  # 1초 대기 후 재시도
                        except Exception as e:
                            self.append_progress_text(f"  - 페이지 {page} 요청 오류 (시도 {retry_count+1}/{max_retries}): {str(e)}")
                            retry_count += 1
                            time.sleep(1)
                    
                    if not success:
                        self.append_progress_text(f"  - 페이지 {page} 요청 실패")
                        break
                    
                    try:
                        data = response.json()
                        if 'result' not in data or 'list' not in data['result']:
                            self.append_progress_text(f"  - 페이지 {page} 데이터 구조 오류")
                            consecutive_empty_pages += 1
                            if consecutive_empty_pages >= 3:  # 3개 연속 오류면 중단
                                break
                            page += 1
                            continue
                        
                        property_list = data['result']['list']
                        
                        if not property_list:
                            self.append_progress_text(f"  - 페이지 {page}에 매물 없음")
                            consecutive_empty_pages += 1
                            if consecutive_empty_pages >= 3:  # 3개 연속 빈 페이지면 중단
                                break
                        else:
                            consecutive_empty_pages = 0  # 데이터가 있으면 카운터 초기화
                            
                            self.append_progress_text(f"  - 페이지 {page}에서 {len(property_list)}개의 매물 발견")
                            for item in property_list:
                                property_data = self.extract_property_data(item, page)
                                all_properties.append(property_data)
                        
                        # 다음 페이지 확인
                        has_next_page = data['result'].get('hasNextPage', False)
                        if not has_next_page:
                            break
                        
                        page += 1
                        
                    except Exception as e:
                        self.append_progress_text(f"  - 페이지 {page} 처리 중 오류: {str(e)}")
                        consecutive_empty_pages += 1
                        if consecutive_empty_pages >= 3:
                            break
                        page += 1
                
                self.append_progress_text(f"  - 총 {len(all_properties)}개의 매물 정보 수집 완료")
                return all_properties
            
            except Exception as e:
                self.append_progress_text(f"  - 데이터 요청 중 오류: {str(e)}")
                return None
        
        except Exception as e:
            self.append_progress_text(f"  - 매물 수집 중 오류: {str(e)}")
            return None



    def save_multi_search_results(self, summary_results, all_results=None):
        """다중 검색 결과를 엑셀로 저장"""
        try:
            # 날짜 형식화
            today = datetime.now().strftime('%Y%m%d')
            
            # 저장 파일명
            filename = f'단지별_매물정보_{today}.xlsx'
            excel_filename = os.path.join(self.save_path, filename)
            
            # 데이터프레임 생성
            summary_df = pd.DataFrame(summary_results)
            
            # 엑셀 저장
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                # 최저가 매물 시트
                summary_df.to_excel(writer, sheet_name='단지별_최저가', index=False)
                
                # 엑셀 시트 서식 설정 (최저가 시트)
                workbook = writer.book
                worksheet = writer.sheets['단지별_최저가']
                
                # 헤더 스타일
                header_font = Font(bold=True, size=11)
                header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
                
                for cell in worksheet[1]:
                    cell.font = header_font
                    cell.fill = header_fill
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                
                # 열 너비 자동 조정
                for column in worksheet.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    
                    for cell in column:
                        if cell.value:
                            try:
                                cell_length = len(str(cell.value))
                                max_length = max(max_length, cell_length)
                            except:
                                pass
                    
                    max_length = max(max_length, 10)
                    max_length = min(max_length, 50)
                    worksheet.column_dimensions[column_letter].width = max_length + 2
                
                # 필터 적용
                worksheet.auto_filter.ref = f"A1:{chr(64 + len(summary_df.columns))}1"
                
                # 각 단지별 모든 매물 시트 추가 (새로 추가된 부분)
                if all_results:
                    for result in all_results:
                        complex_name = result['complex_name']
                        properties_df = result['properties']
                        
                        # 시트명 (Excel 시트명 제약: 31자 이내)
                        sheet_name = complex_name[:30]
                        
                        # 거래유형 변환
                        if '거래유형' in properties_df.columns:
                            trade_type_map = {
                                'A1': '매매',
                                'B1': '전세',
                                'B2': '월세',
                                'B3': '단기임대'
                            }
                            properties_df['거래유형'] = properties_df['거래유형'].map(lambda x: trade_type_map.get(x, x))
                        
                        # 방향 변환
                        if '방향' in properties_df.columns:
                            direction_map = {
                                'SS': '남향',
                                'SE': '남동향',
                                'SW': '남서향',
                                'EE': '동향',
                                'WW': '서향',
                                'NN': '북향',
                                'NE': '북동향',
                                'NW': '북서향'
                            }
                            properties_df['방향'] = properties_df['방향'].map(lambda x: direction_map.get(x, x))
                        
                        # DataFrame을 엑셀 시트로 저장
                        properties_df.to_excel(writer, sheet_name=sheet_name, index=False)
                        
                        # 단지별 시트 서식 설정
                        sheet = writer.sheets[sheet_name]
                        
                        # 헤더 스타일 적용
                        for cell in sheet[1]:
                            cell.font = header_font
                            cell.fill = header_fill
                            cell.alignment = Alignment(horizontal='center', vertical='center')
                        
                        # 열 너비 자동 조정
                        for column in sheet.columns:
                            max_length = 0
                            column_letter = column[0].column_letter
                            
                            for cell in column:
                                if cell.value:
                                    try:
                                        cell_length = len(str(cell.value))
                                        max_length = max(max_length, cell_length)
                                    except:
                                        pass
                            
                            max_length = max(max_length, 10)
                            max_length = min(max_length, 50)
                            sheet.column_dimensions[column_letter].width = max_length + 2
                        
                        # 필터 적용
                        sheet.auto_filter.ref = f"A1:{chr(64 + len(properties_df.columns))}1"
            
            self.append_progress_text(f"검색 완료: 총 {len(summary_df)}개 단지의 매물 정보가 저장되었습니다.")
            self.append_progress_text(f"파일 저장 경로: {excel_filename}")
            messagebox.showinfo("검색 완료", f"총 {len(summary_df)}개 단지의 매물 정보가 저장되었습니다.\n\n파일 저장 경로: {excel_filename}")
            
        except Exception as e:
            self.append_progress_text(f"결과 저장 중 오류: {str(e)}")
            messagebox.showerror("오류", f"결과 저장 중 오류가 발생했습니다.\n{str(e)}")
    # NaverRealEstateApp 클래스에 아래 메서드를 추가해주세요
    def process_log_queue(self):
        """로그 큐의 메시지를 처리"""
        try:
            while True:
                message = self.log_queue.get_nowait()
                
                # 콘솔에 출력 (디버깅용)
                print(message)
                
                # 상태 표시 업데이트
                if "단지 검색을 시작합니다" in message:
                    self.status_label.config(text="검색 중...")
                elif "매물 정보 수집을 시작합니다" in message:
                    self.status_label.config(text="매물 수집 중...")
                elif "총 " in message and "개의 매물 정보가 저장되었습니다" in message:
                    match = re.search(r'총 (\d+)개', message)
                    if match:
                        count = match.group(1)
                        self.status_label.config(text=f"{count}건 매물 검색 완료")
                elif "엑셀 파일" in message and "생성 완료" in message:
                    self.status_label.config(text="파일 저장 완료")
                
                self.log_queue.task_done()
        except queue.Empty:
            pass
        
        # 100ms 후 다시 확인
        self.root.after(100, self.process_log_queue)
    
    def format_price(self, property_data):
        """매물 가격 표시 형식화"""
        trade_type = property_data.get('거래유형')
        
        if trade_type == 'A1':  # 매매
            price = property_data.get('매매가', 0)
            return f"매매 {price/10000:.0f}만원"
        elif trade_type == 'B1':  # 전세
            price = property_data.get('보증금', 0)
            return f"전세 {price/10000:.0f}만원"
        elif trade_type == 'B2':  # 월세
            deposit = property_data.get('보증금', 0)
            monthly = property_data.get('월세', 0)
            return f"월세 {deposit/10000:.0f}/{monthly}만원"
        else:
            return "가격 정보 없음"

    def append_progress_text(self, text):
        """진행 상황 텍스트 추가 (스레드 안전)"""
        def _update():
            self.progress_text.config(state='normal')
            self.progress_text.insert(tk.END, text + "\n")
            self.progress_text.see(tk.END)
            self.progress_text.config(state='disabled')
        
        if threading.current_thread() is threading.main_thread():
            _update()
        else:
            self.root.after(0, _update)
    
    def clear_progress_text(self):
        """진행 상황 텍스트 초기화"""
        self.progress_text.config(state='normal')
        self.progress_text.delete('1.0', tk.END)
        self.progress_text.config(state='disabled')    




            
    def setup_ui(self):
        # 프레임 구성
        search_frame = ttk.LabelFrame(self.root, text="아파트 단지 검색")
        search_frame.pack(fill="x", padx=10, pady=5)
        
        # 검색 프레임 구성
        ttk.Label(search_frame, text="단지명:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.search_entry = ttk.Entry(search_frame, width=40)
        self.search_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.search_entry.bind("<Return>", lambda event: self.start_search())
        
        self.search_button = ttk.Button(search_frame, text="검색 및 매물 수집", command=self.start_search)
        self.search_button.grid(row=0, column=2, padx=5, pady=5)
        
        # 설정 버튼 추가
        self.settings_button = ttk.Button(search_frame, text="⚙", width=3, command=self.open_settings)
        self.settings_button.grid(row=0, column=3, padx=5, pady=5)
        
        search_frame.columnconfigure(1, weight=1)
        
        # 상태 표시 레이블 - 중앙에 크게 표시
        self.status_label = tk.Label(
            self.root, 
            text="단지명을 입력하고 검색하세요", 
            font=("맑은 고딕", 9),
            pady=15
        )
        self.status_label.pack(fill="x")
        
        # 제작자 정보 프레임
        author_frame = ttk.Frame(self.root)
        author_frame.pack(side="bottom", fill="x", padx=10, pady=10)
        
        # 제작자 이름 (클릭 가능한 링크)
        author_label = tk.Label(author_frame, text="만든이 부태리", font=("맑은 고딕", 9, "bold"), fg="black", cursor="hand2")
        author_label.pack(anchor="center")
        author_label.bind("<Button-1>", lambda e: self.open_blog())
        
        # 블로그 주소
        blog_label = tk.Label(author_frame, text="https://blog.naver.com/landlover333", font=("맑은 고딕", 8))
        blog_label.pack(anchor="center")
        blog_label.bind("<Button-1>", lambda e: self.open_blog())
    def open_blog(self):
        """블로그 링크 열기"""
        webbrowser.open("https://blog.naver.com/landlover333")        
        
    def load_config(self):
        """설정 파일 로드"""
        try:
            # 설정 파일 경로
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if os.path.exists(config_path):
                self.config.read(config_path)
                if 'Settings' in self.config and 'save_path' in self.config['Settings']:
                    path = self.config['Settings']['save_path']
                    if os.path.exists(path):
                        self.save_path = path
        except Exception as e:
            print(f"설정 로드 오류: {str(e)}")
            
    def save_config(self):
        """설정 파일 저장"""
        try:
            # 설정 파일 경로
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if 'Settings' not in self.config:
                self.config['Settings'] = {}
            
            self.config['Settings']['save_path'] = self.save_path
            
            with open(config_path, 'w') as f:
                self.config.write(f)
        except Exception as e:
            print(f"설정 저장 오류: {str(e)}")
    
    def open_settings(self):
        """설정 창 열기"""
        # 폴더 선택 대화상자
        path = filedialog.askdirectory(initialdir=self.save_path, title="저장할 폴더 선택")
        
        if path:
            self.save_path = path
            self.save_config()
            self.log(f"저장 경로가 변경되었습니다: {path}")
    
    def log(self, message):
        """간단한 로깅"""
        # 콘솔에만 출력
        print(message)
        
        # 상태 라벨 업데이트 (있는 경우)
        if hasattr(self, 'status_label'):
            if "단지 검색을 시작합니다" in message:
                self.status_label.config(text="검색 중...")
            elif "매물 정보 수집을 시작합니다" in message:
                self.status_label.config(text="매물 수집 중...")
            elif "총 " in message and "개의 매물 정보가 저장되었습니다" in message:
                match = re.search(r'총 (\d+)개', message)
                if match:
                    count = match.group(1)
                    self.status_label.config(text=f"{count}건 매물 검색 완료")
            elif "엑셀 파일" in message and "생성 완료" in message:
                self.status_label.config(text="파일 저장 완료")
        
    def open_settings(self):
        """설정 창 열기"""
        # 폴더 선택 대화상자
        path = filedialog.askdirectory(initialdir=self.save_path, title="저장할 폴더 선택")
        
        if path:
            self.save_path = path
            self.save_config()
            self.log(f"저장 경로가 변경되었습니다: {path}")

    
    # log 메서드를 스레드 안전하게 변경
    def log(self, message):
        """스레드 안전한 로깅"""
        # 로그 큐에 메시지 추가
        self.log_queue.put(message)
        # 나머지 코드 제거

    
    def start_search(self):
        """단지명으로 검색 시작 (스레드로 실행)"""
        search_keyword = self.search_entry.get().strip()
        if not search_keyword:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        self.search_button.config(state=tk.DISABLED)
        threading.Thread(target=self.search_complex, args=(search_keyword,), daemon=True).start()
    
    def search_complex(self, search_keyword):
        """단지명으로 검색 및 단지번호 찾기"""
        try:
            self.log(f"'{search_keyword}' 단지 검색을 시작합니다...")
            
            # Chrome 옵션 설정
            chrome_options = Options()
            chrome_options.add_argument("--headless")  # 헤드리스 모드 (화면 표시 없음)
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
            
            self.log("웹 브라우저를 초기화하는 중...")

            # driver 초기화 부분을 다음과 같이 수정 (search_complex 메서드와 search_complex_detail 메서드 모두)
            # 기존 코드:
            # self.driver = webdriver.Chrome(options=chrome_options)
            
            # 수정할 코드:
            service = Service(ChromeDriverManager().install())
            self.driver = webdriver.Chrome(service=service, options=chrome_options)
            
            # 네이버 부동산 접속
            self.log("네이버 부동산에 접속 중...")
            self.driver.get("https://fin.land.naver.com/")
            time.sleep(2)
            
            # 검색 버튼 클릭
            self.log("검색 버튼 클릭 중...")
            search_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "svg[viewBox='0 0 24 24']"))
            )
            search_button.click()
            time.sleep(1)
            
            # 검색창에 단지명 입력
            self.log(f"검색창에 '{search_keyword}' 입력 중...")
            search_input = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "query"))
            )
            search_input.clear()
            search_input.send_keys(search_keyword)
            search_input.send_keys(Keys.ENTER)
            time.sleep(3)
            
            # 검색 결과 확인
            complex_list = []

            # 페이지 소스 확인 (디버깅용)
            self.log("현재 페이지 HTML 구조 확인 중...")
            page_source = self.driver.page_source
            if "keyword_auto" in page_source:
                self.log("자동완성 결과 영역을 감지했습니다.")
            if "_keywordAutoCompleteUl" in page_source:
                self.log("자동완성 리스트를 감지했습니다.")
            
            # 자동완성 영역 찾기 시도
            try:
                auto_complete = self.driver.find_element(By.ID, "keyword_auto")
                self.log(f"자동완성 영역 표시 상태: {auto_complete.get_attribute('style')}")
            except:
                self.log("자동완성 영역을 찾을 수 없습니다.")
            
      
           
            try:
                # 기존 자동완성 코드 모두 제거 후, 아래 코드로 대체
                
                # 검색 결과 목록 찾기
                self.log("검색 결과 목록을 찾는 중...")
                time.sleep(2)  # 검색 결과가 로드될 시간 확보
                
                # 복합 단지 목록 찾기
                complex_items = self.driver.find_elements(By.CSS_SELECTOR, "#complex_list_ul .result_item")
                
                if complex_items and len(complex_items) > 0:
                    self.log(f"{len(complex_items)}개의 단지 검색 결과를 찾았습니다.")
                    
                    # 검색 결과 목록에서 단지 정보 추출
                    for item in complex_items:
                        try:
                            # a 태그에서 href 속성으로 단지번호 추출
                            link_element = item.find_element(By.CSS_SELECTOR, "a.inner")
                            href = link_element.get_attribute("href")
                            
                            # href에서 단지번호 추출 (/complex/info/22746 형식)
                            complex_id = ""
                            id_match = re.search(r"/complex/info/(\d+)", href)
                            if id_match:
                                complex_id = id_match.group(1)
                            
                            # 단지명 추출
                            name_element = item.find_element(By.CSS_SELECTOR, "span.keyword")
                            complex_name = name_element.text.strip()
                            
                            # 주소 추출
                            address_element = item.find_element(By.CSS_SELECTOR, "span.address")
                            address = address_element.text.strip()
                            
                            self.log(f"단지 발견: {complex_name} (위치: {address}, ID: {complex_id})")
                            
                            # 단지 정보 추가
                            complex_list.append({
                                "name": complex_name,
                                "address": address,
                                "id": complex_id
                            })
                            
                        except Exception as e:
                            self.log(f"검색 결과 항목 처리 중 오류: {str(e)}")
                            continue
                    
                    # 검색 결과가 여러 개면 선택 팝업 표시
                    if len(complex_list) > 1:
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 선택 팝업 표시
                        dialog = ComplexSelectionDialog(self.root, "단지 선택", complex_list)
                        selected_complex = dialog.result
                        
                        if selected_complex and selected_complex["id"]:
                            # ID가 있는 경우 바로 매물 정보 수집
                            self.complex_data = selected_complex
                            self.log(f"단지번호 {selected_complex['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                            self.download_data(selected_complex['id'], selected_complex['name'])
                        else:
                            self.log("단지 선택이 취소되었습니다.")
                        return
                    
                    elif len(complex_list) == 1 and complex_list[0]["id"]:
                        # 단일 결과이고 ID가 있는 경우
                        complex_data = complex_list[0]
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        self.complex_data = complex_data
                        self.log(f"단지번호 {complex_data['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                        self.download_data(complex_data['id'], complex_data['name'])
                        return
                
                # 검색 결과 목록이 없는 경우 (단일 단지 페이지로 이동한 경우)
                current_url = self.driver.current_url
                self.log(f"현재 URL: {current_url}")
                
                # URL에서 단지 ID 추출
                pattern1 = r"complexes/(\d+)"
                pattern2 = r"complexNumber=(\d+)"
                pattern3 = r"/complex/info/(\d+)"
                
                match1 = re.search(pattern1, current_url)
                match2 = re.search(pattern2, current_url)
                match3 = re.search(pattern3, current_url)
                
                if match1:
                    complex_id = match1.group(1)
                elif match2:
                    complex_id = match2.group(1)
                elif match3:
                    complex_id = match3.group(1)
                else:
                    complex_id = None
                
                if complex_id:
                    # 단지명 추출 시도
                    try:
                        complex_name_element = self.driver.find_element(By.CSS_SELECTOR, ".complex_title, h2.text_title, .complex_name")
                        complex_name = complex_name_element.text.strip()
                    except:
                        complex_name = search_keyword
                    
                    self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                    
                    # 단지 정보 저장
                    self.complex_data = {
                        'name': complex_name,
                        'id': complex_id
                    }
                    
                    # 브라우저 종료
                    self.driver.quit()
                    self.driver = None
                    
                    # 매물 정보 수집 시작
                    self.download_data(complex_id, complex_name)
                    return
                        
            except Exception as e:
                self.log(f"검색 결과 처리 중 오류: {str(e)}")
                
                # 단일 단지 페이지로 이동한 경우 URL에서 ID 추출
                if not complex_list:
                    current_url = self.driver.current_url
                    self.log(f"단일 단지 페이지로 이동됨: {current_url}")
                    
                    # 단지명 추출
                    try:
                        complex_name = self.driver.find_element(By.CSS_SELECTOR, ".complex_title, h2.text_title").text
                    except:
                        complex_name = search_keyword
                    
                    # URL에서 단지 ID 추출
                    pattern1 = r"complexes/(\d+)"
                    pattern2 = r"complexNumber=(\d+)"
                    
                    match1 = re.search(pattern1, current_url)
                    match2 = re.search(pattern2, current_url)
                    
                    if match1:
                        complex_id = match1.group(1)
                        self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                        
                        # 단지 정보 저장
                        self.complex_data = {
                            'name': complex_name,
                            'id': complex_id
                        }
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 매물 정보 수집 시작
                        self.download_data(complex_id, complex_name)
                        return
                    elif match2:
                        complex_id = match2.group(1)
                        self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                        
                        # 단지 정보 저장
                        self.complex_data = {
                            'name': complex_name,
                            'id': complex_id
                        }
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 매물 정보 수집 시작
                        self.download_data(complex_id, complex_name)
                        return
            except Exception as e:
                self.log(f"검색 결과 처리 중 오류: {str(e)}")
            
            # 브라우저 종료
            self.driver.quit()
            self.driver = None
            
            # 검색 결과가 있으면 팝업창 표시
            if complex_list:
                # 검색 결과가 하나만 있고 ID가 있는 경우 바로 처리
                if len(complex_list) == 1 and complex_list[0]["id"]:
                    complex_data = complex_list[0]
                    self.complex_data = complex_data
                    self.log(f"단지번호 {complex_data['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                    self.download_data(complex_data['id'], complex_data['name'])
                    return
                
                # 다중 검색 결과인 경우 선택 팝업창 표시
                dialog = ComplexSelectionDialog(self.root, "단지 선택", complex_list)
                selected_complex = dialog.result
                
                if selected_complex and selected_complex["id"]:
                    # ID가 있는 경우 바로 매물 정보 수집
                    self.complex_data = selected_complex
                    self.log(f"단지번호 {selected_complex['id']}를 사용하여 매물 정보 수집을 시작합니다.")
                    self.download_data(selected_complex['id'], selected_complex['name'])
                elif selected_complex:
                    # ID가 없는 경우 클릭하여 상세 페이지 접속 후 ID 추출
                    self.log(f"'{selected_complex['name']}' 단지를 선택했습니다. 단지번호를 찾는 중...")
                    self.search_complex_detail(selected_complex["name"])
                else:
                    self.log("단지 선택이 취소되었습니다.")
            else:
                self.log("검색 결과가 없습니다. 다른 검색어를 입력해보세요.")
                messagebox.showwarning("경고", "검색 결과가 없습니다. 다른 검색어를 입력해보세요.")
        
        except Exception as e:
            self.log(f"오류 발생: {str(e)}")
            messagebox.showerror("오류", f"검색 중 오류가 발생했습니다.\n{str(e)}")
            if self.driver:
                self.driver.quit()
                self.driver = None
        
        finally:
            self.search_button.config(state=tk.NORMAL)

    def search_complex_detail(self, complex_name):
     
        """선택한 단지의 상세 정보를 검색"""
        try:
            # Chrome 옵션 설정
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
            
            self.log("웹 브라우저를 초기화하는 중...")
            self.driver = webdriver.Chrome(options=chrome_options)
            
            # 네이버 부동산 접속
            self.log("네이버 부동산에 접속 중...")
            self.driver.get("https://fin.land.naver.com/")
            time.sleep(2)
            
            # 검색 버튼 클릭
            self.log("검색 버튼 클릭 중...")
            search_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "svg[viewBox='0 0 24 24']"))
            )
            search_button.click()
            time.sleep(1)
            
            # 검색창에 단지명 입력
            self.log(f"검색창에 '{complex_name}' 입력 중...")
            search_input = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "query"))
            )
            search_input.clear()
            search_input.send_keys(complex_name)
            search_input.send_keys(Keys.ENTER)
            time.sleep(3)
            
            # 검색 결과 목록에서 일치하는 단지 찾기
            try:
                # 검색 결과 목록 찾기
                complex_items = self.driver.find_elements(By.CSS_SELECTOR, "#complex_list_ul .result_item")
                
                if complex_items and len(complex_items) > 0:
                    # 첫 번째 결과 클릭
                    first_item = complex_items[0]
                    link = first_item.find_element(By.CSS_SELECTOR, "a.inner")
                    
                    self.log(f"검색 결과 클릭: {link.text}")
                    link.click()
                    time.sleep(3)
                    
                    # 현재 URL에서 단지번호 추출
                    current_url = self.driver.current_url
                    self.log(f"단지 상세 페이지 URL: {current_url}")
                    
                    # URL에서 단지 ID 추출
                    pattern1 = r"complexes/(\d+)"
                    pattern2 = r"complexNumber=(\d+)"
                    pattern3 = r"/complex/info/(\d+)"
                    
                    match1 = re.search(pattern1, current_url)
                    match2 = re.search(pattern2, current_url)
                    match3 = re.search(pattern3, current_url)
                    
                    complex_id = None
                    if match1:
                        complex_id = match1.group(1)
                    elif match2:
                        complex_id = match2.group(1)
                    elif match3:
                        complex_id = match3.group(1)
                    
                    if complex_id:
                        self.log(f"URL에서 단지번호를 찾았습니다: {complex_id}")
                        
                        # 단지 정보 저장
                        self.complex_data = {
                            'name': complex_name,
                            'id': complex_id
                        }
                        
                        # 브라우저 종료
                        self.driver.quit()
                        self.driver = None
                        
                        # 매물 정보 수집 시작
                        self.download_data(complex_id, complex_name)
                        return
                else:
                    self.log("검색 결과 목록을 찾을 수 없습니다.")
            except Exception as e:
                self.log(f"검색 결과 클릭 중 오류: {str(e)}")
            
            # 브라우저 종료
            if self.driver:
                self.driver.quit()
                self.driver = None
            
            self.log("단지번호를 찾을 수 없습니다.")
            messagebox.showwarning("경고", "단지번호를 찾을 수 없습니다. 다른 검색어를 입력해보세요.")
        
        except Exception as e:
            self.log(f"상세 검색 중 오류 발생: {str(e)}")
            messagebox.showerror("오류", f"상세 검색 중 오류가 발생했습니다.\n{str(e)}")
            if self.driver:
                self.driver.quit()
                self.driver = None
            
            self.log("단지번호를 찾을 수 없습니다.")
            messagebox.showwarning("경고", "단지번호를 찾을 수 없습니다. 다른 검색어를 입력해보세요.")
        
        except Exception as e:
            self.log(f"상세 검색 중 오류 발생: {str(e)}")
            messagebox.showerror("오류", f"상세 검색 중 오류가 발생했습니다.\n{str(e)}")
            if self.driver:
                self.driver.quit()
                self.driver = None
    
    def download_data(self, complex_number, complex_name):
        """선택한 단지의 매물 정보 다운로드 실행"""
        self.log(f"'{complex_name}' 단지(번호: {complex_number})의 매물 정보 수집을 시작합니다.")
        
        # # 프로그레스 바 초기화
        # self.setup_progress_bar()
        
        # 스레드 작업 함수
        def download_worker():
            try:
                headers = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
                }
                
                all_properties = []
                
                # 첫 페이지 요청
                url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page=0"
                        
                self.log("첫 번째 페이지 데이터 요청 중...")
                
                try:
                    response = requests.get(url, headers=headers)
                    
                    if response.status_code != 200:
                        self.log(f"API 요청 실패: 상태 코드 {response.status_code}")
                        self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
                        return
                    
                    data = response.json()
                    
                    if 'result' not in data or 'list' not in data['result']:
                        self.log("데이터 구조가 예상과 다릅니다.")
                        self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
                        return
                    
                    property_list = data['result']['list']
                    
                    if not property_list:
                        self.log("매물 정보가 없습니다.")
                        self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
                        return
                    
                    # 첫 페이지 데이터 처리
                    self.log(f"페이지 0에서 {len(property_list)}개의 매물 정보를 찾았습니다.")
                    for item in property_list:
                        property_data = self.extract_property_data(item, 0)
                        all_properties.append(property_data)
                    
                    # 다음 페이지 여부 확인
                    has_next_page = data['result'].get('hasNextPage', False)
                    
                    if has_next_page:
                        # 스레드 수를 5로 고정 (self.thread_var.get() 대신 5 사용)
                        max_threads = 5
                        self.log(f"다음 페이지 존재. {max_threads}개의 스레드로 병렬 처리를 시작합니다.")
                                      
                        # 향상된 배치 처리 방식 적용
                        # 작은 배치로 나누어 실행하고, 빈 결과가 있으면 중단
                        batch_size = 5  # 한 번에 처리할 페이지 수
                        page = 1
                        stop_processing = False
                        
                        while not stop_processing:
                            batch_end = page + batch_size
                            self.log(f"페이지 {page}~{batch_end-1} 배치 처리 중...")
                            
                            # 배치 내 페이지 병렬 처리
                            with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
                                future_to_page = {executor.submit(self.fetch_page, complex_number, p, headers): p 
                                                 for p in range(page, batch_end)}
                                
                                # 배치 내 페이지 결과 확인
                                empty_pages = 0
                                processed_pages = []
                                
                                for future in concurrent.futures.as_completed(future_to_page):
                                    page_num = future_to_page[future]
                                    processed_pages.append(page_num)
                                    
                                    try:
                                        page_properties = future.result()
                                        
                                        if page_properties:
                                            self.log(f"페이지 {page_num}에서 {len(page_properties)}개의 매물 정보를 찾았습니다.")
                                            all_properties.extend(page_properties)
                                            self.update_progress(len(all_properties))
                                        else:
                                            self.log(f"페이지 {page_num}에서 데이터가 없습니다.")
                                            empty_pages += 1
                                    except Exception as exc:
                                        self.log(f"페이지 {page_num} 처리 중 오류: {str(exc)}")
                                        empty_pages += 1
                                
                                # 배치의 절반 이상이 빈 페이지면 중단
                                if empty_pages >= batch_size * 0.5:
                                    self.log(f"빈 페이지가 많아 처리를 중단합니다.")
                                    stop_processing = True
                                    break
                                
                                # 다음 배치 시작 페이지 설정
                                if processed_pages:
                                    page = max(processed_pages) + 1
                                else:
                                    page = batch_end
                    
                    self.log(f"총 {len(all_properties)}개의 매물 정보를 수집했습니다.")
                    
                except Exception as e:
                    self.log(f"데이터 요청 중 오류: {str(e)}")
                
                # 결과 처리 (메인 스레드에서 안전하게 처리하기 위해 after 사용)
                self.root.after(0, lambda: self.process_results(all_properties, complex_name))
                
            except Exception as e:
                self.log(f"데이터 수집 중 오류 발생: {str(e)}")
                # 에러 메시지는 메인 스레드에서 표시
                self.root.after(0, lambda: messagebox.showerror("오류", f"데이터 수집 중 오류가 발생했습니다.\n{str(e)}"))
                
            finally:
                # 버튼 상태 복원은 메인 스레드에서 처리
                self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
        
        # 작업을 별도 스레드로 실행
        worker_thread = threading.Thread(target=download_worker, daemon=True)
        worker_thread.start()

    
    def process_results(self, all_properties, complex_name):
        """수집된 데이터 처리 (메인 스레드에서 실행)"""
        try:
            # self.complete_progress()
            
            if not all_properties:
                self.log("수집된 매물 정보가 없습니다.")
                messagebox.showinfo("알림", "수집된 매물 정보가 없습니다.")
                return
            
            # 데이터프레임 생성
            df = pd.DataFrame(all_properties)
            
            # 거래유형 변환
            trade_type_map = {
                'A1': '매매',
                'B1': '전세',
                'B2': '월세',
                'B3': '단기임대'
            }
            df['거래유형'] = df['거래유형'].map(lambda x: trade_type_map.get(x, x))
            
            # 방향 변환
            direction_map = {
                'SS': '남향',
                'SE': '남동향',
                'SW': '남서향',
                'EE': '동향',
                'WW': '서향',
                'NN': '북향',
                'NE': '북동향',
                'NW': '북서향'
            }
            df['방향'] = df['방향'].map(lambda x: direction_map.get(x, x))
            
            # 가격변동상태 변환 추가
            price_change_map = {
                0: '변동없음',
                1: '가격상승',
                -1: '가격하락'
            }
            df['가격변동상태'] = df['가격변동상태'].map(lambda x: price_change_map.get(x, '알수없음'))
            
            # 금액 데이터 형식 변환 (프리미엄 추가)
            money_columns = ['보증금', '월세', '매매가', '프리미엄', '최소매매가', '최대매매가', 
                             '최소보증금', '최대보증금', '최소월세', '최대월세', '최소프리미엄', '최대프리미엄']
            
            for col in money_columns:
                if col in df.columns:
                    df[col] = df[col].apply(lambda x: f"{x/10000:.0f}" if x > 0 else '')
            
            # 날짜 형식화
            today = datetime.now().strftime('%Y%m%d')
            
            # 엑셀 파일로 저장
            excel_filename = os.path.join(self.save_path, f'{complex_name}_매물정보_{today}.xlsx')
            
            # 여러 시트로 저장 (오픈파이썬 엔진 사용하여 필터 기능 적용)
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                # 전체 매물 시트
                df.to_excel(writer, sheet_name='전체매물', index=False)
                self.apply_filter_to_sheet(writer.sheets['전체매물'], len(df.columns))
                
                # 매매 매물만 필터링 (query 사용)
                deal_properties = df.query('매매가 != "" and 매매가.notnull()')
                if not deal_properties.empty:
                    deal_properties.to_excel(writer, sheet_name='매매매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['매매매물'], len(deal_properties.columns))
                
                # 전세 매물만 필터링 (query 사용)
                full_deposit = df.query('보증금 != "" and 보증금.notnull() and (월세 == "" or 월세.isnull())')
                if not full_deposit.empty:
                    full_deposit.to_excel(writer, sheet_name='전세매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['전세매물'], len(full_deposit.columns))
                
                # 월세 매물만 필터링 (query 사용)
                monthly_rent = df.query('월세 != "" and 월세.notnull()')
                if not monthly_rent.empty:
                    monthly_rent.to_excel(writer, sheet_name='월세매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['월세매물'], len(monthly_rent.columns))
                
                # 중개사 수별 매물 필터링 (query 사용)
                multi_realtor = df.query('`중개사 수` > 1')
                if not multi_realtor.empty:
                    multi_realtor.to_excel(writer, sheet_name='중복매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['중복매물'], len(multi_realtor.columns))
                
                # 열 너비 자동 조정 (모든 시트)
                for sheet_name in writer.sheets:
                    sheet = writer.sheets[sheet_name]
                    self.adjust_column_width(sheet)
            
            self.log(f"엑셀 파일 '{excel_filename}' 생성 완료!")
            self.log(f"총 {len(df)}개의 매물 정보가 저장되었습니다.")
            
            # 저장 완료 메시지 표시
            messagebox.showinfo("완료", f"'{complex_name}' 단지의 매물 정보가 '{excel_filename}' 파일로 저장되었습니다.")
        
        except Exception as e:
            self.log(f"결과 처리 중 오류: {str(e)}")
            messagebox.showerror("오류", f"결과 처리 중 오류가 발생했습니다.\n{str(e)}")
    
    def apply_filter_to_sheet(self, sheet, columns_count):
        """시트의 첫 행에 필터 적용"""
        try:
            # openpyxl 라이브러리가 설치되어 있어야 함
            # 첫 행에 필터 적용
            sheet.auto_filter.ref = f"A1:{chr(64 + columns_count)}1"
            
            # 첫 행(헤더) 스타일 설정
            from openpyxl.styles import Font, PatternFill, Alignment
            
            header_font = Font(bold=True, size=11)
            header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
            
            for cell in sheet[1]:
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = Alignment(horizontal='center', vertical='center')
        except Exception as e:
            self.log(f"필터 적용 중 오류: {str(e)}")
    
    def adjust_column_width(self, sheet):
        """열 너비 자동 조정"""
        try:
            for column in sheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                ㄴ
                for cell in column:
                    if cell.value:
                        # 셀 값의 길이 계산
                        try:
                            cell_length = len(str(cell.value))
                            max_length = max(max_length, cell_length)
                        except:
                            pass
                
                # 최소 너비 설정
                max_length = max(max_length, 10)
                # 최대 너비 제한
                max_length = min(max_length, 50)
                
                # 열 너비 설정 (약간의 여유 공간 추가)
                sheet.column_dimensions[column_letter].width = max_length + 2
        except Exception as e:
            self.log(f"열 너비 조정 중 오류: {str(e)}")

        

    def fetch_page(self, complex_number, page, headers):
        """페이지별 데이터 요청 (병렬 처리용)"""
        url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
                
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                return None
            
            data = response.json()
            
            if 'result' in data and 'list' in data['result']:
                property_list = data['result']['list']
                
                if not property_list:
                    return None
                
                # 매물 정보 추출
                page_properties = []
                for item in property_list:
                    property_data = self.extract_property_data(item, page)
                    page_properties.append(property_data)
                
                return page_properties
        except Exception as e:
            self.log(f"페이지 {page} 요청 오류: {str(e)}")
            return None
        
        return None
    
    def extract_property_data(self, item, page):
        """매물 항목에서 필요한 정보 추출"""
        rep_info = item['representativeArticleInfo']
        
        # 기존 매물 정보 추출 코드에 프리미엄 가격 추가
        property_data = {
            '단지명': rep_info.get('complexName', ''),
            '동': rep_info.get('dongName', ''),
            '거래유형': rep_info.get('tradeType', ''),
            '전용면적': rep_info['spaceInfo'].get('exclusiveSpace', ''),
            '타입구분': rep_info['spaceInfo'].get('nameType', ''),
            '층/전체층': rep_info['articleDetail'].get('floorInfo', ''),
            '방향': rep_info['articleDetail'].get('direction', ''),
            '매물특징': rep_info['articleDetail'].get('articleFeatureDescription', ''),
            '보증금': rep_info['priceInfo'].get('warrantyPrice', 0),
            '월세': rep_info['priceInfo'].get('rentPrice', 0),
            '매매가': rep_info['priceInfo'].get('dealPrice', 0),
            '프리미엄': rep_info['priceInfo'].get('premiumPrice', 0),  # 프리미엄 가격 추가
            '가격변동상태': rep_info['priceInfo'].get('priceChangeStatus', 0),
            '중개사명': rep_info['brokerInfo'].get('brokerageName', ''),
            'VR노출여부': rep_info['articleMediaDto'].get('isVrExposed', False) if rep_info.get('articleMediaDto') else False,
            '중개사 수': 0,
            '최소매매가': 0,
            '최대매매가': 0,
            '최소보증금': 0,
            '최대보증금': 0,
            '최소월세': 0,
            '최대월세': 0,
            '최소프리미엄': 0,  # 최소 프리미엄 추가
            '최대프리미엄': 0,  # 최대 프리미엄 추가
            '페이지번호': page
        }
        
        # 중복 매물 정보 처리
        if 'duplicatedArticlesInfo' in item and item['duplicatedArticlesInfo'] is not None:
            # 중개사 수 추가
            property_data['중개사 수'] = item['duplicatedArticlesInfo'].get('realtorCount', 0)
            
            # 중복 매물 가격 정보
            if 'representativePriceInfo' in item['duplicatedArticlesInfo']:
                price_info = item['duplicatedArticlesInfo']['representativePriceInfo']
                
                if 'dealPrice' in price_info:
                    property_data['최소매매가'] = price_info['dealPrice'].get('minPrice', 0)
                    property_data['최대매매가'] = price_info['dealPrice'].get('maxPrice', 0)
                
                if 'warrantyPrice' in price_info:
                    property_data['최소보증금'] = price_info['warrantyPrice'].get('minPrice', 0)
                    property_data['최대보증금'] = price_info['warrantyPrice'].get('maxPrice', 0)
                
                if 'rentPrice' in price_info:
                    property_data['최소월세'] = price_info['rentPrice'].get('minPrice', 0)
                    property_data['최대월세'] = price_info['rentPrice'].get('maxPrice', 0)
                
                # 프리미엄 최소/최대 가격 추가
                if 'premiumPrice' in price_info:
                    property_data['최소프리미엄'] = price_info['premiumPrice'].get('minPrice', 0)
                    property_data['최대프리미엄'] = price_info['premiumPrice'].get('maxPrice', 0)
        
        return property_data

    def setup_progress_bar(self):
        """프로그레스 바 초기화"""
        # # 프로그레스 바가 이미 존재하면 제거
        # if hasattr(self, 'progress_frame'):
        #     self.progress_frame.destroy()
        
        # # 프레임 생성
        # self.progress_frame = ttk.Frame(self.root)
        # self.progress_frame.pack(fill="x", padx=10, pady=5)
        
        # # 레이블
        # self.progress_label = ttk.Label(self.progress_frame, text="데이터 수집 중...")
        # self.progress_label.pack(anchor="w", pady=(0, 5))
        
        # # 프로그레스 바
        # self.progress_bar = ttk.Progressbar(self.progress_frame, orient="horizontal", mode="indeterminate", length=300)
        # self.progress_bar.pack(fill="x")
        # self.progress_bar.start(10)
        
        # # 항목 카운터 레이블
        # self.count_label = ttk.Label(self.progress_frame, text="0개 매물 수집됨")
        # self.count_label.pack(anchor="e", pady=(5, 0))
        
        # # UI 업데이트
        # self.root.update_idletasks()
        pass
    
    def update_progress(self, count):
        """프로그레스 상태 업데이트 (스레드 안전)"""
        # def _update_progress():
        #     if hasattr(self, 'count_label'):
        #         self.count_label.config(text=f"{count}개 매물 수집됨")
        #         self.root.update_idletasks()
        
        # # 메인 스레드에서 실행
        # if threading.current_thread() is threading.main_thread():
        #     _update_progress()
        # else:
        #     # 다른 스레드에서는 after 메서드를 사용
        #     self.root.after(0, _update_progress)
        # 로그에만 정보 출력
        self.log(f"현재까지 {count}개 매물 수집됨")
    
    def complete_progress(self):
        """프로그레스 바 완료 처리 (스레드 안전)"""
        # def _complete_progress():
        #     if hasattr(self, 'progress_bar'):
        #         self.progress_bar.stop()
        #         self.progress_bar.configure(mode="determinate", value=100)
        #         self.progress_label.config(text="데이터 수집 완료")
        #         self.root.update_idletasks()
        
        # # 메인 스레드에서 실행
        # if threading.current_thread() is threading.main_thread():
        #     _complete_progress()
        # else:
        #     # 다른 스레드에서는 after 메서드를 사용
        #     self.root.after(0, _complete_progress)
        pass

# 메인 함수
def main():
    
    # 아직 유효한 경우 프로그램 실행
    root = tk.Tk()
    app = NaverRealEstateApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

'잠실엘' 단지 검색을 시작합니다...
웹 브라우저를 초기화하는 중...
네이버 부동산에 접속 중...
검색 버튼 클릭 중...
오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0x93fd33+62915]
	GetHandleVerifier [0x0x93fd74+62980]
	(No symbol) [0x0x773e13]
	(No symbol) [0x0x7bc89e]
	(No symbol) [0x0x7bcc3b]
	(No symbol) [0x0x804ec2]
	(No symbol) [0x0x7e1424]
	(No symbol) [0x0x8026ea]
	(No symbol) [0x0x7e11d6]
	(No symbol) [0x0x7b0833]
	(No symbol) [0x0x7b16a4]
	GetHandleVerifier [0x0xba8d23+2590131]
	GetHandleVerifier [0x0xba3f6a+2570234]
	GetHandleVerifier [0x0x9659ea+217722]
	GetHandleVerifier [0x0x956058+153832]
	GetHandleVerifier [0x0x95c4bd+179533]
	GetHandleVerifier [0x0x947738+94152]
	GetHandleVerifier [0x0x9478c2+94546]
	GetHandleVerifier [0x0x932bda+9322]
	BaseThreadInitThunk [0x0x75ff5d49+25]
	RtlInitializeExceptionChain [0x0x77afd2fb+107]
	RtlGetAppContainerNamedObjectPath [0x0x77afd281+561]



In [ ]:
import requests
import pandas as pd
import json
import time
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import threading
from datetime import datetime
import re
from tkinter import filedialog
import configparser
import os
import concurrent.futures
import queue
import webbrowser
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment

class ComplexSelectionDialog:
    def __init__(self, parent, title, complex_list):
        self.result = None
        
        # 대화상자 생성
        self.dialog = tk.Toplevel(parent)
        self.dialog.title(title)
        self.dialog.geometry("700x500")
        self.dialog.minsize(700, 500)
        self.dialog.transient(parent)
        self.dialog.grab_set()
    
        # 프레임 생성
        frame = ttk.Frame(self.dialog, padding="10")
        frame.pack(fill=tk.BOTH, expand=True)
        
        # 라벨 생성
        ttk.Label(frame, text="검색된 단지 목록. 선택하세요:").pack(anchor="w", pady=(0, 5))
        
        # 트리뷰 생성
        columns = ("name", "address")
        self.tree = ttk.Treeview(frame, columns=columns, show="headings", height=15)
        self.tree.heading("name", text="단지명")
        self.tree.heading("address", text="주소")
        
        self.tree.column("name", width=250)
        self.tree.column("address", width=400)
        
        # 스크롤바 생성
        scrollbar = ttk.Scrollbar(frame, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        
        self.tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # 데이터 추가
        for item in complex_list:
            self.tree.insert("", "end", values=(item["name"], item["address"]), tags=(item["id"],))
        
        # 버튼 프레임
        button_frame = ttk.Frame(self.dialog, padding="20")
        button_frame.pack(fill="x", pady=10)
        
        # 확인 버튼
        select_button = ttk.Button(button_frame, text="선택", command=self.on_select, width=10)
        select_button.pack(side="right", padx=10)
        
        # 취소 버튼
        cancel_button = ttk.Button(button_frame, text="취소", command=self.on_cancel, width=10)
        cancel_button.pack(side="right", padx=10)
        
        # 더블 클릭 이벤트 바인딩
        self.tree.bind("<Double-1>", lambda e: self.on_select())
        
        # 창 중앙 배치
        self.center_dialog()
        
        # 대화상자가 닫힐 때까지 대기
        parent.wait_window(self.dialog)
    
    def center_dialog(self):
        """대화상자를 화면 중앙에 배치"""
        self.dialog.update_idletasks()
        
        screen_width = self.dialog.winfo_screenwidth()
        screen_height = self.dialog.winfo_screenheight()
        
        dialog_width = self.dialog.winfo_width()
        dialog_height = self.dialog.winfo_height()
        
        x = (screen_width - dialog_width) // 2
        y = (screen_height - dialog_height) // 2
        
        self.dialog.geometry(f"+{x}+{y}")
    
    def on_select(self):
        selected_items = self.tree.selection()
        if selected_items:
            item_id = selected_items[0]
            item_values = self.tree.item(item_id)['values']
            item_tags = self.tree.item(item_id)['tags']
            
            self.result = {
                "name": item_values[0],
                "address": item_values[1],
                "id": item_tags[0]
            }
            
        self.dialog.destroy()
    
    def on_cancel(self):
        self.dialog.destroy()

class NaverRealEstateApp:
    def __init__(self, root):
        self.root = root
        self.root.title("네이버 부동산 매물 수집기")
        self.root.geometry("600x450")
        self.root.resizable(True, True)
        
        # 변수 초기화
        self.complex_data = None
        
        # 로그 큐 초기화
        self.log_queue = queue.Queue()
        
        # 설정 관리
        self.config = configparser.ConfigParser()
        self.save_path = os.path.expanduser("~/Documents")
        self.load_config()
        
        # 탭 컨트롤 생성
        self.tab_control = ttk.Notebook(self.root)
        
        # 탭 1: 단일 단지 검색
        self.tab1 = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab1, text='단일 단지 검색')
        
        # 탭 2: 다중 단지 검색
        self.tab2 = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab2, text='다중 단지 검색')
        
        self.tab_control.pack(expand=1, fill="both")
        
        # UI 설정
        self.setup_single_search_ui()
        self.setup_multi_search_ui()
        
        # 상태 표시 레이블
        self.status_label = tk.Label(
            self.root, 
            text="단지명을 입력하고 검색하세요", 
            font=("맑은 고딕", 9),
            pady=15
        )
        self.status_label.pack(fill="x")
        
        # 제작자 정보
        self.setup_author_info()
        
        # 로그 큐 처리 시작
        self.root.after(100, self.process_log_queue)
    
    def setup_single_search_ui(self):
        """단일 단지 검색 UI 설정"""
        search_frame = ttk.LabelFrame(self.tab1, text="아파트 단지 검색")
        search_frame.pack(fill="x", padx=10, pady=5)
        
        ttk.Label(search_frame, text="단지명:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.search_entry = ttk.Entry(search_frame, width=40)
        self.search_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.search_entry.bind("<Return>", lambda event: self.start_search())
        
        self.search_button = ttk.Button(search_frame, text="검색 및 매물 수집", command=self.start_search)
        self.search_button.grid(row=0, column=2, padx=5, pady=5)
        
        self.settings_button = ttk.Button(search_frame, text="⚙", width=3, command=self.open_settings)
        self.settings_button.grid(row=0, column=3, padx=5, pady=5)
        
        search_frame.columnconfigure(1, weight=1)
    
    def setup_multi_search_ui(self):
        """다중 단지 검색 UI 설정"""
        main_frame = ttk.Frame(self.tab2, padding=10)
        main_frame.pack(fill="both", expand=True)
        
        left_frame = ttk.LabelFrame(main_frame, text="단지 목록 입력 (최대 50개)")
        left_frame.pack(side="left", fill="both", expand=True, padx=(0, 5))
        
        ttk.Label(left_frame, text="단지명을 한 줄에 하나씩 입력하세요:").pack(anchor="w", pady=(5, 0))
        self.complex_text = scrolledtext.ScrolledText(left_frame, width=30, height=15)
        self.complex_text.pack(fill="both", expand=True, pady=5)
        
        right_frame = ttk.LabelFrame(main_frame, text="검색 옵션")
        right_frame.pack(side="right", fill="both", padx=(5, 0))
        
        ttk.Label(right_frame, text="전용면적 범위:").grid(row=0, column=0, sticky="w", pady=5)
        
        area_frame = ttk.Frame(right_frame)
        area_frame.grid(row=0, column=1, sticky="ew", pady=5)
        
        self.min_area_var = tk.StringVar()
        self.max_area_var = tk.StringVar()
        
        ttk.Entry(area_frame, textvariable=self.min_area_var, width=6).pack(side="left")
        ttk.Label(area_frame, text="㎡ ~").pack(side="left", padx=2)
        ttk.Entry(area_frame, textvariable=self.max_area_var, width=6).pack(side="left")
        ttk.Label(area_frame, text="㎡").pack(side="left", padx=2)
        
        ttk.Label(right_frame, text="층수 조건:").grid(row=1, column=0, sticky="w", pady=5)
        ttk.Label(right_frame, text="중층/고층/5층 이상 매물만 검색").grid(row=1, column=1, sticky="w", pady=5)
        
        ttk.Label(right_frame, text="거래 유형:").grid(row=2, column=0, sticky="w", pady=5)
        ttk.Label(right_frame, text="매매 및 전세 최저가 조사").grid(row=2, column=1, sticky="w", pady=5)
        
        button_frame = ttk.Frame(right_frame)
        button_frame.grid(row=3, column=0, columnspan=2, pady=10)
        
        self.multi_search_button = ttk.Button(button_frame, text="다중 검색 시작", command=self.start_multi_search, width=20)
        self.multi_search_button.pack(pady=5)
        
        self.progress_text = scrolledtext.ScrolledText(right_frame, width=30, height=10, state='disabled')
        self.progress_text.grid(row=4, column=0, columnspan=2, sticky="ew", pady=5)
    
    def setup_author_info(self):
        """제작자 정보 프레임 설정"""
        author_frame = ttk.Frame(self.root)
        author_frame.pack(side="bottom", fill="x", padx=10, pady=10)
        
        author_label = tk.Label(author_frame, text="만든이 부태리", font=("맑은 고딕", 9, "bold"), fg="black", cursor="hand2")
        author_label.pack(anchor="center")
        author_label.bind("<Button-1>", lambda e: self.open_blog())
        
        blog_label = tk.Label(author_frame, text="https://blog.naver.com/landlover333", font=("맑은 고딕", 8))
        blog_label.pack(anchor="center")
        blog_label.bind("<Button-1>", lambda e: self.open_blog())
    
    def open_blog(self):
        """블로그 링크 열기"""
        webbrowser.open("https://blog.naver.com/landlover333")
    
    def search_complex_by_api(self, keyword):
        """API를 사용하여 단지 검색"""
        try:
            # URL 인코딩
            import urllib.parse
            encoded_keyword = urllib.parse.quote(keyword)
            
            # 자동완성 API 호출
            url = f"https://fin.land.naver.com/front-api/v1/search/autocomplete/complexes?keyword={encoded_keyword}&size=10&page=0"
            
            # 더 완전한 헤더 설정으로 봇 감지 회피
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                "Accept": "application/json, text/plain, */*",
                "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                "Accept-Encoding": "gzip, deflate, br",
                "Origin": "https://fin.land.naver.com",
                "Referer": "https://fin.land.naver.com/",
                "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
                "Sec-Ch-Ua-Mobile": "?0",
                "Sec-Ch-Ua-Platform": '"Windows"',
                "Sec-Fetch-Dest": "empty",
                "Sec-Fetch-Mode": "cors",
                "Sec-Fetch-Site": "same-origin",
                "Cache-Control": "no-cache",
                "Pragma": "no-cache"
            }
            
            # 세션 사용으로 쿠키 유지
            if not hasattr(self, 'session'):
                self.session = requests.Session()
                # 먼저 메인 페이지 방문하여 쿠키 획득
                self.session.get("https://fin.land.naver.com/", headers=headers)
                time.sleep(0.5)
            
            self.log(f"API로 '{keyword}' 검색 중...")
            
            # 재시도 로직 추가
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = self.session.get(url, headers=headers, timeout=10)
                    
                    if response.status_code == 429:
                        # Rate limit 에러 시 대기 후 재시도
                        wait_time = (attempt + 1) * 2  # 2, 4, 6초 대기
                        self.log(f"요청 제한 발생. {wait_time}초 대기 후 재시도... (시도 {attempt+1}/{max_retries})")
                        time.sleep(wait_time)
                        continue
                    
                    if response.status_code != 200:
                        self.log(f"API 요청 실패: 상태 코드 {response.status_code}")
                        
                        # 대체 방법: 일반 검색 API 사용
                        if attempt == max_retries - 1:
                            return self.search_complex_alternative(keyword)
                        continue
                    
                    data = response.json()
                    
                    # 결과 파싱
                    if 'result' in data and 'list' in data['result']:
                        complex_list = []
                        for item in data['result']['list']:
                            complex_info = {
                                'name': item.get('complexName', ''),
                                'id': str(item.get('complexNumber', '')),
                                'address': item.get('addressName', '')
                            }
                            complex_list.append(complex_info)
                        
                        if complex_list:
                            return complex_list
                        else:
                            # 자동완성 결과가 없으면 대체 검색 시도
                            return self.search_complex_alternative(keyword)
                    else:
                        self.log("API 응답에 예상된 데이터가 없습니다.")
                        if attempt == max_retries - 1:
                            return self.search_complex_alternative(keyword)
                        
                except requests.exceptions.RequestException as e:
                    self.log(f"네트워크 오류 (시도 {attempt+1}/{max_retries}): {str(e)}")
                    if attempt < max_retries - 1:
                        time.sleep(2)
                    else:
                        return None
            
            return None
                
        except Exception as e:
            self.log(f"API 검색 중 오류: {str(e)}")
            return None
    
    def search_complex_alternative(self, keyword):
        """대체 검색 방법 - 일반 검색 API 사용"""
        try:
            import urllib.parse
            encoded_keyword = urllib.parse.quote(keyword)
            
            # 대체 URL 패턴들
            alternative_urls = [
                f"https://fin.land.naver.com/front-api/v1/search/complexes?keyword={encoded_keyword}&page=0&pageSize=20",
                f"https://fin.land.naver.com/front-api/v1/complex/search?query={encoded_keyword}"
            ]
            
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                "Accept": "application/json, text/plain, */*",
                "Referer": "https://fin.land.naver.com/",
                "Origin": "https://fin.land.naver.com"
            }
            
            self.log("대체 검색 방법 시도 중...")
            
            for url in alternative_urls:
                try:
                    if hasattr(self, 'session'):
                        response = self.session.get(url, headers=headers, timeout=10)
                    else:
                        response = requests.get(url, headers=headers, timeout=10)
                    
                    if response.status_code == 200:
                        data = response.json()
                        # 응답 구조에 따라 파싱 조정
                        if 'result' in data:
                            # 결과 처리 로직
                            complex_list = []
                            items = data['result'].get('list', []) or data['result'].get('items', [])
                            
                            for item in items:
                                complex_info = {
                                    'name': item.get('complexName', item.get('name', '')),
                                    'id': str(item.get('complexNumber', item.get('complexNo', ''))),
                                    'address': item.get('addressName', item.get('address', ''))
                                }
                                if complex_info['id']:  # ID가 있는 경우만 추가
                                    complex_list.append(complex_info)
                            
                            if complex_list:
                                self.log(f"대체 방법으로 {len(complex_list)}개 단지 발견")
                                return complex_list
                        
                except Exception as e:
                    continue
            
            # 모든 방법 실패 시 기본 Selenium 방식으로 전환
            self.log("API 검색 실패. 웹 스크래핑 방식으로 전환합니다...")
            return self.search_complex_with_selenium(keyword)
            
        except Exception as e:
            self.log(f"대체 검색 중 오류: {str(e)}")
            return None
    
    def search_complex_with_selenium(self, keyword):
        """Selenium을 사용한 폴백 검색 방법"""
        try:
            from selenium import webdriver
            from selenium.webdriver.common.by import By
            from selenium.webdriver.common.keys import Keys
            from selenium.webdriver.chrome.options import Options
            from selenium.webdriver.chrome.service import Service
            from webdriver_manager.chrome import ChromeDriverManager
            
            self.log("웹 브라우저 방식으로 검색 중...")
            
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("--disable-blink-features=AutomationControlled")
            chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
            chrome_options.add_experimental_option('useAutomationExtension', False)
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
            
            service = Service(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service, options=chrome_options)
            
            # 자동화 감지 방지 스크립트
            driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
                'source': '''
                    Object.defineProperty(navigator, 'webdriver', {
                        get: () => undefined
                    })
                '''
            })
            
            # 검색 수행
            search_url = f"https://fin.land.naver.com/search?q={keyword}"
            driver.get(search_url)
            time.sleep(3)
            
            # 검색 결과 파싱
            complex_list = []
            
            # 여러 가능한 선택자 시도
            selectors = [
                "a[href*='/complexes/']",
                ".search_list .item_link",
                ".complex_item a"
            ]
            
            for selector in selectors:
                try:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    if elements:
                        for element in elements[:10]:  # 최대 10개만
                            href = element.get_attribute('href')
                            if href and '/complexes/' in href:
                                # URL에서 단지 ID 추출
                                import re
                                match = re.search(r'/complexes/(\d+)', href)
                                if match:
                                    complex_id = match.group(1)
                                    complex_name = element.text.strip() or keyword
                                    
                                    complex_list.append({
                                        'name': complex_name,
                                        'id': complex_id,
                                        'address': ''
                                    })
                        
                        if complex_list:
                            break
                            
                except Exception as e:
                    continue
            
            driver.quit()
            
            if complex_list:
                self.log(f"웹 스크래핑으로 {len(complex_list)}개 단지 발견")
                return complex_list
            
            return None
            
        except Exception as e:
            self.log(f"Selenium 검색 중 오류: {str(e)}")
            return None
    
    def start_search(self):
        """단지명으로 검색 시작"""
        search_keyword = self.search_entry.get().strip()
        if not search_keyword:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        self.search_button.config(state=tk.DISABLED)
        threading.Thread(target=self.search_complex, args=(search_keyword,), daemon=True).start()
    
    def search_complex(self, search_keyword):
        """단지명으로 검색 및 단지번호 찾기 - API 방식"""
        try:
            self.log(f"'{search_keyword}' 단지 검색을 시작합니다...")
            
            # API로 단지 검색
            complex_list = self.search_complex_by_api(search_keyword)
            
            if not complex_list:
                self.log("검색 결과가 없습니다.")
                self.root.after(0, lambda: messagebox.showwarning("경고", "검색 결과가 없습니다. 다른 검색어를 입력해보세요."))
                return
            
            self.log(f"{len(complex_list)}개의 단지를 찾았습니다.")
            
            if len(complex_list) == 1:
                # 단일 결과 - 바로 사용
                selected_complex = complex_list[0]
                self.complex_data = selected_complex
                self.log(f"단지 선택: {selected_complex['name']} (번호: {selected_complex['id']})")
                self.download_data(selected_complex['id'], selected_complex['name'])
            else:
                # 여러 결과 - 선택 대화상자 표시
                def show_selection_dialog():
                    dialog = ComplexSelectionDialog(self.root, "단지 선택", complex_list)
                    selected_complex = dialog.result
                    
                    if selected_complex and selected_complex["id"]:
                        self.complex_data = selected_complex
                        self.log(f"단지 선택: {selected_complex['name']} (번호: {selected_complex['id']})")
                        # 다시 스레드로 실행
                        threading.Thread(target=self.download_data, 
                                       args=(selected_complex['id'], selected_complex['name']), 
                                       daemon=True).start()
                    else:
                        self.log("단지 선택이 취소되었습니다.")
                        self.search_button.config(state=tk.NORMAL)
                
                # 메인 스레드에서 대화상자 표시
                self.root.after(0, show_selection_dialog)
                
        except Exception as e:
            self.log(f"오류 발생: {str(e)}")
            self.root.after(0, lambda: messagebox.showerror("오류", f"검색 중 오류가 발생했습니다.\n{str(e)}"))
        finally:
            # 대화상자가 표시되는 경우가 아니면 버튼 활성화
            if not complex_list or len(complex_list) == 1:
                self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
    
    def find_complex_id(self, complex_name):
        """다중 검색용 단지번호 찾기 - API 방식"""
        try:
            self.append_progress_text(f"  - '{complex_name}' 단지 검색 중...")
            
            complex_list = self.search_complex_by_api(complex_name)
            
            if not complex_list:
                return None
            
            # 첫 번째 결과 사용 (다중 검색에서는 자동 선택)
            return complex_list[0]['id']
            
        except Exception as e:
            self.append_progress_text(f"  - 단지번호 검색 중 오류: {str(e)}")
            return None
    
    def start_multi_search(self):
        """다중 단지 검색 시작"""
        complex_text = self.complex_text.get('1.0', tk.END).strip()
        complex_list = [name.strip() for name in complex_text.split('\n') if name.strip()]
        
        if len(complex_list) > 50:
            messagebox.showwarning("경고", "최대 50개까지만 검색 가능합니다. 처음 50개만 처리합니다.")
            complex_list = complex_list[:50]
        
        if not complex_list:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        min_area = self.min_area_var.get().strip()
        max_area = self.max_area_var.get().strip()
        
        try:
            min_area = float(min_area) if min_area else None
            max_area = float(max_area) if max_area else None
        except ValueError:
            messagebox.showwarning("경고", "전용면적은 숫자로 입력하세요.")
            return
        
        search_options = {
            'min_area': min_area,
            'max_area': max_area
        }
        
        self.clear_progress_text()
        self.append_progress_text(f"총 {len(complex_list)}개 단지 검색을 시작합니다.\n")
        
        self.multi_search_button.config(state=tk.DISABLED)
        
        threading.Thread(target=self.process_multi_search, 
                        args=(complex_list, search_options), 
                        daemon=True).start()
    
    def process_multi_search(self, complex_list, search_options):
        """다중 단지 검색 처리"""
        try:
            all_results = []
            summary_results = []
            
            for idx, complex_name in enumerate(complex_list):
                self.append_progress_text(f"[{idx+1}/{len(complex_list)}] '{complex_name}' 검색 중...")
                
                # API로 단지 정보 검색
                complex_id = self.find_complex_id(complex_name)
                
                if not complex_id:
                    self.append_progress_text(f"  - 단지를 찾을 수 없습니다.\n")
                    continue
                
                self.append_progress_text(f"  - 단지번호: {complex_id}")
                
                # 매물 정보 수집
                property_data = self.collect_property_data(complex_id, complex_name, search_options)
                
                if not property_data:
                    self.append_progress_text(f"  - 매물 정보가 없습니다.\n")
                    continue
                
                df = pd.DataFrame(property_data)
                
                all_results.append({
                    'complex_name': complex_name,
                    'complex_id': complex_id,
                    'properties': df
                })
                
                # 최저가 찾기 로직 (기존 코드 유지)
                # ... (최저가 찾기 및 summary_results 추가 코드)
                self.append_progress_text("")
            
            if summary_results:
                self.save_multi_search_results(summary_results, all_results)
            else:
                self.append_progress_text("검색 완료: 조건에 맞는 매물이 없습니다.")
                messagebox.showinfo("검색 완료", "조건에 맞는 매물이 없습니다.")
        
        except Exception as e:
            self.append_progress_text(f"오류 발생: {str(e)}")
            messagebox.showerror("오류", f"다중 검색 중 오류가 발생했습니다.\n{str(e)}")
        
        finally:
            self.root.after(0, lambda: self.multi_search_button.config(state=tk.NORMAL))
    
    def collect_property_data(self, complex_number, complex_name, search_options):
        """단지의 매물 정보 수집 (다중 검색용 - 순차 처리)"""
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
                "Referer": "https://fin.land.naver.com/",
                "Accept": "application/json, text/plain, */*"
            }
            
            all_properties = []
            page = 0
            consecutive_empty = 0
            max_pages = 50  # 다중 검색은 페이지 제한을 좀 더 적게
            
            while page < max_pages:
                url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
                
                # 페이지 간 딜레이 (다중 검색은 조금 더 보수적으로)
                if page > 0:
                    time.sleep(1.5)  # 1.5초 대기
                
                try:
                    # 세션 사용
                    if hasattr(self, 'session'):
                        response = self.session.get(url, headers=headers, timeout=10)
                    else:
                        response = requests.get(url, headers=headers, timeout=10)
                    
                    if response.status_code == 429:
                        self.append_progress_text(f"  - 요청 제한. 5초 대기...")
                        time.sleep(5)
                        continue
                    
                    if response.status_code != 200:
                        self.append_progress_text(f"  - 페이지 {page} 요청 실패: {response.status_code}")
                        consecutive_empty += 1
                        if consecutive_empty >= 3:
                            break
                        page += 1
                        continue
                    
                    data = response.json()
                    
                    if 'result' not in data or 'list' not in data['result']:
                        consecutive_empty += 1
                        if consecutive_empty >= 3:
                            break
                        page += 1
                        continue
                    
                    property_list = data['result']['list']
                    
                    if not property_list:
                        # 빈 페이지면 종료
                        if page == 0:
                            self.append_progress_text("  - 매물 정보가 없습니다.")
                        break
                    
                    # 첫 페이지만 로그 출력
                    if page == 0:
                        self.append_progress_text(f"  - {len(property_list)}개의 매물 발견")
                    
                    for item in property_list:
                        property_data = self.extract_property_data(item, page)
                        all_properties.append(property_data)
                    
                    # 다음 페이지 확인
                    has_next_page = data['result'].get('hasNextPage', False)
                    if not has_next_page:
                        break
                    
                    consecutive_empty = 0
                    page += 1
                    
                except requests.exceptions.Timeout:
                    consecutive_empty += 1
                    if consecutive_empty >= 3:
                        break
                    page += 1
                    time.sleep(3)
                    
                except Exception as e:
                    self.append_progress_text(f"  - 페이지 {page} 오류: {str(e)}")
                    consecutive_empty += 1
                    if consecutive_empty >= 3:
                        break
                    page += 1
                    time.sleep(3)
            
            if all_properties:
                self.append_progress_text(f"  - 총 {len(all_properties)}개 매물 수집 완료")
            
            return all_properties
            
        except Exception as e:
            self.append_progress_text(f"  - 매물 수집 중 오류: {str(e)}")
            return None
    
    def download_data(self, complex_number, complex_name):
        """선택한 단지의 매물 정보 다운로드 실행"""
        self.log(f"'{complex_name}' 단지(번호: {complex_number})의 매물 정보 수집을 시작합니다.")
        
        def download_worker():
            try:
                # 더 완전한 헤더 설정
                headers = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                    "Accept": "application/json, text/plain, */*",
                    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                    "Accept-Encoding": "gzip, deflate, br",
                    "Origin": "https://fin.land.naver.com",
                    "Referer": f"https://fin.land.naver.com/complexes/{complex_number}?ms=37.5146,127.1079,16&a=APT:PRE:ABYG:JGC&e=RETAIL",
                    "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
                    "Sec-Ch-Ua-Mobile": "?0",
                    "Sec-Ch-Ua-Platform": '"Windows"',
                    "Sec-Fetch-Dest": "empty",
                    "Sec-Fetch-Mode": "cors",
                    "Sec-Fetch-Site": "same-origin",
                    "Cache-Control": "no-cache",
                    "Pragma": "no-cache",
                    "Connection": "keep-alive"
                }
                
                # 세션 초기화 및 쿠키 설정
                if not hasattr(self, 'session'):
                    self.session = requests.Session()
                    # 먼저 단지 페이지 방문
                    complex_url = f"https://fin.land.naver.com/complexes/{complex_number}"
                    self.log("단지 페이지 방문 중...")
                    self.session.get(complex_url, headers={
                        "User-Agent": headers["User-Agent"],
                        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
                    })
                    time.sleep(3)  # 충분한 대기 시간
                
                all_properties = []
                page = 0
                consecutive_empty = 0
                max_pages = 100
                retry_count = 0
                max_retries = 5
                
                while page < max_pages:
                    url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
                    
                    self.log(f"페이지 {page} 데이터 요청 중...")
                    
                    # 페이지 간 더 긴 딜레이
                    if page > 0:
                        time.sleep(3)  # 3초로 증가
                    
                    try:
                        response = self.session.get(url, headers=headers, timeout=15)
                        
                        if response.status_code == 429:
                            retry_count += 1
                            if retry_count >= max_retries:
                                self.log("너무 많은 재시도. 10초 후 다른 방법 시도...")
                                time.sleep(10)
                                # 세션 재생성
                                self.session = requests.Session()
                                complex_url = f"https://fin.land.naver.com/complexes/{complex_number}"
                                self.session.get(complex_url, headers={
                                    "User-Agent": headers["User-Agent"],
                                    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
                                })
                                time.sleep(5)
                                retry_count = 0
                                continue
                            
                            wait_time = min(retry_count * 5, 30)  # 최대 30초까지 대기
                            self.log(f"요청 제한 발생. {wait_time}초 대기 후 재시도... (시도 {retry_count}/{max_retries})")
                            time.sleep(wait_time)
                            continue
                        
                        if response.status_code != 200:
                            self.log(f"페이지 {page} 요청 실패: 상태 코드 {response.status_code}")
                            consecutive_empty += 1
                            if consecutive_empty >= 3:
                                break
                            page += 1
                            continue
                        
                        # 성공 시 retry 카운터 리셋
                        retry_count = 0
                        
                        data = response.json()
                        
                        if 'result' not in data or 'list' not in data['result']:
                            self.log(f"페이지 {page} 데이터 구조 오류")
                            consecutive_empty += 1
                            if consecutive_empty >= 3:
                                break
                            page += 1
                            continue
                        
                        property_list = data['result']['list']
                        
                        if not property_list:
                            self.log(f"페이지 {page}에 매물이 없습니다. 수집 종료.")
                            break
                        
                        # 매물 정보 추출
                        self.log(f"페이지 {page}에서 {len(property_list)}개의 매물 발견")
                        for item in property_list:
                            property_data = self.extract_property_data(item, page)
                            all_properties.append(property_data)
                        
                        # 다음 페이지 확인
                        has_next_page = data['result'].get('hasNextPage', False)
                        if not has_next_page:
                            self.log("마지막 페이지입니다. 수집 완료.")
                            break
                        
                        consecutive_empty = 0
                        page += 1
                        
                    except requests.exceptions.Timeout:
                        self.log(f"페이지 {page} 요청 시간 초과")
                        consecutive_empty += 1
                        if consecutive_empty >= 3:
                            break
                        page += 1
                        time.sleep(5)
                        
                    except Exception as e:
                        self.log(f"페이지 {page} 처리 중 오류: {str(e)}")
                        consecutive_empty += 1
                        if consecutive_empty >= 3:
                            break
                        page += 1
                        time.sleep(5)
                
                if not all_properties:
                    self.log("매물 정보를 수집하지 못했습니다. 다른 방법을 시도해보세요.")
                    self.root.after(0, lambda: messagebox.showwarning("경고", 
                        "매물 정보를 수집하지 못했습니다.\n"
                        "다음 방법을 시도해보세요:\n"
                        "1. 5-10분 후 다시 시도\n"
                        "2. VPN 사용\n"
                        "3. 다른 네트워크에서 시도"))
                else:
                    self.log(f"총 {len(all_properties)}개의 매물 정보를 수집했습니다.")
                    self.root.after(0, lambda: self.process_results(all_properties, complex_name))
                
            except Exception as e:
                self.log(f"데이터 수집 중 오류 발생: {str(e)}")
                self.root.after(0, lambda: messagebox.showerror("오류", f"데이터 수집 중 오류가 발생했습니다.\n{str(e)}"))
                
            finally:
                self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
        
        worker_thread = threading.Thread(target=download_worker, daemon=True)
        worker_thread.start()
    
    def fetch_page(self, complex_number, page, headers):
        """페이지별 데이터 요청 (병렬 처리용)"""
        url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                return None
            
            data = response.json()
            
            if 'result' in data and 'list' in data['result']:
                property_list = data['result']['list']
                
                if not property_list:
                    return None
                
                page_properties = []
                for item in property_list:
                    property_data = self.extract_property_data(item, page)
                    page_properties.append(property_data)
                
                return page_properties
        except Exception as e:
            self.log(f"페이지 {page} 요청 오류: {str(e)}")
            return None
        
        return None
    
    def extract_property_data(self, item, page):
        """매물 항목에서 필요한 정보 추출"""
        rep_info = item['representativeArticleInfo']
        
        property_data = {
            '단지명': rep_info.get('complexName', ''),
            '동': rep_info.get('dongName', ''),
            '거래유형': rep_info.get('tradeType', ''),
            '전용면적': rep_info['spaceInfo'].get('exclusiveSpace', ''),
            '타입구분': rep_info['spaceInfo'].get('nameType', ''),
            '층/전체층': rep_info['articleDetail'].get('floorInfo', ''),
            '방향': rep_info['articleDetail'].get('direction', ''),
            '매물특징': rep_info['articleDetail'].get('articleFeatureDescription', ''),
            '보증금': rep_info['priceInfo'].get('warrantyPrice', 0),
            '월세': rep_info['priceInfo'].get('rentPrice', 0),
            '매매가': rep_info['priceInfo'].get('dealPrice', 0),
            '프리미엄': rep_info['priceInfo'].get('premiumPrice', 0),
            '가격변동상태': rep_info['priceInfo'].get('priceChangeStatus', 0),
            '중개사명': rep_info['brokerInfo'].get('brokerageName', ''),
            'VR노출여부': rep_info['articleMediaDto'].get('isVrExposed', False) if rep_info.get('articleMediaDto') else False,
            '중개사 수': 0,
            '최소매매가': 0,
            '최대매매가': 0,
            '최소보증금': 0,
            '최대보증금': 0,
            '최소월세': 0,
            '최대월세': 0,
            '최소프리미엄': 0,
            '최대프리미엄': 0,
            '페이지번호': page
        }
        
        if 'duplicatedArticlesInfo' in item and item['duplicatedArticlesInfo'] is not None:
            property_data['중개사 수'] = item['duplicatedArticlesInfo'].get('realtorCount', 0)
            
            if 'representativePriceInfo' in item['duplicatedArticlesInfo']:
                price_info = item['duplicatedArticlesInfo']['representativePriceInfo']
                
                if 'dealPrice' in price_info:
                    property_data['최소매매가'] = price_info['dealPrice'].get('minPrice', 0)
                    property_data['최대매매가'] = price_info['dealPrice'].get('maxPrice', 0)
                
                if 'warrantyPrice' in price_info:
                    property_data['최소보증금'] = price_info['warrantyPrice'].get('minPrice', 0)
                    property_data['최대보증금'] = price_info['warrantyPrice'].get('maxPrice', 0)
                
                if 'rentPrice' in price_info:
                    property_data['최소월세'] = price_info['rentPrice'].get('minPrice', 0)
                    property_data['최대월세'] = price_info['rentPrice'].get('maxPrice', 0)
                
                if 'premiumPrice' in price_info:
                    property_data['최소프리미엄'] = price_info['premiumPrice'].get('minPrice', 0)
                    property_data['최대프리미엄'] = price_info['premiumPrice'].get('maxPrice', 0)
        
        return property_data
    
    def process_results(self, all_properties, complex_name):
        """수집된 데이터 처리 (메인 스레드에서 실행)"""
        try:
            if not all_properties:
                self.log("수집된 매물 정보가 없습니다.")
                messagebox.showinfo("알림", "수집된 매물 정보가 없습니다.")
                return
            
            df = pd.DataFrame(all_properties)
            
            trade_type_map = {
                'A1': '매매',
                'B1': '전세',
                'B2': '월세',
                'B3': '단기임대'
            }
            df['거래유형'] = df['거래유형'].map(lambda x: trade_type_map.get(x, x))
            
            direction_map = {
                'SS': '남향',
                'SE': '남동향',
                'SW': '남서향',
                'EE': '동향',
                'WW': '서향',
                'NN': '북향',
                'NE': '북동향',
                'NW': '북서향'
            }
            df['방향'] = df['방향'].map(lambda x: direction_map.get(x, x))
            
            price_change_map = {
                0: '변동없음',
                1: '가격상승',
                -1: '가격하락'
            }
            df['가격변동상태'] = df['가격변동상태'].map(lambda x: price_change_map.get(x, '알수없음'))
            
            money_columns = ['보증금', '월세', '매매가', '프리미엄', '최소매매가', '최대매매가', 
                           '최소보증금', '최대보증금', '최소월세', '최대월세', '최소프리미엄', '최대프리미엄']
            
            for col in money_columns:
                if col in df.columns:
                    df[col] = df[col].apply(lambda x: f"{x/10000:.0f}" if x > 0 else '')
            
            today = datetime.now().strftime('%Y%m%d')
            
            excel_filename = os.path.join(self.save_path, f'{complex_name}_매물정보_{today}.xlsx')
            
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='전체매물', index=False)
                self.apply_filter_to_sheet(writer.sheets['전체매물'], len(df.columns))
                
                deal_properties = df.query('매매가 != "" and 매매가.notnull()')
                if not deal_properties.empty:
                    deal_properties.to_excel(writer, sheet_name='매매매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['매매매물'], len(deal_properties.columns))
                
                full_deposit = df.query('보증금 != "" and 보증금.notnull() and (월세 == "" or 월세.isnull())')
                if not full_deposit.empty:
                    full_deposit.to_excel(writer, sheet_name='전세매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['전세매물'], len(full_deposit.columns))
                
                monthly_rent = df.query('월세 != "" and 월세.notnull()')
                if not monthly_rent.empty:
                    monthly_rent.to_excel(writer, sheet_name='월세매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['월세매물'], len(monthly_rent.columns))
                
                multi_realtor = df.query('`중개사 수` > 1')
                if not multi_realtor.empty:
                    multi_realtor.to_excel(writer, sheet_name='중복매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['중복매물'], len(multi_realtor.columns))
                
                for sheet_name in writer.sheets:
                    sheet = writer.sheets[sheet_name]
                    self.adjust_column_width(sheet)
            
            self.log(f"엑셀 파일 '{excel_filename}' 생성 완료!")
            self.log(f"총 {len(df)}개의 매물 정보가 저장되었습니다.")
            
            messagebox.showinfo("완료", f"'{complex_name}' 단지의 매물 정보가 '{excel_filename}' 파일로 저장되었습니다.")
        
        except Exception as e:
            self.log(f"결과 처리 중 오류: {str(e)}")
            messagebox.showerror("오류", f"결과 처리 중 오류가 발생했습니다.\n{str(e)}")
    
    def apply_filter_to_sheet(self, sheet, columns_count):
        """시트의 첫 행에 필터 적용"""
        try:
            sheet.auto_filter.ref = f"A1:{chr(64 + columns_count)}1"
            
            from openpyxl.styles import Font, PatternFill, Alignment
            
            header_font = Font(bold=True, size=11)
            header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
            
            for cell in sheet[1]:
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = Alignment(horizontal='center', vertical='center')
        except Exception as e:
            self.log(f"필터 적용 중 오류: {str(e)}")
    
    def adjust_column_width(self, sheet):
        """열 너비 자동 조정"""
        try:
            for column in sheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                
                for cell in column:
                    if cell.value:
                        try:
                            cell_length = len(str(cell.value))
                            max_length = max(max_length, cell_length)
                        except:
                            pass
                
                max_length = max(max_length, 10)
                max_length = min(max_length, 50)
                
                sheet.column_dimensions[column_letter].width = max_length + 2
        except Exception as e:
            self.log(f"열 너비 조정 중 오류: {str(e)}")
    
    def save_multi_search_results(self, summary_results, all_results=None):
        """다중 검색 결과를 엑셀로 저장"""
        try:
            today = datetime.now().strftime('%Y%m%d')
            filename = f'단지별_매물정보_{today}.xlsx'
            excel_filename = os.path.join(self.save_path, filename)
            
            summary_df = pd.DataFrame(summary_results)
            
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                summary_df.to_excel(writer, sheet_name='단지별_최저가', index=False)
                
                workbook = writer.book
                worksheet = writer.sheets['단지별_최저가']
                
                header_font = Font(bold=True, size=11)
                header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
                
                for cell in worksheet[1]:
                    cell.font = header_font
                    cell.fill = header_fill
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                
                for column in worksheet.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    
                    for cell in column:
                        if cell.value:
                            try:
                                cell_length = len(str(cell.value))
                                max_length = max(max_length, cell_length)
                            except:
                                pass
                    
                    max_length = max(max_length, 10)
                    max_length = min(max_length, 50)
                    worksheet.column_dimensions[column_letter].width = max_length + 2
                
                worksheet.auto_filter.ref = f"A1:{chr(64 + len(summary_df.columns))}1"
            
            self.append_progress_text(f"검색 완료: 총 {len(summary_df)}개 단지의 매물 정보가 저장되었습니다.")
            self.append_progress_text(f"파일 저장 경로: {excel_filename}")
            messagebox.showinfo("검색 완료", f"총 {len(summary_df)}개 단지의 매물 정보가 저장되었습니다.\n\n파일 저장 경로: {excel_filename}")
            
        except Exception as e:
            self.append_progress_text(f"결과 저장 중 오류: {str(e)}")
            messagebox.showerror("오류", f"결과 저장 중 오류가 발생했습니다.\n{str(e)}")
    
    def append_progress_text(self, text):
        """진행 상황 텍스트 추가 (스레드 안전)"""
        def _update():
            self.progress_text.config(state='normal')
            self.progress_text.insert(tk.END, text + "\n")
            self.progress_text.see(tk.END)
            self.progress_text.config(state='disabled')
        
        if threading.current_thread() is threading.main_thread():
            _update()
        else:
            self.root.after(0, _update)
    
    def clear_progress_text(self):
        """진행 상황 텍스트 초기화"""
        self.progress_text.config(state='normal')
        self.progress_text.delete('1.0', tk.END)
        self.progress_text.config(state='disabled')
    
    def update_progress(self, count):
        """프로그레스 상태 업데이트"""
        self.log(f"현재까지 {count}개 매물 수집됨")
    
    def log(self, message):
        """스레드 안전한 로깅"""
        self.log_queue.put(message)
    
    def process_log_queue(self):
        """로그 큐의 메시지를 처리"""
        try:
            while True:
                message = self.log_queue.get_nowait()
                
                print(message)
                
                if "단지 검색을 시작합니다" in message:
                    self.status_label.config(text="검색 중...")
                elif "매물 정보 수집을 시작합니다" in message:
                    self.status_label.config(text="매물 수집 중...")
                elif "총 " in message and "개의 매물 정보가 저장되었습니다" in message:
                    match = re.search(r'총 (\d+)개', message)
                    if match:
                        count = match.group(1)
                        self.status_label.config(text=f"{count}건 매물 검색 완료")
                elif "엑셀 파일" in message and "생성 완료" in message:
                    self.status_label.config(text="파일 저장 완료")
                
                self.log_queue.task_done()
        except queue.Empty:
            pass
        
        self.root.after(100, self.process_log_queue)
    
    def load_config(self):
        """설정 파일 로드"""
        try:
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if os.path.exists(config_path):
                self.config.read(config_path)
                if 'Settings' in self.config and 'save_path' in self.config['Settings']:
                    path = self.config['Settings']['save_path']
                    if os.path.exists(path):
                        self.save_path = path
        except Exception as e:
            print(f"설정 로드 오류: {str(e)}")
    
    def save_config(self):
        """설정 파일 저장"""
        try:
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if 'Settings' not in self.config:
                self.config['Settings'] = {}
            
            self.config['Settings']['save_path'] = self.save_path
            
            with open(config_path, 'w') as f:
                self.config.write(f)
        except Exception as e:
            print(f"설정 저장 오류: {str(e)}")
    
    def open_settings(self):
        """설정 창 열기"""
        path = filedialog.askdirectory(initialdir=self.save_path, title="저장할 폴더 선택")
        
        if path:
            self.save_path = path
            self.save_config()
            self.log(f"저장 경로가 변경되었습니다: {path}")

# 메인 함수
def main():
    root = tk.Tk()
    app = NaverRealEstateApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
import requests
import pandas as pd
import json
import time
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import threading
from datetime import datetime
import re
from tkinter import filedialog
import configparser
import os
import concurrent.futures
import queue
import webbrowser
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment

class ComplexSelectionDialog:
    def __init__(self, parent, title, complex_list):
        self.result = None
        
        # 대화상자 생성
        self.dialog = tk.Toplevel(parent)
        self.dialog.title(title)
        self.dialog.geometry("700x500")
        self.dialog.minsize(700, 500)
        self.dialog.transient(parent)
        self.dialog.grab_set()
    
        # 프레임 생성
        frame = ttk.Frame(self.dialog, padding="10")
        frame.pack(fill=tk.BOTH, expand=True)
        
        # 라벨 생성
        ttk.Label(frame, text="검색된 단지 목록. 선택하세요:").pack(anchor="w", pady=(0, 5))
        
        # 트리뷰 생성
        columns = ("name", "address")
        self.tree = ttk.Treeview(frame, columns=columns, show="headings", height=15)
        self.tree.heading("name", text="단지명")
        self.tree.heading("address", text="주소")
        
        self.tree.column("name", width=250)
        self.tree.column("address", width=400)
        
        # 스크롤바 생성
        scrollbar = ttk.Scrollbar(frame, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        
        self.tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # 데이터 추가
        for item in complex_list:
            self.tree.insert("", "end", values=(item["name"], item["address"]), tags=(item["id"],))
        
        # 버튼 프레임
        button_frame = ttk.Frame(self.dialog, padding="20")
        button_frame.pack(fill="x", pady=10)
        
        # 확인 버튼
        select_button = ttk.Button(button_frame, text="선택", command=self.on_select, width=10)
        select_button.pack(side="right", padx=10)
        
        # 취소 버튼
        cancel_button = ttk.Button(button_frame, text="취소", command=self.on_cancel, width=10)
        cancel_button.pack(side="right", padx=10)
        
        # 더블 클릭 이벤트 바인딩
        self.tree.bind("<Double-1>", lambda e: self.on_select())
        
        # 창 중앙 배치
        self.center_dialog()
        
        # 대화상자가 닫힐 때까지 대기
        parent.wait_window(self.dialog)
    
    def center_dialog(self):
        """대화상자를 화면 중앙에 배치"""
        self.dialog.update_idletasks()
        
        screen_width = self.dialog.winfo_screenwidth()
        screen_height = self.dialog.winfo_screenheight()
        
        dialog_width = self.dialog.winfo_width()
        dialog_height = self.dialog.winfo_height()
        
        x = (screen_width - dialog_width) // 2
        y = (screen_height - dialog_height) // 2
        
        self.dialog.geometry(f"+{x}+{y}")
    
    def on_select(self):
        selected_items = self.tree.selection()
        if selected_items:
            item_id = selected_items[0]
            item_values = self.tree.item(item_id)['values']
            item_tags = self.tree.item(item_id)['tags']
            
            self.result = {
                "name": item_values[0],
                "address": item_values[1],
                "id": item_tags[0]
            }
            
        self.dialog.destroy()
    
    def on_cancel(self):
        self.dialog.destroy()

class NaverRealEstateApp:
    def __init__(self, root):
        self.root = root
        self.root.title("부태리의 부동산 매물 수집기")
        self.root.geometry("650x600")  # 세로 길이 증가
        self.root.resizable(True, True)
        
        # 다크 테마 색상 설정
        self.bg_color = "#000000"  # 순수 검정색으로 변경
        self.fg_color = "#00bfff"  # 글자색 (하늘색)
        self.secondary_fg = "#87ceeb"  # 보조 글자색 (연한 하늘색)
        self.button_bg = "#0a0a0a"  # 버튼 배경색 (진한 검정)
        self.button_fg = "#00bfff"  # 버튼 글자색
        self.entry_bg = "#0a0a0a"  # 입력 필드 배경 (진한 검정)
        
        # 루트 윈도우 배경색 설정
        self.root.configure(bg=self.bg_color)
        
        # 스타일 설정
        style = ttk.Style()
        style.theme_use('clam')
        
        # 다크 테마 스타일 적용
        style.configure("TNotebook", background=self.bg_color, borderwidth=0)
        style.configure("TNotebook.Tab", background=self.button_bg, foreground=self.fg_color, padding=[20, 10])
        style.map("TNotebook.Tab", 
                 background=[("selected", "#000000")], 
                 foreground=[("selected", "#00ff00")])  # 선택된 탭은 밝은 녹색
        style.configure("TFrame", background=self.bg_color)
        style.configure("TLabelFrame", background=self.bg_color, foreground=self.fg_color, bordercolor=self.fg_color)
        style.configure("TLabelFrame.Label", background=self.bg_color, foreground=self.fg_color)
        style.configure("TLabel", background=self.bg_color, foreground=self.fg_color)
        style.configure("TButton", background=self.button_bg, foreground=self.button_fg, borderwidth=1, relief="flat")
        style.map("TButton", 
                 background=[("active", "#1a1a1a")],
                 foreground=[("active", "#00ff00")])
        style.configure("TEntry", fieldbackground=self.entry_bg, foreground="#00bfff", insertcolor="#00bfff", bordercolor="#00bfff")
        
        # 변수 초기화
        self.complex_data = None
        
        # 로그 큐 초기화
        self.log_queue = queue.Queue()
        
        # 설정 관리
        self.config = configparser.ConfigParser()
        self.save_path = os.path.expanduser("~/Documents")
        self.load_config()
        
        # 타이틀 라벨 추가 (상단 중앙)
        title_frame = tk.Frame(self.root, bg=self.bg_color)
        title_frame.pack(fill="x", pady=15)
        
        title_label = tk.Label(
            title_frame,
            text="부태리의 부동산 매물 수집기",
            font=("맑은 고딕", 18, "bold"),
            fg=self.fg_color,
            bg=self.bg_color
        )
        title_label.pack()
        
        subtitle_label = tk.Label(
            title_frame,
            text="네이버 부동산 매물 정보 자동 수집 프로그램",
            font=("맑은 고딕", 10),
            fg=self.secondary_fg,
            bg=self.bg_color
        )
        subtitle_label.pack()
        
        # 탭 컨트롤 생성
        self.tab_control = ttk.Notebook(self.root)
        
        # 탭 1: 단일 단지 검색
        self.tab1 = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab1, text='단일 단지 검색')
        
        # 탭 2: 다중 단지 검색
        self.tab2 = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab2, text='다중 단지 검색')
        
        self.tab_control.pack(expand=1, fill="both", padx=10)
        
        # UI 설정
        self.setup_single_search_ui()
        self.setup_multi_search_ui()
        
        # 상태 표시 레이블
        self.status_label = tk.Label(
            self.root, 
            text="단지명을 입력하고 검색하세요", 
            font=("맑은 고딕", 10),
            fg=self.secondary_fg,
            bg=self.bg_color,
            pady=10
        )
        self.status_label.pack(fill="x")
        
        # 제작자 정보
        self.setup_author_info()
        
        # 로그 큐 처리 시작
        self.root.after(100, self.process_log_queue)
    
    def setup_single_search_ui(self):
        """단일 단지 검색 UI 설정"""
        search_frame = ttk.LabelFrame(self.tab1, text="아파트 단지 검색")
        search_frame.pack(fill="x", padx=10, pady=5)
        
        ttk.Label(search_frame, text="단지명:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.search_entry = ttk.Entry(search_frame, width=40)
        self.search_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.search_entry.bind("<Return>", lambda event: self.start_search())
        
        self.search_button = ttk.Button(search_frame, text="검색 및 매물 수집", command=self.start_search)
        self.search_button.grid(row=0, column=2, padx=5, pady=5)
        
        self.settings_button = ttk.Button(search_frame, text="⚙", width=3, command=self.open_settings)
        self.settings_button.grid(row=0, column=3, padx=5, pady=5)
        
        search_frame.columnconfigure(1, weight=1)
    
    def setup_multi_search_ui(self):
        """다중 단지 검색 UI 설정"""
        main_frame = ttk.Frame(self.tab2, padding=10)
        main_frame.pack(fill="both", expand=True)
        
        left_frame = ttk.LabelFrame(main_frame, text="단지 목록 입력 (최대 50개)")
        left_frame.pack(side="left", fill="both", expand=True, padx=(0, 5))
        
        ttk.Label(left_frame, text="단지명을 한 줄에 하나씩 입력하세요:").pack(anchor="w", pady=(5, 0))
        
        # 스크롤 텍스트 완전 검정 테마 적용
        self.complex_text = scrolledtext.ScrolledText(
            left_frame, 
            width=30, 
            height=15,
            bg="#0a0a0a",  # 진한 검정
            fg="#00bfff",  # 파란색 텍스트
            insertbackground="#00bfff",
            selectbackground="#1a1a1a",
            selectforeground="#00ff00"
        )
        self.complex_text.pack(fill="both", expand=True, pady=5)
        
        right_frame = ttk.LabelFrame(main_frame, text="검색 옵션")
        right_frame.pack(side="right", fill="both", padx=(5, 0))
        
        ttk.Label(right_frame, text="전용면적 범위:").grid(row=0, column=0, sticky="w", pady=5)
        
        area_frame = ttk.Frame(right_frame)
        area_frame.grid(row=0, column=1, sticky="ew", pady=5)
        
        self.min_area_var = tk.StringVar()
        self.max_area_var = tk.StringVar()
        
        ttk.Entry(area_frame, textvariable=self.min_area_var, width=6).pack(side="left")
        ttk.Label(area_frame, text="㎡ ~").pack(side="left", padx=2)
        ttk.Entry(area_frame, textvariable=self.max_area_var, width=6).pack(side="left")
        ttk.Label(area_frame, text="㎡").pack(side="left", padx=2)
        
        ttk.Label(right_frame, text="층수 조건:").grid(row=1, column=0, sticky="w", pady=5)
        ttk.Label(right_frame, text="중층/고층/5층 이상 매물만 검색").grid(row=1, column=1, sticky="w", pady=5)
        
        ttk.Label(right_frame, text="거래 유형:").grid(row=2, column=0, sticky="w", pady=5)
        ttk.Label(right_frame, text="매매 및 전세 최저가 조사").grid(row=2, column=1, sticky="w", pady=5)
        
        button_frame = ttk.Frame(right_frame)
        button_frame.grid(row=3, column=0, columnspan=2, pady=10)
        
        self.multi_search_button = ttk.Button(button_frame, text="다중 검색 시작", command=self.start_multi_search, width=20)
        self.multi_search_button.pack(pady=5)
        
        # 진행상황 표시 완전 검정 테마 적용
        self.progress_text = scrolledtext.ScrolledText(
            right_frame, 
            width=30, 
            height=10, 
            state='disabled',
            bg="#0a0a0a",  # 진한 검정
            fg="#00ff00",  # 녹색 텍스트
            selectbackground="#1a1a1a"
        )
        self.progress_text.grid(row=4, column=0, columnspan=2, sticky="ew", pady=5)
    
    def setup_author_info(self):
        """제작자 정보 프레임 설정"""
        author_frame = ttk.Frame(self.root)
        author_frame.pack(side="bottom", fill="x", padx=10, pady=10)
        
        author_label = tk.Label(author_frame, text="만든이 부태리", font=("맑은 고딕", 9, "bold"), fg="black", cursor="hand2")
        author_label.pack(anchor="center")
        author_label.bind("<Button-1>", lambda e: self.open_blog())
        
        blog_label = tk.Label(author_frame, text="https://blog.naver.com/landlover333", font=("맑은 고딕", 8))
        blog_label.pack(anchor="center")
        blog_label.bind("<Button-1>", lambda e: self.open_blog())
    
    def open_blog(self):
        """블로그 링크 열기"""
        webbrowser.open("https://blog.naver.com/landlover333")
    
    def search_complex_by_api(self, keyword):
        """API를 사용하여 단지 검색"""
        try:
            # URL 인코딩
            import urllib.parse
            encoded_keyword = urllib.parse.quote(keyword)
            
            # 자동완성 API 호출
            url = f"https://fin.land.naver.com/front-api/v1/search/autocomplete/complexes?keyword={encoded_keyword}&size=10&page=0"
            
            # 더 완전한 헤더 설정으로 봇 감지 회피
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                "Accept": "application/json, text/plain, */*",
                "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                "Accept-Encoding": "gzip, deflate, br",
                "Origin": "https://fin.land.naver.com",
                "Referer": "https://fin.land.naver.com/",
                "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
                "Sec-Ch-Ua-Mobile": "?0",
                "Sec-Ch-Ua-Platform": '"Windows"',
                "Sec-Fetch-Dest": "empty",
                "Sec-Fetch-Mode": "cors",
                "Sec-Fetch-Site": "same-origin",
                "Cache-Control": "no-cache",
                "Pragma": "no-cache"
            }
            
            # 세션 사용으로 쿠키 유지
            if not hasattr(self, 'session'):
                self.session = requests.Session()
                # 먼저 메인 페이지 방문하여 쿠키 획득
                self.session.get("https://fin.land.naver.com/", headers=headers)
                time.sleep(0.5)
            
            self.log(f"API로 '{keyword}' 검색 중...")
            
            # 재시도 로직 추가
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = self.session.get(url, headers=headers, timeout=10)
                    
                    if response.status_code == 429:
                        # Rate limit 에러 시 대기 후 재시도
                        wait_time = (attempt + 1) * 2  # 2, 4, 6초 대기
                        self.log(f"요청 제한 발생. {wait_time}초 대기 후 재시도... (시도 {attempt+1}/{max_retries})")
                        time.sleep(wait_time)
                        continue
                    
                    if response.status_code != 200:
                        self.log(f"API 요청 실패: 상태 코드 {response.status_code}")
                        
                        # 대체 방법: 일반 검색 API 사용
                        if attempt == max_retries - 1:
                            return self.search_complex_alternative(keyword)
                        continue
                    
                    data = response.json()
                    
                    # 결과 파싱
                    if 'result' in data and 'list' in data['result']:
                        complex_list = []
                        for item in data['result']['list']:
                            complex_info = {
                                'name': item.get('complexName', ''),
                                'id': str(item.get('complexNumber', '')),
                                'address': item.get('addressName', '')
                            }
                            complex_list.append(complex_info)
                        
                        if complex_list:
                            return complex_list
                        else:
                            # 자동완성 결과가 없으면 대체 검색 시도
                            return self.search_complex_alternative(keyword)
                    else:
                        self.log("API 응답에 예상된 데이터가 없습니다.")
                        if attempt == max_retries - 1:
                            return self.search_complex_alternative(keyword)
                        
                except requests.exceptions.RequestException as e:
                    self.log(f"네트워크 오류 (시도 {attempt+1}/{max_retries}): {str(e)}")
                    if attempt < max_retries - 1:
                        time.sleep(2)
                    else:
                        return None
            
            return None
                
        except Exception as e:
            self.log(f"API 검색 중 오류: {str(e)}")
            return None
    
    def search_complex_alternative(self, keyword):
        """대체 검색 방법 - 일반 검색 API 사용"""
        try:
            import urllib.parse
            encoded_keyword = urllib.parse.quote(keyword)
            
            # 대체 URL 패턴들
            alternative_urls = [
                f"https://fin.land.naver.com/front-api/v1/search/complexes?keyword={encoded_keyword}&page=0&pageSize=20",
                f"https://fin.land.naver.com/front-api/v1/complex/search?query={encoded_keyword}"
            ]
            
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                "Accept": "application/json, text/plain, */*",
                "Referer": "https://fin.land.naver.com/",
                "Origin": "https://fin.land.naver.com"
            }
            
            self.log("대체 검색 방법 시도 중...")
            
            for url in alternative_urls:
                try:
                    if hasattr(self, 'session'):
                        response = self.session.get(url, headers=headers, timeout=10)
                    else:
                        response = requests.get(url, headers=headers, timeout=10)
                    
                    if response.status_code == 200:
                        data = response.json()
                        # 응답 구조에 따라 파싱 조정
                        if 'result' in data:
                            # 결과 처리 로직
                            complex_list = []
                            items = data['result'].get('list', []) or data['result'].get('items', [])
                            
                            for item in items:
                                complex_info = {
                                    'name': item.get('complexName', item.get('name', '')),
                                    'id': str(item.get('complexNumber', item.get('complexNo', ''))),
                                    'address': item.get('addressName', item.get('address', ''))
                                }
                                if complex_info['id']:  # ID가 있는 경우만 추가
                                    complex_list.append(complex_info)
                            
                            if complex_list:
                                self.log(f"대체 방법으로 {len(complex_list)}개 단지 발견")
                                return complex_list
                        
                except Exception as e:
                    continue
            
            # 모든 방법 실패 시 기본 Selenium 방식으로 전환
            self.log("API 검색 실패. 웹 스크래핑 방식으로 전환합니다...")
            return self.search_complex_with_selenium(keyword)
            
        except Exception as e:
            self.log(f"대체 검색 중 오류: {str(e)}")
            return None
    
    def search_complex_with_selenium(self, keyword):
        """Selenium을 사용한 폴백 검색 방법"""
        try:
            from selenium import webdriver
            from selenium.webdriver.common.by import By
            from selenium.webdriver.common.keys import Keys
            from selenium.webdriver.chrome.options import Options
            from selenium.webdriver.chrome.service import Service
            from webdriver_manager.chrome import ChromeDriverManager
            
            self.log("웹 브라우저 방식으로 검색 중...")
            
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("--disable-blink-features=AutomationControlled")
            chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
            chrome_options.add_experimental_option('useAutomationExtension', False)
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
            
            service = Service(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service, options=chrome_options)
            
            # 자동화 감지 방지 스크립트
            driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
                'source': '''
                    Object.defineProperty(navigator, 'webdriver', {
                        get: () => undefined
                    })
                '''
            })
            
            # 검색 수행
            search_url = f"https://fin.land.naver.com/search?q={keyword}"
            driver.get(search_url)
            time.sleep(3)
            
            # 검색 결과 파싱
            complex_list = []
            
            # 여러 가능한 선택자 시도
            selectors = [
                "a[href*='/complexes/']",
                ".search_list .item_link",
                ".complex_item a"
            ]
            
            for selector in selectors:
                try:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    if elements:
                        for element in elements[:10]:  # 최대 10개만
                            href = element.get_attribute('href')
                            if href and '/complexes/' in href:
                                # URL에서 단지 ID 추출
                                import re
                                match = re.search(r'/complexes/(\d+)', href)
                                if match:
                                    complex_id = match.group(1)
                                    complex_name = element.text.strip() or keyword
                                    
                                    complex_list.append({
                                        'name': complex_name,
                                        'id': complex_id,
                                        'address': ''
                                    })
                        
                        if complex_list:
                            break
                            
                except Exception as e:
                    continue
            
            driver.quit()
            
            if complex_list:
                self.log(f"웹 스크래핑으로 {len(complex_list)}개 단지 발견")
                return complex_list
            
            return None
            
        except Exception as e:
            self.log(f"Selenium 검색 중 오류: {str(e)}")
            return None
    
    def start_search(self):
        """단지명으로 검색 시작"""
        search_keyword = self.search_entry.get().strip()
        if not search_keyword:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        self.search_button.config(state=tk.DISABLED)
        threading.Thread(target=self.search_complex, args=(search_keyword,), daemon=True).start()
    
    def search_complex(self, search_keyword):
        """단지명으로 검색 및 단지번호 찾기 - API 방식"""
        try:
            self.log(f"'{search_keyword}' 단지 검색을 시작합니다...")
            
            # API로 단지 검색
            complex_list = self.search_complex_by_api(search_keyword)
            
            if not complex_list:
                self.log("검색 결과가 없습니다.")
                self.root.after(0, lambda: messagebox.showwarning("경고", "검색 결과가 없습니다. 다른 검색어를 입력해보세요."))
                return
            
            self.log(f"{len(complex_list)}개의 단지를 찾았습니다.")
            
            if len(complex_list) == 1:
                # 단일 결과 - 바로 사용
                selected_complex = complex_list[0]
                self.complex_data = selected_complex
                self.log(f"단지 선택: {selected_complex['name']} (번호: {selected_complex['id']})")
                self.download_data(selected_complex['id'], selected_complex['name'])
            else:
                # 여러 결과 - 선택 대화상자 표시
                def show_selection_dialog():
                    dialog = ComplexSelectionDialog(self.root, "단지 선택", complex_list)
                    selected_complex = dialog.result
                    
                    if selected_complex and selected_complex["id"]:
                        self.complex_data = selected_complex
                        self.log(f"단지 선택: {selected_complex['name']} (번호: {selected_complex['id']})")
                        # 다시 스레드로 실행
                        threading.Thread(target=self.download_data, 
                                       args=(selected_complex['id'], selected_complex['name']), 
                                       daemon=True).start()
                    else:
                        self.log("단지 선택이 취소되었습니다.")
                        self.search_button.config(state=tk.NORMAL)
                
                # 메인 스레드에서 대화상자 표시
                self.root.after(0, show_selection_dialog)
                
        except Exception as e:
            self.log(f"오류 발생: {str(e)}")
            self.root.after(0, lambda: messagebox.showerror("오류", f"검색 중 오류가 발생했습니다.\n{str(e)}"))
        finally:
            # 대화상자가 표시되는 경우가 아니면 버튼 활성화
            if not complex_list or len(complex_list) == 1:
                self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
    
    def find_complex_id(self, complex_name):
        """다중 검색용 단지번호 찾기 - API 방식"""
        try:
            self.append_progress_text(f"  - '{complex_name}' 단지 검색 중...")
            
            complex_list = self.search_complex_by_api(complex_name)
            
            if not complex_list:
                return None
            
            # 첫 번째 결과 사용 (다중 검색에서는 자동 선택)
            return complex_list[0]['id']
            
        except Exception as e:
            self.append_progress_text(f"  - 단지번호 검색 중 오류: {str(e)}")
            return None
    
    def start_multi_search(self):
        """다중 단지 검색 시작"""
        complex_text = self.complex_text.get('1.0', tk.END).strip()
        complex_list = [name.strip() for name in complex_text.split('\n') if name.strip()]
        
        if len(complex_list) > 50:
            messagebox.showwarning("경고", "최대 50개까지만 검색 가능합니다. 처음 50개만 처리합니다.")
            complex_list = complex_list[:50]
        
        if not complex_list:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        min_area = self.min_area_var.get().strip()
        max_area = self.max_area_var.get().strip()
        
        try:
            min_area = float(min_area) if min_area else None
            max_area = float(max_area) if max_area else None
        except ValueError:
            messagebox.showwarning("경고", "전용면적은 숫자로 입력하세요.")
            return
        
        search_options = {
            'min_area': min_area,
            'max_area': max_area
        }
        
        self.clear_progress_text()
        self.append_progress_text(f"총 {len(complex_list)}개 단지 검색을 시작합니다.\n")
        
        self.multi_search_button.config(state=tk.DISABLED)
        
        threading.Thread(target=self.process_multi_search, 
                        args=(complex_list, search_options), 
                        daemon=True).start()
    
    def process_multi_search(self, complex_list, search_options):
        """다중 단지 검색 처리"""
        try:
            all_results = []
            summary_results = []
            
            # 세션 초기화 (한 번만)
            if not hasattr(self, 'session'):
                self.session = requests.Session()
                # 메인 페이지 방문
                try:
                    self.session.get("https://fin.land.naver.com/", timeout=10)
                    time.sleep(2)
                except:
                    pass
            
            for idx, complex_name in enumerate(complex_list):
                self.append_progress_text(f"\n[{idx+1}/{len(complex_list)}] '{complex_name}' 검색 중...")
                
                # 각 단지 사이에 충분한 대기 시간
                if idx > 0:
                    self.append_progress_text("  - 다음 단지 검색 전 5초 대기...")
                    time.sleep(5)
                
                # API로 단지 정보 검색
                complex_id = self.find_complex_id(complex_name)
                
                if not complex_id:
                    self.append_progress_text(f"  - 단지를 찾을 수 없습니다.")
                    continue
                
                self.append_progress_text(f"  - 단지번호: {complex_id}")
                
                # 매물 정보 수집 (개선된 방식 사용)
                property_data = self.collect_property_data(complex_id, complex_name, search_options)
                
                if not property_data:
                    self.append_progress_text(f"  - 매물 정보가 없거나 수집 실패")
                    continue
                
                df = pd.DataFrame(property_data)
                
                all_results.append({
                    'complex_name': complex_name,
                    'complex_id': complex_id,
                    'properties': df
                })
                
                # 최저가 찾기 로직은 기존과 동일
                # ... (기존 최저가 찾기 코드 유지)
                
                # 전용면적 필터링
                if search_options['min_area'] or search_options['max_area']:
                    area_conditions = []
                    if search_options['min_area']:
                        area_conditions.append(f"전용면적 >= {search_options['min_area']}")
                    if search_options['max_area']:
                        area_conditions.append(f"전용면적 <= {search_options['max_area']}")
                    
                    area_query = " and ".join(area_conditions)
                    try:
                        df = df.query(area_query)
                    except Exception as e:
                        self.append_progress_text(f"  - 전용면적 필터링 오류: {str(e)}")
                
                # 층수 필터링
                try:
                    df = df.copy()
                    
                    def is_high_floor(floor_info):
                        floor_str = str(floor_info)
                        if pd.isna(floor_info) or floor_str.strip() == '':
                            return False
                        if '저' in floor_str:
                            return False
                        if '중' in floor_str or '고' in floor_str:
                            return True
                        parts = floor_str.split('/')
                        if len(parts) >= 1:
                            try:
                                floor_nums = re.findall(r'\d+', parts[0])
                                if floor_nums:
                                    floor_num = int(floor_nums[0])
                                    return floor_num >= 5
                            except:
                                pass
                        return False
                    
                    df['is_high_floor'] = df['층/전체층'].apply(is_high_floor)
                    df_high = df[df['is_high_floor']]
                    
                    if not df_high.empty:
                        df = df_high
                
                except Exception as e:
                    self.append_progress_text(f"  - 층수 필터링 오류: {str(e)}")
                
                # 매매/전세 최저가 찾기
                summary = {
                    '단지명': complex_name,
                    '단지ID': complex_id,
                    '전용면적_Min': search_options.get('min_area', ''),
                    '전용면적_Max': search_options.get('max_area', '')
                }
                
                # 매매 최저가
                try:
                    df_deal = df[df['거래유형'] == 'A1']
                    if not df_deal.empty:
                        df_deal = df_deal.copy()
                        df_deal.loc[:, '매매가_숫자'] = pd.to_numeric(df_deal['매매가'], errors='coerce')
                        df_deal = df_deal.dropna(subset=['매매가_숫자'])
                        
                        if not df_deal.empty:
                            df_deal = df_deal.sort_values('매매가_숫자')
                            deal_min = df_deal.iloc[0]
                            
                            summary['매매가'] = float(deal_min['매매가']) / 10000
                            summary['매매_동'] = deal_min['동']
                            summary['매매_층'] = deal_min['층/전체층']
                            summary['매매_면적'] = deal_min['전용면적']
                            
                            self.append_progress_text(f"  - 매매 최저가: {summary['매매가']:.1f}만원")
                    else:
                        summary['매매가'] = ''
                        summary['매매_동'] = ''
                        summary['매매_층'] = ''
                        summary['매매_면적'] = ''
                except Exception as e:
                    self.append_progress_text(f"  - 매매 최저가 계산 오류: {str(e)}")
                    summary['매매가'] = ''
                    summary['매매_동'] = ''
                    summary['매매_층'] = ''
                    summary['매매_면적'] = ''
                
                # 전세 최저가
                try:
                    df_jeonse = df[df['거래유형'] == 'B1']
                    if not df_jeonse.empty:
                        df_jeonse = df_jeonse.copy()
                        df_jeonse.loc[:, '보증금_숫자'] = pd.to_numeric(df_jeonse['보증금'], errors='coerce')
                        df_jeonse = df_jeonse.dropna(subset=['보증금_숫자'])
                        
                        if not df_jeonse.empty:
                            df_jeonse = df_jeonse.sort_values('보증금_숫자')
                            jeonse_min = df_jeonse.iloc[0]
                            
                            summary['전세가'] = float(jeonse_min['보증금']) / 10000
                            summary['전세_동'] = jeonse_min['동']
                            summary['전세_층'] = jeonse_min['층/전체층']
                            summary['전세_면적'] = jeonse_min['전용면적']
                            
                            self.append_progress_text(f"  - 전세 최저가: {summary['전세가']:.1f}만원")
                    else:
                        summary['전세가'] = ''
                        summary['전세_동'] = ''
                        summary['전세_층'] = ''
                        summary['전세_면적'] = ''
                except Exception as e:
                    self.append_progress_text(f"  - 전세 최저가 계산 오류: {str(e)}")
                    summary['전세가'] = ''
                    summary['전세_동'] = ''
                    summary['전세_층'] = ''
                    summary['전세_면적'] = ''
                
                summary_results.append(summary)
            
            if summary_results:
                self.save_multi_search_results(summary_results, all_results)
            else:
                self.append_progress_text("\n검색 완료: 조건에 맞는 매물이 없습니다.")
                messagebox.showinfo("검색 완료", "조건에 맞는 매물이 없습니다.")
        
        except Exception as e:
            self.append_progress_text(f"\n오류 발생: {str(e)}")
            messagebox.showerror("오류", f"다중 검색 중 오류가 발생했습니다.\n{str(e)}")
        
        finally:
            self.root.after(0, lambda: self.multi_search_button.config(state=tk.NORMAL))
    
    def collect_property_data(self, complex_number, complex_name, search_options):
        """단지의 매물 정보 수집 (다중 검색용 - 단일 검색과 동일한 방식)"""
        try:
            # 더 완전한 헤더 설정 (단일 검색과 동일)
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                "Accept": "application/json, text/plain, */*",
                "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                "Accept-Encoding": "gzip, deflate, br",
                "Origin": "https://fin.land.naver.com",
                "Referer": f"https://fin.land.naver.com/complexes/{complex_number}?ms=37.5146,127.1079,16&a=APT:PRE:ABYG:JGC&e=RETAIL",
                "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
                "Sec-Ch-Ua-Mobile": "?0",
                "Sec-Ch-Ua-Platform": '"Windows"',
                "Sec-Fetch-Dest": "empty",
                "Sec-Fetch-Mode": "cors",
                "Sec-Fetch-Site": "same-origin",
                "Cache-Control": "no-cache",
                "Pragma": "no-cache",
                "Connection": "keep-alive"
            }
            
            # 세션 초기화 또는 재사용
            if not hasattr(self, 'session'):
                self.session = requests.Session()
            
            # 단지 페이지 먼저 방문 (정상 사용자처럼)
            complex_url = f"https://fin.land.naver.com/complexes/{complex_number}"
            try:
                self.session.get(complex_url, headers={
                    "User-Agent": headers["User-Agent"],
                    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
                }, timeout=10)
                time.sleep(2)  # 대기
            except:
                pass  # 실패해도 계속 진행
            
            all_properties = []
            page = 0
            consecutive_empty = 0
            max_pages = 30  # 다중 검색은 페이지 제한을 적게
            retry_count = 0
            max_retries = 3  # 다중 검색은 재시도 횟수 줄임
            
            while page < max_pages:
                url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
                
                # 페이지 간 딜레이 (다중 검색은 더 보수적으로)
                if page > 0:
                    time.sleep(4)  # 4초 대기
                
                try:
                    response = self.session.get(url, headers=headers, timeout=15)
                    
                    if response.status_code == 429:
                        retry_count += 1
                        if retry_count >= max_retries:
                            self.append_progress_text(f"  - 요청 제한 초과. 이 단지는 건너뜁니다.")
                            return None
                        
                        wait_time = min(retry_count * 5, 20)  # 최대 20초
                        self.append_progress_text(f"  - 요청 제한. {wait_time}초 대기... (시도 {retry_count}/{max_retries})")
                        time.sleep(wait_time)
                        continue
                    
                    if response.status_code != 200:
                        consecutive_empty += 1
                        if consecutive_empty >= 2:  # 다중 검색은 빠르게 포기
                            break
                        page += 1
                        continue
                    
                    # 성공 시 retry 카운터 리셋
                    retry_count = 0
                    
                    data = response.json()
                    
                    if 'result' not in data or 'list' not in data['result']:
                        consecutive_empty += 1
                        if consecutive_empty >= 2:
                            break
                        page += 1
                        continue
                    
                    property_list = data['result']['list']
                    
                    if not property_list:
                        # 빈 페이지면 종료
                        if page == 0:
                            self.append_progress_text("  - 매물 정보가 없습니다.")
                        break
                    
                    # 첫 페이지만 로그 출력
                    if page == 0:
                        self.append_progress_text(f"  - {len(property_list)}개의 매물 발견")
                    
                    for item in property_list:
                        property_data = self.extract_property_data(item, page)
                        all_properties.append(property_data)
                    
                    # 다음 페이지 확인
                    has_next_page = data['result'].get('hasNextPage', False)
                    if not has_next_page:
                        break
                    
                    consecutive_empty = 0
                    page += 1
                    
                except requests.exceptions.Timeout:
                    consecutive_empty += 1
                    if consecutive_empty >= 2:
                        break
                    page += 1
                    time.sleep(5)
                    
                except Exception as e:
                    self.append_progress_text(f"  - 페이지 {page} 오류: {str(e)}")
                    consecutive_empty += 1
                    if consecutive_empty >= 2:
                        break
                    page += 1
                    time.sleep(5)
            
            if all_properties:
                self.append_progress_text(f"  - 총 {len(all_properties)}개 매물 수집 완료")
            
            return all_properties
            
        except Exception as e:
            self.append_progress_text(f"  - 매물 수집 중 오류: {str(e)}")
            return None
    
    def download_data(self, complex_number, complex_name):
        """선택한 단지의 매물 정보 다운로드 실행"""
        self.log(f"'{complex_name}' 단지(번호: {complex_number})의 매물 정보 수집을 시작합니다.")
        
        def download_worker():
            try:
                # 더 완전한 헤더 설정
                headers = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                    "Accept": "application/json, text/plain, */*",
                    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                    "Accept-Encoding": "gzip, deflate, br",
                    "Origin": "https://fin.land.naver.com",
                    "Referer": f"https://fin.land.naver.com/complexes/{complex_number}?ms=37.5146,127.1079,16&a=APT:PRE:ABYG:JGC&e=RETAIL",
                    "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
                    "Sec-Ch-Ua-Mobile": "?0",
                    "Sec-Ch-Ua-Platform": '"Windows"',
                    "Sec-Fetch-Dest": "empty",
                    "Sec-Fetch-Mode": "cors",
                    "Sec-Fetch-Site": "same-origin",
                    "Cache-Control": "no-cache",
                    "Pragma": "no-cache",
                    "Connection": "keep-alive"
                }
                
                # 세션 초기화 및 쿠키 설정
                if not hasattr(self, 'session'):
                    self.session = requests.Session()
                    # 먼저 단지 페이지 방문
                    complex_url = f"https://fin.land.naver.com/complexes/{complex_number}"
                    self.log("단지 페이지 방문 중...")
                    self.session.get(complex_url, headers={
                        "User-Agent": headers["User-Agent"],
                        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
                    })
                    time.sleep(3)  # 충분한 대기 시간
                
                all_properties = []
                page = 0
                consecutive_empty = 0
                max_pages = 100
                retry_count = 0
                max_retries = 5
                
                while page < max_pages:
                    url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
                    
                    self.log(f"페이지 {page} 데이터 요청 중...")
                    
                    # 페이지 간 더 긴 딜레이
                    if page > 0:
                        time.sleep(3)  # 3초로 증가
                    
                    try:
                        response = self.session.get(url, headers=headers, timeout=15)
                        
                        if response.status_code == 429:
                            retry_count += 1
                            if retry_count >= max_retries:
                                self.log("너무 많은 재시도. 10초 후 다른 방법 시도...")
                                time.sleep(10)
                                # 세션 재생성
                                self.session = requests.Session()
                                complex_url = f"https://fin.land.naver.com/complexes/{complex_number}"
                                self.session.get(complex_url, headers={
                                    "User-Agent": headers["User-Agent"],
                                    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
                                })
                                time.sleep(5)
                                retry_count = 0
                                continue
                            
                            wait_time = min(retry_count * 5, 30)  # 최대 30초까지 대기
                            self.log(f"요청 제한 발생. {wait_time}초 대기 후 재시도... (시도 {retry_count}/{max_retries})")
                            time.sleep(wait_time)
                            continue
                        
                        if response.status_code != 200:
                            self.log(f"페이지 {page} 요청 실패: 상태 코드 {response.status_code}")
                            consecutive_empty += 1
                            if consecutive_empty >= 3:
                                break
                            page += 1
                            continue
                        
                        # 성공 시 retry 카운터 리셋
                        retry_count = 0
                        
                        data = response.json()
                        
                        if 'result' not in data or 'list' not in data['result']:
                            self.log(f"페이지 {page} 데이터 구조 오류")
                            consecutive_empty += 1
                            if consecutive_empty >= 3:
                                break
                            page += 1
                            continue
                        
                        property_list = data['result']['list']
                        
                        if not property_list:
                            self.log(f"페이지 {page}에 매물이 없습니다. 수집 종료.")
                            break
                        
                        # 매물 정보 추출
                        self.log(f"페이지 {page}에서 {len(property_list)}개의 매물 발견")
                        for item in property_list:
                            property_data = self.extract_property_data(item, page)
                            all_properties.append(property_data)
                        
                        # 다음 페이지 확인
                        has_next_page = data['result'].get('hasNextPage', False)
                        if not has_next_page:
                            self.log("마지막 페이지입니다. 수집 완료.")
                            break
                        
                        consecutive_empty = 0
                        page += 1
                        
                    except requests.exceptions.Timeout:
                        self.log(f"페이지 {page} 요청 시간 초과")
                        consecutive_empty += 1
                        if consecutive_empty >= 3:
                            break
                        page += 1
                        time.sleep(5)
                        
                    except Exception as e:
                        self.log(f"페이지 {page} 처리 중 오류: {str(e)}")
                        consecutive_empty += 1
                        if consecutive_empty >= 3:
                            break
                        page += 1
                        time.sleep(5)
                
                if not all_properties:
                    self.log("매물 정보를 수집하지 못했습니다. 다른 방법을 시도해보세요.")
                    self.root.after(0, lambda: messagebox.showwarning("경고", 
                        "매물 정보를 수집하지 못했습니다.\n"
                        "다음 방법을 시도해보세요:\n"
                        "1. 5-10분 후 다시 시도\n"
                        "2. VPN 사용\n"
                        "3. 다른 네트워크에서 시도"))
                else:
                    self.log(f"총 {len(all_properties)}개의 매물 정보를 수집했습니다.")
                    self.root.after(0, lambda: self.process_results(all_properties, complex_name))
                
            except Exception as e:
                self.log(f"데이터 수집 중 오류 발생: {str(e)}")
                self.root.after(0, lambda: messagebox.showerror("오류", f"데이터 수집 중 오류가 발생했습니다.\n{str(e)}"))
                
            finally:
                self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
        
        worker_thread = threading.Thread(target=download_worker, daemon=True)
        worker_thread.start()
    
    def fetch_page(self, complex_number, page, headers):
        """페이지별 데이터 요청 (병렬 처리용)"""
        url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                return None
            
            data = response.json()
            
            if 'result' in data and 'list' in data['result']:
                property_list = data['result']['list']
                
                if not property_list:
                    return None
                
                page_properties = []
                for item in property_list:
                    property_data = self.extract_property_data(item, page)
                    page_properties.append(property_data)
                
                return page_properties
        except Exception as e:
            self.log(f"페이지 {page} 요청 오류: {str(e)}")
            return None
        
        return None
    
    def extract_property_data(self, item, page):
        """매물 항목에서 필요한 정보 추출"""
        rep_info = item['representativeArticleInfo']
        
        property_data = {
            '단지명': rep_info.get('complexName', ''),
            '동': rep_info.get('dongName', ''),
            '거래유형': rep_info.get('tradeType', ''),
            '전용면적': rep_info['spaceInfo'].get('exclusiveSpace', ''),
            '타입구분': rep_info['spaceInfo'].get('nameType', ''),
            '층/전체층': rep_info['articleDetail'].get('floorInfo', ''),
            '방향': rep_info['articleDetail'].get('direction', ''),
            '매물특징': rep_info['articleDetail'].get('articleFeatureDescription', ''),
            '보증금': rep_info['priceInfo'].get('warrantyPrice', 0),
            '월세': rep_info['priceInfo'].get('rentPrice', 0),
            '매매가': rep_info['priceInfo'].get('dealPrice', 0),
            '프리미엄': rep_info['priceInfo'].get('premiumPrice', 0),
            '가격변동상태': rep_info['priceInfo'].get('priceChangeStatus', 0),
            '중개사명': rep_info['brokerInfo'].get('brokerageName', ''),
            'VR노출여부': rep_info['articleMediaDto'].get('isVrExposed', False) if rep_info.get('articleMediaDto') else False,
            '중개사 수': 0,
            '최소매매가': 0,
            '최대매매가': 0,
            '최소보증금': 0,
            '최대보증금': 0,
            '최소월세': 0,
            '최대월세': 0,
            '최소프리미엄': 0,
            '최대프리미엄': 0,
            '페이지번호': page
        }
        
        if 'duplicatedArticlesInfo' in item and item['duplicatedArticlesInfo'] is not None:
            property_data['중개사 수'] = item['duplicatedArticlesInfo'].get('realtorCount', 0)
            
            if 'representativePriceInfo' in item['duplicatedArticlesInfo']:
                price_info = item['duplicatedArticlesInfo']['representativePriceInfo']
                
                if 'dealPrice' in price_info:
                    property_data['최소매매가'] = price_info['dealPrice'].get('minPrice', 0)
                    property_data['최대매매가'] = price_info['dealPrice'].get('maxPrice', 0)
                
                if 'warrantyPrice' in price_info:
                    property_data['최소보증금'] = price_info['warrantyPrice'].get('minPrice', 0)
                    property_data['최대보증금'] = price_info['warrantyPrice'].get('maxPrice', 0)
                
                if 'rentPrice' in price_info:
                    property_data['최소월세'] = price_info['rentPrice'].get('minPrice', 0)
                    property_data['최대월세'] = price_info['rentPrice'].get('maxPrice', 0)
                
                if 'premiumPrice' in price_info:
                    property_data['최소프리미엄'] = price_info['premiumPrice'].get('minPrice', 0)
                    property_data['최대프리미엄'] = price_info['premiumPrice'].get('maxPrice', 0)
        
        return property_data
    
    def process_results(self, all_properties, complex_name):
        """수집된 데이터 처리 (메인 스레드에서 실행)"""
        try:
            if not all_properties:
                self.log("수집된 매물 정보가 없습니다.")
                messagebox.showinfo("알림", "수집된 매물 정보가 없습니다.")
                return
            
            df = pd.DataFrame(all_properties)
            
            trade_type_map = {
                'A1': '매매',
                'B1': '전세',
                'B2': '월세',
                'B3': '단기임대'
            }
            df['거래유형'] = df['거래유형'].map(lambda x: trade_type_map.get(x, x))
            
            direction_map = {
                'SS': '남향',
                'SE': '남동향',
                'SW': '남서향',
                'EE': '동향',
                'WW': '서향',
                'NN': '북향',
                'NE': '북동향',
                'NW': '북서향'
            }
            df['방향'] = df['방향'].map(lambda x: direction_map.get(x, x))
            
            price_change_map = {
                0: '변동없음',
                1: '가격상승',
                -1: '가격하락'
            }
            df['가격변동상태'] = df['가격변동상태'].map(lambda x: price_change_map.get(x, '알수없음'))
            
            money_columns = ['보증금', '월세', '매매가', '프리미엄', '최소매매가', '최대매매가', 
                           '최소보증금', '최대보증금', '최소월세', '최대월세', '최소프리미엄', '최대프리미엄']
            
            for col in money_columns:
                if col in df.columns:
                    df[col] = df[col].apply(lambda x: f"{x/10000:.0f}" if x > 0 else '')
            
            today = datetime.now().strftime('%Y%m%d')
            
            excel_filename = os.path.join(self.save_path, f'{complex_name}_매물정보_{today}.xlsx')
            
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='전체매물', index=False)
                self.apply_filter_to_sheet(writer.sheets['전체매물'], len(df.columns))
                
                deal_properties = df.query('매매가 != "" and 매매가.notnull()')
                if not deal_properties.empty:
                    deal_properties.to_excel(writer, sheet_name='매매매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['매매매물'], len(deal_properties.columns))
                
                full_deposit = df.query('보증금 != "" and 보증금.notnull() and (월세 == "" or 월세.isnull())')
                if not full_deposit.empty:
                    full_deposit.to_excel(writer, sheet_name='전세매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['전세매물'], len(full_deposit.columns))
                
                monthly_rent = df.query('월세 != "" and 월세.notnull()')
                if not monthly_rent.empty:
                    monthly_rent.to_excel(writer, sheet_name='월세매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['월세매물'], len(monthly_rent.columns))
                
                multi_realtor = df.query('`중개사 수` > 1')
                if not multi_realtor.empty:
                    multi_realtor.to_excel(writer, sheet_name='중복매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['중복매물'], len(multi_realtor.columns))
                
                for sheet_name in writer.sheets:
                    sheet = writer.sheets[sheet_name]
                    self.adjust_column_width(sheet)
            
            self.log(f"엑셀 파일 '{excel_filename}' 생성 완료!")
            self.log(f"총 {len(df)}개의 매물 정보가 저장되었습니다.")
            
            messagebox.showinfo("완료", f"'{complex_name}' 단지의 매물 정보가 '{excel_filename}' 파일로 저장되었습니다.")
        
        except Exception as e:
            self.log(f"결과 처리 중 오류: {str(e)}")
            messagebox.showerror("오류", f"결과 처리 중 오류가 발생했습니다.\n{str(e)}")
    
    def apply_filter_to_sheet(self, sheet, columns_count):
        """시트의 첫 행에 필터 적용"""
        try:
            sheet.auto_filter.ref = f"A1:{chr(64 + columns_count)}1"
            
            from openpyxl.styles import Font, PatternFill, Alignment
            
            header_font = Font(bold=True, size=11)
            header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
            
            for cell in sheet[1]:
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = Alignment(horizontal='center', vertical='center')
        except Exception as e:
            self.log(f"필터 적용 중 오류: {str(e)}")
    
    def adjust_column_width(self, sheet):
        """열 너비 자동 조정"""
        try:
            for column in sheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                
                for cell in column:
                    if cell.value:
                        try:
                            cell_length = len(str(cell.value))
                            max_length = max(max_length, cell_length)
                        except:
                            pass
                
                max_length = max(max_length, 10)
                max_length = min(max_length, 50)
                
                sheet.column_dimensions[column_letter].width = max_length + 2
        except Exception as e:
            self.log(f"열 너비 조정 중 오류: {str(e)}")
    
    def save_multi_search_results(self, summary_results, all_results=None):
        """다중 검색 결과를 엑셀로 저장"""
        try:
            today = datetime.now().strftime('%Y%m%d')
            filename = f'단지별_매물정보_{today}.xlsx'
            excel_filename = os.path.join(self.save_path, filename)
            
            summary_df = pd.DataFrame(summary_results)
            
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                summary_df.to_excel(writer, sheet_name='단지별_최저가', index=False)
                
                workbook = writer.book
                worksheet = writer.sheets['단지별_최저가']
                
                header_font = Font(bold=True, size=11)
                header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
                
                for cell in worksheet[1]:
                    cell.font = header_font
                    cell.fill = header_fill
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                
                for column in worksheet.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    
                    for cell in column:
                        if cell.value:
                            try:
                                cell_length = len(str(cell.value))
                                max_length = max(max_length, cell_length)
                            except:
                                pass
                    
                    max_length = max(max_length, 10)
                    max_length = min(max_length, 50)
                    worksheet.column_dimensions[column_letter].width = max_length + 2
                
                worksheet.auto_filter.ref = f"A1:{chr(64 + len(summary_df.columns))}1"
            
            self.append_progress_text(f"검색 완료: 총 {len(summary_df)}개 단지의 매물 정보가 저장되었습니다.")
            self.append_progress_text(f"파일 저장 경로: {excel_filename}")
            messagebox.showinfo("검색 완료", f"총 {len(summary_df)}개 단지의 매물 정보가 저장되었습니다.\n\n파일 저장 경로: {excel_filename}")
            
        except Exception as e:
            self.append_progress_text(f"결과 저장 중 오류: {str(e)}")
            messagebox.showerror("오류", f"결과 저장 중 오류가 발생했습니다.\n{str(e)}")
    
    def append_progress_text(self, text):
        """진행 상황 텍스트 추가 (스레드 안전)"""
        def _update():
            self.progress_text.config(state='normal')
            self.progress_text.insert(tk.END, text + "\n")
            self.progress_text.see(tk.END)
            self.progress_text.config(state='disabled')
        
        if threading.current_thread() is threading.main_thread():
            _update()
        else:
            self.root.after(0, _update)
    
    def clear_progress_text(self):
        """진행 상황 텍스트 초기화"""
        self.progress_text.config(state='normal')
        self.progress_text.delete('1.0', tk.END)
        self.progress_text.config(state='disabled')
    
    def update_progress(self, count):
        """프로그레스 상태 업데이트"""
        self.log(f"현재까지 {count}개 매물 수집됨")
    
    def log(self, message):
        """스레드 안전한 로깅"""
        self.log_queue.put(message)
    
    def process_log_queue(self):
        """로그 큐의 메시지를 처리"""
        try:
            while True:
                message = self.log_queue.get_nowait()
                
                print(message)
                
                if "단지 검색을 시작합니다" in message:
                    self.status_label.config(text="검색 중...")
                elif "매물 정보 수집을 시작합니다" in message:
                    self.status_label.config(text="매물 수집 중...")
                elif "총 " in message and "개의 매물 정보가 저장되었습니다" in message:
                    match = re.search(r'총 (\d+)개', message)
                    if match:
                        count = match.group(1)
                        self.status_label.config(text=f"{count}건 매물 검색 완료")
                elif "엑셀 파일" in message and "생성 완료" in message:
                    self.status_label.config(text="파일 저장 완료")
                
                self.log_queue.task_done()
        except queue.Empty:
            pass
        
        self.root.after(100, self.process_log_queue)
    
    def load_config(self):
        """설정 파일 로드"""
        try:
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if os.path.exists(config_path):
                self.config.read(config_path)
                if 'Settings' in self.config and 'save_path' in self.config['Settings']:
                    path = self.config['Settings']['save_path']
                    if os.path.exists(path):
                        self.save_path = path
        except Exception as e:
            print(f"설정 로드 오류: {str(e)}")
    
    def save_config(self):
        """설정 파일 저장"""
        try:
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if 'Settings' not in self.config:
                self.config['Settings'] = {}
            
            self.config['Settings']['save_path'] = self.save_path
            
            with open(config_path, 'w') as f:
                self.config.write(f)
        except Exception as e:
            print(f"설정 저장 오류: {str(e)}")
    
    def open_settings(self):
        """설정 창 열기"""
        path = filedialog.askdirectory(initialdir=self.save_path, title="저장할 폴더 선택")
        
        if path:
            self.save_path = path
            self.save_config()
            self.log(f"저장 경로가 변경되었습니다: {path}")

# 메인 함수
def main():
    root = tk.Tk()
    app = NaverRealEstateApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

In [4]:
#다중 단지검색시 중, 고 층 및 5층 이상 매물 없으면 저층이라도 조사

In [ ]:
# 그래도 없으면  아파트명만 엑셀에 넣고 가격은 비워 두게 하기

In [ ]:
import requests
import pandas as pd
import json
import time
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import threading
from datetime import datetime
import re
from tkinter import filedialog
import configparser
import os
import concurrent.futures
import queue
import webbrowser
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment

class ComplexSelectionDialog:
    def __init__(self, parent, title, complex_list):
        self.result = None
        
        # 대화상자 생성
        self.dialog = tk.Toplevel(parent)
        self.dialog.title(title)
        self.dialog.geometry("700x500")
        self.dialog.minsize(700, 500)
        self.dialog.transient(parent)
        self.dialog.grab_set()
    
        # 프레임 생성
        frame = ttk.Frame(self.dialog, padding="10")
        frame.pack(fill=tk.BOTH, expand=True)
        
        # 라벨 생성
        ttk.Label(frame, text="검색된 단지 목록. 선택하세요:").pack(anchor="w", pady=(0, 5))
        
        # 트리뷰 생성
        columns = ("name", "address")
        self.tree = ttk.Treeview(frame, columns=columns, show="headings", height=15)
        self.tree.heading("name", text="단지명")
        self.tree.heading("address", text="주소")
        
        self.tree.column("name", width=250)
        self.tree.column("address", width=400)
        
        # 스크롤바 생성
        scrollbar = ttk.Scrollbar(frame, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        
        self.tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # 데이터 추가
        for item in complex_list:
            self.tree.insert("", "end", values=(item["name"], item["address"]), tags=(item["id"],))
        
        # 버튼 프레임
        button_frame = ttk.Frame(self.dialog, padding="20")
        button_frame.pack(fill="x", pady=10)
        
        # 확인 버튼
        select_button = ttk.Button(button_frame, text="선택", command=self.on_select, width=10)
        select_button.pack(side="right", padx=10)
        
        # 취소 버튼
        cancel_button = ttk.Button(button_frame, text="취소", command=self.on_cancel, width=10)
        cancel_button.pack(side="right", padx=10)
        
        # 더블 클릭 이벤트 바인딩
        self.tree.bind("<Double-1>", lambda e: self.on_select())
        
        # 창 중앙 배치
        self.center_dialog()
        
        # 대화상자가 닫힐 때까지 대기
        parent.wait_window(self.dialog)
    
    def center_dialog(self):
        """대화상자를 화면 중앙에 배치"""
        self.dialog.update_idletasks()
        
        screen_width = self.dialog.winfo_screenwidth()
        screen_height = self.dialog.winfo_screenheight()
        
        dialog_width = self.dialog.winfo_width()
        dialog_height = self.dialog.winfo_height()
        
        x = (screen_width - dialog_width) // 2
        y = (screen_height - dialog_height) // 2
        
        self.dialog.geometry(f"+{x}+{y}")
    
    def on_select(self):
        selected_items = self.tree.selection()
        if selected_items:
            item_id = selected_items[0]
            item_values = self.tree.item(item_id)['values']
            item_tags = self.tree.item(item_id)['tags']
            
            self.result = {
                "name": item_values[0],
                "address": item_values[1],
                "id": item_tags[0]
            }
            
        self.dialog.destroy()
    
    def on_cancel(self):
        self.dialog.destroy()

class NaverRealEstateApp:
    def __init__(self, root):
        self.root = root
        self.root.title("부태리의 부동산 매물 수집기")
        self.root.geometry("600x600")
        self.root.resizable(True, True)
        
        # 변수 초기화
        self.complex_data = None
        
        # 로그 큐 초기화
        self.log_queue = queue.Queue()
        
        # 설정 관리
        self.config = configparser.ConfigParser()
        self.save_path = os.path.expanduser("~/Documents")
        self.load_config()
        
        # 타이틀 라벨 추가 (상단 중앙)
        title_frame = ttk.Frame(self.root)
        title_frame.pack(fill="x", pady=10)
        
        title_label = tk.Label(
            title_frame,
            text="부태리의 부동산 매물 수집기",
            font=("맑은 고딕", 16, "bold"),
            fg="#2C3E50"
        )
        title_label.pack()
        
        subtitle_label = tk.Label(
            title_frame,
            text="네이버 부동산 매물 정보 자동 수집 프로그램",
            font=("맑은 고딕", 10),
            fg="#7F8C8D"
        )
        subtitle_label.pack()
        
        # 탭 컨트롤 생성
        self.tab_control = ttk.Notebook(self.root)
        
        # 탭 1: 단일 단지 검색
        self.tab1 = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab1, text='단일 단지 검색')
        
        # 탭 2: 다중 단지 검색
        self.tab2 = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab2, text='다중 단지 검색')
        
        self.tab_control.pack(expand=1, fill="both", padx=10)
        
        # UI 설정
        self.setup_single_search_ui()
        self.setup_multi_search_ui()
        
        # 상태 표시 레이블
        self.status_label = tk.Label(
            self.root, 
            text="단지명을 입력하고 검색하세요", 
            font=("맑은 고딕", 9),
            pady=10
        )
        self.status_label.pack(fill="x")
        
        # 제작자 정보
        self.setup_author_info()
        
        # 로그 큐 처리 시작
        self.root.after(100, self.process_log_queue)
    
    def setup_single_search_ui(self):
        """단일 단지 검색 UI 설정"""
        search_frame = ttk.LabelFrame(self.tab1, text="아파트 단지 검색")
        search_frame.pack(fill="x", padx=10, pady=5)
        
        ttk.Label(search_frame, text="단지명:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.search_entry = ttk.Entry(search_frame, width=40)
        self.search_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.search_entry.bind("<Return>", lambda event: self.start_search())
        
        self.search_button = ttk.Button(search_frame, text="검색 및 매물 수집", command=self.start_search)
        self.search_button.grid(row=0, column=2, padx=5, pady=5)
        
        self.settings_button = ttk.Button(search_frame, text="⚙", width=3, command=self.open_settings)
        self.settings_button.grid(row=0, column=3, padx=5, pady=5)
        
        search_frame.columnconfigure(1, weight=1)
    
    def setup_multi_search_ui(self):
        """다중 단지 검색 UI 설정"""
        main_frame = ttk.Frame(self.tab2, padding=10)
        main_frame.pack(fill="both", expand=True)
        
        left_frame = ttk.LabelFrame(main_frame, text="단지 목록 입력 (최대 50개)")
        left_frame.pack(side="left", fill="both", expand=True, padx=(0, 5))
        
        ttk.Label(left_frame, text="단지명을 한 줄에 하나씩 입력하세요:").pack(anchor="w", pady=(5, 0))
        self.complex_text = scrolledtext.ScrolledText(left_frame, width=30, height=15)
        self.complex_text.pack(fill="both", expand=True, pady=5)
        
        right_frame = ttk.LabelFrame(main_frame, text="검색 옵션")
        right_frame.pack(side="right", fill="both", padx=(5, 0))
        
        ttk.Label(right_frame, text="전용면적 범위:").grid(row=0, column=0, sticky="w", pady=5)
        
        area_frame = ttk.Frame(right_frame)
        area_frame.grid(row=0, column=1, sticky="ew", pady=5)
        
        self.min_area_var = tk.StringVar()
        self.max_area_var = tk.StringVar()
        
        ttk.Entry(area_frame, textvariable=self.min_area_var, width=6).pack(side="left")
        ttk.Label(area_frame, text="㎡ ~").pack(side="left", padx=2)
        ttk.Entry(area_frame, textvariable=self.max_area_var, width=6).pack(side="left")
        ttk.Label(area_frame, text="㎡").pack(side="left", padx=2)
        
        ttk.Label(right_frame, text="층수 조건:").grid(row=1, column=0, sticky="w", pady=5)
        ttk.Label(right_frame, text="중층/고층/5층 이상 매물만 검색").grid(row=1, column=1, sticky="w", pady=5)
        
        ttk.Label(right_frame, text="거래 유형:").grid(row=2, column=0, sticky="w", pady=5)
        ttk.Label(right_frame, text="매매 및 전세 최저가 조사").grid(row=2, column=1, sticky="w", pady=5)
        
        button_frame = ttk.Frame(right_frame)
        button_frame.grid(row=3, column=0, columnspan=2, pady=10)
        
        self.multi_search_button = ttk.Button(button_frame, text="다중 검색 시작", command=self.start_multi_search, width=20)
        self.multi_search_button.pack(pady=5)
        
        self.progress_text = scrolledtext.ScrolledText(right_frame, width=30, height=10, state='disabled')
        self.progress_text.grid(row=4, column=0, columnspan=2, sticky="ew", pady=5)
    
    def setup_author_info(self):
        """제작자 정보 프레임 설정"""
        author_frame = ttk.Frame(self.root)
        author_frame.pack(side="bottom", fill="x", padx=10, pady=10)
        
        author_label = tk.Label(author_frame, text="만든이 부태리", font=("맑은 고딕", 9, "bold"), fg="black", cursor="hand2")
        author_label.pack(anchor="center")
        author_label.bind("<Button-1>", lambda e: self.open_blog())
        
        blog_label = tk.Label(author_frame, text="https://blog.naver.com/landlover333", font=("맑은 고딕", 8))
        blog_label.pack(anchor="center")
        blog_label.bind("<Button-1>", lambda e: self.open_blog())
    
    def open_blog(self):
        """블로그 링크 열기"""
        webbrowser.open("https://blog.naver.com/landlover333")
    
    def search_complex_by_api(self, keyword):
        """API를 사용하여 단지 검색"""
        try:
            # URL 인코딩
            import urllib.parse
            encoded_keyword = urllib.parse.quote(keyword)
            
            # 자동완성 API 호출
            url = f"https://fin.land.naver.com/front-api/v1/search/autocomplete/complexes?keyword={encoded_keyword}&size=10&page=0"
            
            # 더 완전한 헤더 설정으로 봇 감지 회피
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                "Accept": "application/json, text/plain, */*",
                "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                "Accept-Encoding": "gzip, deflate, br",
                "Origin": "https://fin.land.naver.com",
                "Referer": "https://fin.land.naver.com/",
                "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
                "Sec-Ch-Ua-Mobile": "?0",
                "Sec-Ch-Ua-Platform": '"Windows"',
                "Sec-Fetch-Dest": "empty",
                "Sec-Fetch-Mode": "cors",
                "Sec-Fetch-Site": "same-origin",
                "Cache-Control": "no-cache",
                "Pragma": "no-cache"
            }
            
            # 세션 사용으로 쿠키 유지
            if not hasattr(self, 'session'):
                self.session = requests.Session()
                # 먼저 메인 페이지 방문하여 쿠키 획득
                self.session.get("https://fin.land.naver.com/", headers=headers)
                time.sleep(0.5)
            
            self.log(f"API로 '{keyword}' 검색 중...")
            
            # 재시도 로직 추가
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = self.session.get(url, headers=headers, timeout=10)
                    
                    if response.status_code == 429:
                        # Rate limit 에러 시 대기 후 재시도
                        wait_time = (attempt + 1) * 2  # 2, 4, 6초 대기
                        self.log(f"요청 제한 발생. {wait_time}초 대기 후 재시도... (시도 {attempt+1}/{max_retries})")
                        time.sleep(wait_time)
                        continue
                    
                    if response.status_code != 200:
                        self.log(f"API 요청 실패: 상태 코드 {response.status_code}")
                        
                        # 대체 방법: 일반 검색 API 사용
                        if attempt == max_retries - 1:
                            return self.search_complex_alternative(keyword)
                        continue
                    
                    data = response.json()
                    
                    # 결과 파싱
                    if 'result' in data and 'list' in data['result']:
                        complex_list = []
                        for item in data['result']['list']:
                            complex_info = {
                                'name': item.get('complexName', ''),
                                'id': str(item.get('complexNumber', '')),
                                'address': item.get('addressName', '')
                            }
                            complex_list.append(complex_info)
                        
                        if complex_list:
                            return complex_list
                        else:
                            # 자동완성 결과가 없으면 대체 검색 시도
                            return self.search_complex_alternative(keyword)
                    else:
                        self.log("API 응답에 예상된 데이터가 없습니다.")
                        if attempt == max_retries - 1:
                            return self.search_complex_alternative(keyword)
                        
                except requests.exceptions.RequestException as e:
                    self.log(f"네트워크 오류 (시도 {attempt+1}/{max_retries}): {str(e)}")
                    if attempt < max_retries - 1:
                        time.sleep(2)
                    else:
                        return None
            
            return None
                
        except Exception as e:
            self.log(f"API 검색 중 오류: {str(e)}")
            return None
    
    def search_complex_alternative(self, keyword):
        """대체 검색 방법 - 일반 검색 API 사용"""
        try:
            import urllib.parse
            encoded_keyword = urllib.parse.quote(keyword)
            
            # 대체 URL 패턴들
            alternative_urls = [
                f"https://fin.land.naver.com/front-api/v1/search/complexes?keyword={encoded_keyword}&page=0&pageSize=20",
                f"https://fin.land.naver.com/front-api/v1/complex/search?query={encoded_keyword}"
            ]
            
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                "Accept": "application/json, text/plain, */*",
                "Referer": "https://fin.land.naver.com/",
                "Origin": "https://fin.land.naver.com"
            }
            
            self.log("대체 검색 방법 시도 중...")
            
            for url in alternative_urls:
                try:
                    if hasattr(self, 'session'):
                        response = self.session.get(url, headers=headers, timeout=10)
                    else:
                        response = requests.get(url, headers=headers, timeout=10)
                    
                    if response.status_code == 200:
                        data = response.json()
                        # 응답 구조에 따라 파싱 조정
                        if 'result' in data:
                            # 결과 처리 로직
                            complex_list = []
                            items = data['result'].get('list', []) or data['result'].get('items', [])
                            
                            for item in items:
                                complex_info = {
                                    'name': item.get('complexName', item.get('name', '')),
                                    'id': str(item.get('complexNumber', item.get('complexNo', ''))),
                                    'address': item.get('addressName', item.get('address', ''))
                                }
                                if complex_info['id']:  # ID가 있는 경우만 추가
                                    complex_list.append(complex_info)
                            
                            if complex_list:
                                self.log(f"대체 방법으로 {len(complex_list)}개 단지 발견")
                                return complex_list
                        
                except Exception as e:
                    continue
            
            # 모든 방법 실패 시 기본 Selenium 방식으로 전환
            self.log("API 검색 실패. 웹 스크래핑 방식으로 전환합니다...")
            return self.search_complex_with_selenium(keyword)
            
        except Exception as e:
            self.log(f"대체 검색 중 오류: {str(e)}")
            return None
    
    def search_complex_with_selenium(self, keyword):
        """Selenium을 사용한 폴백 검색 방법"""
        try:
            from selenium import webdriver
            from selenium.webdriver.common.by import By
            from selenium.webdriver.common.keys import Keys
            from selenium.webdriver.chrome.options import Options
            from selenium.webdriver.chrome.service import Service
            from webdriver_manager.chrome import ChromeDriverManager
            
            self.log("웹 브라우저 방식으로 검색 중...")
            
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("--disable-blink-features=AutomationControlled")
            chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
            chrome_options.add_experimental_option('useAutomationExtension', False)
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
            
            service = Service(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service, options=chrome_options)
            
            # 자동화 감지 방지 스크립트
            driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
                'source': '''
                    Object.defineProperty(navigator, 'webdriver', {
                        get: () => undefined
                    })
                '''
            })
            
            # 검색 수행
            search_url = f"https://fin.land.naver.com/search?q={keyword}"
            driver.get(search_url)
            time.sleep(3)
            
            # 검색 결과 파싱
            complex_list = []
            
            # 여러 가능한 선택자 시도
            selectors = [
                "a[href*='/complexes/']",
                ".search_list .item_link",
                ".complex_item a"
            ]
            
            for selector in selectors:
                try:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    if elements:
                        for element in elements[:10]:  # 최대 10개만
                            href = element.get_attribute('href')
                            if href and '/complexes/' in href:
                                # URL에서 단지 ID 추출
                                import re
                                match = re.search(r'/complexes/(\d+)', href)
                                if match:
                                    complex_id = match.group(1)
                                    complex_name = element.text.strip() or keyword
                                    
                                    complex_list.append({
                                        'name': complex_name,
                                        'id': complex_id,
                                        'address': ''
                                    })
                        
                        if complex_list:
                            break
                            
                except Exception as e:
                    continue
            
            driver.quit()
            
            if complex_list:
                self.log(f"웹 스크래핑으로 {len(complex_list)}개 단지 발견")
                return complex_list
            
            return None
            
        except Exception as e:
            self.log(f"Selenium 검색 중 오류: {str(e)}")
            return None
    
    def start_search(self):
        """단지명으로 검색 시작"""
        search_keyword = self.search_entry.get().strip()
        if not search_keyword:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        self.search_button.config(state=tk.DISABLED)
        threading.Thread(target=self.search_complex, args=(search_keyword,), daemon=True).start()
    
    def search_complex(self, search_keyword):
        """단지명으로 검색 및 단지번호 찾기 - API 방식"""
        try:
            self.log(f"'{search_keyword}' 단지 검색을 시작합니다...")
            
            # API로 단지 검색
            complex_list = self.search_complex_by_api(search_keyword)
            
            if not complex_list:
                self.log("검색 결과가 없습니다.")
                self.root.after(0, lambda: messagebox.showwarning("경고", "검색 결과가 없습니다. 다른 검색어를 입력해보세요."))
                return
            
            self.log(f"{len(complex_list)}개의 단지를 찾았습니다.")
            
            if len(complex_list) == 1:
                # 단일 결과 - 바로 사용
                selected_complex = complex_list[0]
                self.complex_data = selected_complex
                self.log(f"단지 선택: {selected_complex['name']} (번호: {selected_complex['id']})")
                self.download_data(selected_complex['id'], selected_complex['name'])
            else:
                # 여러 결과 - 선택 대화상자 표시
                def show_selection_dialog():
                    dialog = ComplexSelectionDialog(self.root, "단지 선택", complex_list)
                    selected_complex = dialog.result
                    
                    if selected_complex and selected_complex["id"]:
                        self.complex_data = selected_complex
                        self.log(f"단지 선택: {selected_complex['name']} (번호: {selected_complex['id']})")
                        # 다시 스레드로 실행
                        threading.Thread(target=self.download_data, 
                                       args=(selected_complex['id'], selected_complex['name']), 
                                       daemon=True).start()
                    else:
                        self.log("단지 선택이 취소되었습니다.")
                        self.search_button.config(state=tk.NORMAL)
                
                # 메인 스레드에서 대화상자 표시
                self.root.after(0, show_selection_dialog)
                
        except Exception as e:
            self.log(f"오류 발생: {str(e)}")
            self.root.after(0, lambda: messagebox.showerror("오류", f"검색 중 오류가 발생했습니다.\n{str(e)}"))
        finally:
            # 대화상자가 표시되는 경우가 아니면 버튼 활성화
            if not complex_list or len(complex_list) == 1:
                self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
    
    def find_complex_id(self, complex_name):
        """다중 검색용 단지번호 찾기 - API 방식"""
        try:
            self.append_progress_text(f"  - '{complex_name}' 단지 검색 중...")
            
            complex_list = self.search_complex_by_api(complex_name)
            
            if not complex_list:
                return None
            
            # 첫 번째 결과 사용 (다중 검색에서는 자동 선택)
            return complex_list[0]['id']
            
        except Exception as e:
            self.append_progress_text(f"  - 단지번호 검색 중 오류: {str(e)}")
            return None
    
    def start_multi_search(self):
        """다중 단지 검색 시작"""
        complex_text = self.complex_text.get('1.0', tk.END).strip()
        complex_list = [name.strip() for name in complex_text.split('\n') if name.strip()]
        
        if len(complex_list) > 50:
            messagebox.showwarning("경고", "최대 50개까지만 검색 가능합니다. 처음 50개만 처리합니다.")
            complex_list = complex_list[:50]
        
        if not complex_list:
            messagebox.showwarning("경고", "검색할 단지명을 입력하세요.")
            return
        
        min_area = self.min_area_var.get().strip()
        max_area = self.max_area_var.get().strip()
        
        try:
            min_area = float(min_area) if min_area else None
            max_area = float(max_area) if max_area else None
        except ValueError:
            messagebox.showwarning("경고", "전용면적은 숫자로 입력하세요.")
            return
        
        search_options = {
            'min_area': min_area,
            'max_area': max_area
        }
        
        self.clear_progress_text()
        self.append_progress_text(f"총 {len(complex_list)}개 단지 검색을 시작합니다.\n")
        
        self.multi_search_button.config(state=tk.DISABLED)
        
        threading.Thread(target=self.process_multi_search, 
                        args=(complex_list, search_options), 
                        daemon=True).start()
    
    def process_multi_search(self, complex_list, search_options):
        """다중 단지 검색 처리"""
        try:
            all_results = []
            summary_results = []
            
            # 세션 초기화 (한 번만)
            if not hasattr(self, 'session'):
                self.session = requests.Session()
                # 메인 페이지 방문
                try:
                    self.session.get("https://fin.land.naver.com/", timeout=10)
                    time.sleep(2)
                except:
                    pass
            
            for idx, complex_name in enumerate(complex_list):
                self.append_progress_text(f"\n[{idx+1}/{len(complex_list)}] '{complex_name}' 검색 중...")
                
                # 각 단지 사이에 충분한 대기 시간
                if idx > 0:
                    self.append_progress_text("  - 다음 단지 검색 전 5초 대기...")
                    time.sleep(5)
                
                # API로 단지 정보 검색
                complex_id = self.find_complex_id(complex_name)
                
                if not complex_id:
                    self.append_progress_text(f"  - 단지를 찾을 수 없습니다.")
                    continue
                
                self.append_progress_text(f"  - 단지번호: {complex_id}")
                
                # 매물 정보 수집 (개선된 방식 사용)
                property_data = self.collect_property_data(complex_id, complex_name, search_options)
                
                if not property_data:
                    self.append_progress_text(f"  - 매물 정보가 없거나 수집 실패")
                    continue
                
                df = pd.DataFrame(property_data)
                
                all_results.append({
                    'complex_name': complex_name,
                    'complex_id': complex_id,
                    'properties': df
                })
                
                # 최저가 찾기 로직은 기존과 동일
                # ... (기존 최저가 찾기 코드 유지)
                
                # 전용면적 필터링
                if search_options['min_area'] or search_options['max_area']:
                    area_conditions = []
                    if search_options['min_area']:
                        area_conditions.append(f"전용면적 >= {search_options['min_area']}")
                    if search_options['max_area']:
                        area_conditions.append(f"전용면적 <= {search_options['max_area']}")
                    
                    area_query = " and ".join(area_conditions)
                    try:
                        df = df.query(area_query)
                    except Exception as e:
                        self.append_progress_text(f"  - 전용면적 필터링 오류: {str(e)}")
                
                # 층수 필터링
                try:
                    df = df.copy()
                    
                    def is_high_floor(floor_info):
                        floor_str = str(floor_info)
                        if pd.isna(floor_info) or floor_str.strip() == '':
                            return False
                        if '저' in floor_str:
                            return False
                        if '중' in floor_str or '고' in floor_str:
                            return True
                        parts = floor_str.split('/')
                        if len(parts) >= 1:
                            try:
                                floor_nums = re.findall(r'\d+', parts[0])
                                if floor_nums:
                                    floor_num = int(floor_nums[0])
                                    return floor_num >= 5
                            except:
                                pass
                        return False
                    
                    df['is_high_floor'] = df['층/전체층'].apply(is_high_floor)
                    df_high = df[df['is_high_floor']]
                    
                    if not df_high.empty:
                        df = df_high
                
                except Exception as e:
                    self.append_progress_text(f"  - 층수 필터링 오류: {str(e)}")
                
                # 매매/전세 최저가 찾기
                summary = {
                    '단지명': complex_name,
                    '단지ID': complex_id,
                    '전용면적_Min': search_options.get('min_area', ''),
                    '전용면적_Max': search_options.get('max_area', '')
                }
                
                # 매매 최저가
                try:
                    df_deal = df[df['거래유형'] == 'A1']
                    if not df_deal.empty:
                        df_deal = df_deal.copy()
                        df_deal.loc[:, '매매가_숫자'] = pd.to_numeric(df_deal['매매가'], errors='coerce')
                        df_deal = df_deal.dropna(subset=['매매가_숫자'])
                        
                        if not df_deal.empty:
                            df_deal = df_deal.sort_values('매매가_숫자')
                            deal_min = df_deal.iloc[0]
                            
                            summary['매매가'] = float(deal_min['매매가']) / 10000
                            summary['매매_동'] = deal_min['동']
                            summary['매매_층'] = deal_min['층/전체층']
                            summary['매매_면적'] = deal_min['전용면적']
                            
                            self.append_progress_text(f"  - 매매 최저가: {summary['매매가']:.1f}만원")
                    else:
                        summary['매매가'] = ''
                        summary['매매_동'] = ''
                        summary['매매_층'] = ''
                        summary['매매_면적'] = ''
                except Exception as e:
                    self.append_progress_text(f"  - 매매 최저가 계산 오류: {str(e)}")
                    summary['매매가'] = ''
                    summary['매매_동'] = ''
                    summary['매매_층'] = ''
                    summary['매매_면적'] = ''
                
                # 전세 최저가
                try:
                    df_jeonse = df[df['거래유형'] == 'B1']
                    if not df_jeonse.empty:
                        df_jeonse = df_jeonse.copy()
                        df_jeonse.loc[:, '보증금_숫자'] = pd.to_numeric(df_jeonse['보증금'], errors='coerce')
                        df_jeonse = df_jeonse.dropna(subset=['보증금_숫자'])
                        
                        if not df_jeonse.empty:
                            df_jeonse = df_jeonse.sort_values('보증금_숫자')
                            jeonse_min = df_jeonse.iloc[0]
                            
                            summary['전세가'] = float(jeonse_min['보증금']) / 10000
                            summary['전세_동'] = jeonse_min['동']
                            summary['전세_층'] = jeonse_min['층/전체층']
                            summary['전세_면적'] = jeonse_min['전용면적']
                            
                            self.append_progress_text(f"  - 전세 최저가: {summary['전세가']:.1f}만원")
                    else:
                        summary['전세가'] = ''
                        summary['전세_동'] = ''
                        summary['전세_층'] = ''
                        summary['전세_면적'] = ''
                except Exception as e:
                    self.append_progress_text(f"  - 전세 최저가 계산 오류: {str(e)}")
                    summary['전세가'] = ''
                    summary['전세_동'] = ''
                    summary['전세_층'] = ''
                    summary['전세_면적'] = ''
                
                summary_results.append(summary)
            
            if summary_results:
                self.save_multi_search_results(summary_results, all_results)
            else:
                self.append_progress_text("\n검색 완료: 조건에 맞는 매물이 없습니다.")
                messagebox.showinfo("검색 완료", "조건에 맞는 매물이 없습니다.")
        
        except Exception as e:
            self.append_progress_text(f"\n오류 발생: {str(e)}")
            messagebox.showerror("오류", f"다중 검색 중 오류가 발생했습니다.\n{str(e)}")
        
        finally:
            self.root.after(0, lambda: self.multi_search_button.config(state=tk.NORMAL))
    
    def collect_property_data(self, complex_number, complex_name, search_options):
        """단지의 매물 정보 수집 (다중 검색용 - 단일 검색과 동일한 방식)"""
        try:
            # 더 완전한 헤더 설정 (단일 검색과 동일)
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                "Accept": "application/json, text/plain, */*",
                "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                "Accept-Encoding": "gzip, deflate, br",
                "Origin": "https://fin.land.naver.com",
                "Referer": f"https://fin.land.naver.com/complexes/{complex_number}?ms=37.5146,127.1079,16&a=APT:PRE:ABYG:JGC&e=RETAIL",
                "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
                "Sec-Ch-Ua-Mobile": "?0",
                "Sec-Ch-Ua-Platform": '"Windows"',
                "Sec-Fetch-Dest": "empty",
                "Sec-Fetch-Mode": "cors",
                "Sec-Fetch-Site": "same-origin",
                "Cache-Control": "no-cache",
                "Pragma": "no-cache",
                "Connection": "keep-alive"
            }
            
            # 세션 초기화 또는 재사용
            if not hasattr(self, 'session'):
                self.session = requests.Session()
            
            # 단지 페이지 먼저 방문 (정상 사용자처럼)
            complex_url = f"https://fin.land.naver.com/complexes/{complex_number}"
            try:
                self.session.get(complex_url, headers={
                    "User-Agent": headers["User-Agent"],
                    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
                }, timeout=10)
                time.sleep(2)  # 대기
            except:
                pass  # 실패해도 계속 진행
            
            all_properties = []
            page = 0
            consecutive_empty = 0
            max_pages = 30  # 다중 검색은 페이지 제한을 적게
            retry_count = 0
            max_retries = 3  # 다중 검색은 재시도 횟수 줄임
            
            while page < max_pages:
                url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
                
                # 페이지 간 딜레이 (다중 검색은 더 보수적으로)
                if page > 0:
                    time.sleep(4)  # 4초 대기
                
                try:
                    response = self.session.get(url, headers=headers, timeout=15)
                    
                    if response.status_code == 429:
                        retry_count += 1
                        if retry_count >= max_retries:
                            self.append_progress_text(f"  - 요청 제한 초과. 이 단지는 건너뜁니다.")
                            return None
                        
                        wait_time = min(retry_count * 5, 20)  # 최대 20초
                        self.append_progress_text(f"  - 요청 제한. {wait_time}초 대기... (시도 {retry_count}/{max_retries})")
                        time.sleep(wait_time)
                        continue
                    
                    if response.status_code != 200:
                        consecutive_empty += 1
                        if consecutive_empty >= 2:  # 다중 검색은 빠르게 포기
                            break
                        page += 1
                        continue
                    
                    # 성공 시 retry 카운터 리셋
                    retry_count = 0
                    
                    data = response.json()
                    
                    if 'result' not in data or 'list' not in data['result']:
                        consecutive_empty += 1
                        if consecutive_empty >= 2:
                            break
                        page += 1
                        continue
                    
                    property_list = data['result']['list']
                    
                    if not property_list:
                        # 빈 페이지면 종료
                        if page == 0:
                            self.append_progress_text("  - 매물 정보가 없습니다.")
                        break
                    
                    # 첫 페이지만 로그 출력
                    if page == 0:
                        self.append_progress_text(f"  - {len(property_list)}개의 매물 발견")
                    
                    for item in property_list:
                        property_data = self.extract_property_data(item, page)
                        all_properties.append(property_data)
                    
                    # 다음 페이지 확인
                    has_next_page = data['result'].get('hasNextPage', False)
                    if not has_next_page:
                        break
                    
                    consecutive_empty = 0
                    page += 1
                    
                except requests.exceptions.Timeout:
                    consecutive_empty += 1
                    if consecutive_empty >= 2:
                        break
                    page += 1
                    time.sleep(5)
                    
                except Exception as e:
                    self.append_progress_text(f"  - 페이지 {page} 오류: {str(e)}")
                    consecutive_empty += 1
                    if consecutive_empty >= 2:
                        break
                    page += 1
                    time.sleep(5)
            
            if all_properties:
                self.append_progress_text(f"  - 총 {len(all_properties)}개 매물 수집 완료")
            
            return all_properties
            
        except Exception as e:
            self.append_progress_text(f"  - 매물 수집 중 오류: {str(e)}")
            return None
    
    def download_data(self, complex_number, complex_name):
        """선택한 단지의 매물 정보 다운로드 실행"""
        self.log(f"'{complex_name}' 단지(번호: {complex_number})의 매물 정보 수집을 시작합니다.")
        
        def download_worker():
            try:
                # 더 완전한 헤더 설정
                headers = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                    "Accept": "application/json, text/plain, */*",
                    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                    "Accept-Encoding": "gzip, deflate, br",
                    "Origin": "https://fin.land.naver.com",
                    "Referer": f"https://fin.land.naver.com/complexes/{complex_number}?ms=37.5146,127.1079,16&a=APT:PRE:ABYG:JGC&e=RETAIL",
                    "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
                    "Sec-Ch-Ua-Mobile": "?0",
                    "Sec-Ch-Ua-Platform": '"Windows"',
                    "Sec-Fetch-Dest": "empty",
                    "Sec-Fetch-Mode": "cors",
                    "Sec-Fetch-Site": "same-origin",
                    "Cache-Control": "no-cache",
                    "Pragma": "no-cache",
                    "Connection": "keep-alive"
                }
                
                # 세션 초기화 및 쿠키 설정
                if not hasattr(self, 'session'):
                    self.session = requests.Session()
                    # 먼저 단지 페이지 방문
                    complex_url = f"https://fin.land.naver.com/complexes/{complex_number}"
                    self.log("단지 페이지 방문 중...")
                    self.session.get(complex_url, headers={
                        "User-Agent": headers["User-Agent"],
                        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
                    })
                    time.sleep(3)  # 충분한 대기 시간
                
                all_properties = []
                page = 0
                consecutive_empty = 0
                max_pages = 100
                retry_count = 0
                max_retries = 5
                
                while page < max_pages:
                    url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
                    
                    self.log(f"페이지 {page} 데이터 요청 중...")
                    
                    # 페이지 간 더 긴 딜레이
                    if page > 0:
                        time.sleep(3)  # 3초로 증가
                    
                    try:
                        response = self.session.get(url, headers=headers, timeout=15)
                        
                        if response.status_code == 429:
                            retry_count += 1
                            if retry_count >= max_retries:
                                self.log("너무 많은 재시도. 10초 후 다른 방법 시도...")
                                time.sleep(10)
                                # 세션 재생성
                                self.session = requests.Session()
                                complex_url = f"https://fin.land.naver.com/complexes/{complex_number}"
                                self.session.get(complex_url, headers={
                                    "User-Agent": headers["User-Agent"],
                                    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
                                })
                                time.sleep(5)
                                retry_count = 0
                                continue
                            
                            wait_time = min(retry_count * 5, 30)  # 최대 30초까지 대기
                            self.log(f"요청 제한 발생. {wait_time}초 대기 후 재시도... (시도 {retry_count}/{max_retries})")
                            time.sleep(wait_time)
                            continue
                        
                        if response.status_code != 200:
                            self.log(f"페이지 {page} 요청 실패: 상태 코드 {response.status_code}")
                            consecutive_empty += 1
                            if consecutive_empty >= 3:
                                break
                            page += 1
                            continue
                        
                        # 성공 시 retry 카운터 리셋
                        retry_count = 0
                        
                        data = response.json()
                        
                        if 'result' not in data or 'list' not in data['result']:
                            self.log(f"페이지 {page} 데이터 구조 오류")
                            consecutive_empty += 1
                            if consecutive_empty >= 3:
                                break
                            page += 1
                            continue
                        
                        property_list = data['result']['list']
                        
                        if not property_list:
                            self.log(f"페이지 {page}에 매물이 없습니다. 수집 종료.")
                            break
                        
                        # 매물 정보 추출
                        self.log(f"페이지 {page}에서 {len(property_list)}개의 매물 발견")
                        for item in property_list:
                            property_data = self.extract_property_data(item, page)
                            all_properties.append(property_data)
                        
                        # 다음 페이지 확인
                        has_next_page = data['result'].get('hasNextPage', False)
                        if not has_next_page:
                            self.log("마지막 페이지입니다. 수집 완료.")
                            break
                        
                        consecutive_empty = 0
                        page += 1
                        
                    except requests.exceptions.Timeout:
                        self.log(f"페이지 {page} 요청 시간 초과")
                        consecutive_empty += 1
                        if consecutive_empty >= 3:
                            break
                        page += 1
                        time.sleep(5)
                        
                    except Exception as e:
                        self.log(f"페이지 {page} 처리 중 오류: {str(e)}")
                        consecutive_empty += 1
                        if consecutive_empty >= 3:
                            break
                        page += 1
                        time.sleep(5)
                
                if not all_properties:
                    self.log("매물 정보를 수집하지 못했습니다. 다른 방법을 시도해보세요.")
                    self.root.after(0, lambda: messagebox.showwarning("경고", 
                        "매물 정보를 수집하지 못했습니다.\n"
                        "다음 방법을 시도해보세요:\n"
                        "1. 5-10분 후 다시 시도\n"
                        "2. VPN 사용\n"
                        "3. 다른 네트워크에서 시도"))
                else:
                    self.log(f"총 {len(all_properties)}개의 매물 정보를 수집했습니다.")
                    self.root.after(0, lambda: self.process_results(all_properties, complex_name))
                
            except Exception as e:
                self.log(f"데이터 수집 중 오류 발생: {str(e)}")
                self.root.after(0, lambda: messagebox.showerror("오류", f"데이터 수집 중 오류가 발생했습니다.\n{str(e)}"))
                
            finally:
                self.root.after(0, lambda: self.search_button.config(state=tk.NORMAL))
        
        worker_thread = threading.Thread(target=download_worker, daemon=True)
        worker_thread.start()
    
    def fetch_page(self, complex_number, page, headers):
        """페이지별 데이터 요청 (병렬 처리용)"""
        url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={complex_number}&dateDescending=false&userChannelType=PC&page={page}"
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                return None
            
            data = response.json()
            
            if 'result' in data and 'list' in data['result']:
                property_list = data['result']['list']
                
                if not property_list:
                    return None
                
                page_properties = []
                for item in property_list:
                    property_data = self.extract_property_data(item, page)
                    page_properties.append(property_data)
                
                return page_properties
        except Exception as e:
            self.log(f"페이지 {page} 요청 오류: {str(e)}")
            return None
        
        return None
    
    def extract_property_data(self, item, page):
        """매물 항목에서 필요한 정보 추출"""
        rep_info = item['representativeArticleInfo']
        
        property_data = {
            '단지명': rep_info.get('complexName', ''),
            '동': rep_info.get('dongName', ''),
            '거래유형': rep_info.get('tradeType', ''),
            '전용면적': rep_info['spaceInfo'].get('exclusiveSpace', ''),
            '타입구분': rep_info['spaceInfo'].get('nameType', ''),
            '층/전체층': rep_info['articleDetail'].get('floorInfo', ''),
            '방향': rep_info['articleDetail'].get('direction', ''),
            '매물특징': rep_info['articleDetail'].get('articleFeatureDescription', ''),
            '보증금': rep_info['priceInfo'].get('warrantyPrice', 0),
            '월세': rep_info['priceInfo'].get('rentPrice', 0),
            '매매가': rep_info['priceInfo'].get('dealPrice', 0),
            '프리미엄': rep_info['priceInfo'].get('premiumPrice', 0),
            '가격변동상태': rep_info['priceInfo'].get('priceChangeStatus', 0),
            '중개사명': rep_info['brokerInfo'].get('brokerageName', ''),
            'VR노출여부': rep_info['articleMediaDto'].get('isVrExposed', False) if rep_info.get('articleMediaDto') else False,
            '중개사 수': 0,
            '최소매매가': 0,
            '최대매매가': 0,
            '최소보증금': 0,
            '최대보증금': 0,
            '최소월세': 0,
            '최대월세': 0,
            '최소프리미엄': 0,
            '최대프리미엄': 0,
            '페이지번호': page
        }
        
        if 'duplicatedArticlesInfo' in item and item['duplicatedArticlesInfo'] is not None:
            property_data['중개사 수'] = item['duplicatedArticlesInfo'].get('realtorCount', 0)
            
            if 'representativePriceInfo' in item['duplicatedArticlesInfo']:
                price_info = item['duplicatedArticlesInfo']['representativePriceInfo']
                
                if 'dealPrice' in price_info:
                    property_data['최소매매가'] = price_info['dealPrice'].get('minPrice', 0)
                    property_data['최대매매가'] = price_info['dealPrice'].get('maxPrice', 0)
                
                if 'warrantyPrice' in price_info:
                    property_data['최소보증금'] = price_info['warrantyPrice'].get('minPrice', 0)
                    property_data['최대보증금'] = price_info['warrantyPrice'].get('maxPrice', 0)
                
                if 'rentPrice' in price_info:
                    property_data['최소월세'] = price_info['rentPrice'].get('minPrice', 0)
                    property_data['최대월세'] = price_info['rentPrice'].get('maxPrice', 0)
                
                if 'premiumPrice' in price_info:
                    property_data['최소프리미엄'] = price_info['premiumPrice'].get('minPrice', 0)
                    property_data['최대프리미엄'] = price_info['premiumPrice'].get('maxPrice', 0)
        
        return property_data
    
    def process_results(self, all_properties, complex_name):
        """수집된 데이터 처리 (메인 스레드에서 실행)"""
        try:
            if not all_properties:
                self.log("수집된 매물 정보가 없습니다.")
                messagebox.showinfo("알림", "수집된 매물 정보가 없습니다.")
                return
            
            df = pd.DataFrame(all_properties)
            
            trade_type_map = {
                'A1': '매매',
                'B1': '전세',
                'B2': '월세',
                'B3': '단기임대'
            }
            df['거래유형'] = df['거래유형'].map(lambda x: trade_type_map.get(x, x))
            
            direction_map = {
                'SS': '남향',
                'SE': '남동향',
                'SW': '남서향',
                'EE': '동향',
                'WW': '서향',
                'NN': '북향',
                'NE': '북동향',
                'NW': '북서향'
            }
            df['방향'] = df['방향'].map(lambda x: direction_map.get(x, x))
            
            price_change_map = {
                0: '변동없음',
                1: '가격상승',
                -1: '가격하락'
            }
            df['가격변동상태'] = df['가격변동상태'].map(lambda x: price_change_map.get(x, '알수없음'))
            
            money_columns = ['보증금', '월세', '매매가', '프리미엄', '최소매매가', '최대매매가', 
                           '최소보증금', '최대보증금', '최소월세', '최대월세', '최소프리미엄', '최대프리미엄']
            
            for col in money_columns:
                if col in df.columns:
                    df[col] = df[col].apply(lambda x: f"{x/10000:.0f}" if x > 0 else '')
            
            today = datetime.now().strftime('%Y%m%d')
            
            excel_filename = os.path.join(self.save_path, f'{complex_name}_매물정보_{today}.xlsx')
            
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='전체매물', index=False)
                self.apply_filter_to_sheet(writer.sheets['전체매물'], len(df.columns))
                
                deal_properties = df.query('매매가 != "" and 매매가.notnull()')
                if not deal_properties.empty:
                    deal_properties.to_excel(writer, sheet_name='매매매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['매매매물'], len(deal_properties.columns))
                
                full_deposit = df.query('보증금 != "" and 보증금.notnull() and (월세 == "" or 월세.isnull())')
                if not full_deposit.empty:
                    full_deposit.to_excel(writer, sheet_name='전세매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['전세매물'], len(full_deposit.columns))
                
                monthly_rent = df.query('월세 != "" and 월세.notnull()')
                if not monthly_rent.empty:
                    monthly_rent.to_excel(writer, sheet_name='월세매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['월세매물'], len(monthly_rent.columns))
                
                multi_realtor = df.query('`중개사 수` > 1')
                if not multi_realtor.empty:
                    multi_realtor.to_excel(writer, sheet_name='중복매물', index=False)
                    self.apply_filter_to_sheet(writer.sheets['중복매물'], len(multi_realtor.columns))
                
                for sheet_name in writer.sheets:
                    sheet = writer.sheets[sheet_name]
                    self.adjust_column_width(sheet)
            
            self.log(f"엑셀 파일 '{excel_filename}' 생성 완료!")
            self.log(f"총 {len(df)}개의 매물 정보가 저장되었습니다.")
            
            messagebox.showinfo("완료", f"'{complex_name}' 단지의 매물 정보가 '{excel_filename}' 파일로 저장되었습니다.")
        
        except Exception as e:
            self.log(f"결과 처리 중 오류: {str(e)}")
            messagebox.showerror("오류", f"결과 처리 중 오류가 발생했습니다.\n{str(e)}")
    
    def apply_filter_to_sheet(self, sheet, columns_count):
        """시트의 첫 행에 필터 적용"""
        try:
            sheet.auto_filter.ref = f"A1:{chr(64 + columns_count)}1"
            
            from openpyxl.styles import Font, PatternFill, Alignment
            
            header_font = Font(bold=True, size=11)
            header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
            
            for cell in sheet[1]:
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = Alignment(horizontal='center', vertical='center')
        except Exception as e:
            self.log(f"필터 적용 중 오류: {str(e)}")
    
    def adjust_column_width(self, sheet):
        """열 너비 자동 조정"""
        try:
            for column in sheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                
                for cell in column:
                    if cell.value:
                        try:
                            cell_length = len(str(cell.value))
                            max_length = max(max_length, cell_length)
                        except:
                            pass
                
                max_length = max(max_length, 10)
                max_length = min(max_length, 50)
                
                sheet.column_dimensions[column_letter].width = max_length + 2
        except Exception as e:
            self.log(f"열 너비 조정 중 오류: {str(e)}")
    
    def save_multi_search_results(self, summary_results, all_results=None):
        """다중 검색 결과를 엑셀로 저장"""
        try:
            today = datetime.now().strftime('%Y%m%d')
            filename = f'단지별_매물정보_{today}.xlsx'
            excel_filename = os.path.join(self.save_path, filename)
            
            summary_df = pd.DataFrame(summary_results)
            
            with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                summary_df.to_excel(writer, sheet_name='단지별_최저가', index=False)
                
                workbook = writer.book
                worksheet = writer.sheets['단지별_최저가']
                
                header_font = Font(bold=True, size=11)
                header_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
                
                for cell in worksheet[1]:
                    cell.font = header_font
                    cell.fill = header_fill
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                
                for column in worksheet.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    
                    for cell in column:
                        if cell.value:
                            try:
                                cell_length = len(str(cell.value))
                                max_length = max(max_length, cell_length)
                            except:
                                pass
                    
                    max_length = max(max_length, 10)
                    max_length = min(max_length, 50)
                    worksheet.column_dimensions[column_letter].width = max_length + 2
                
                worksheet.auto_filter.ref = f"A1:{chr(64 + len(summary_df.columns))}1"
            
            self.append_progress_text(f"검색 완료: 총 {len(summary_df)}개 단지의 매물 정보가 저장되었습니다.")
            self.append_progress_text(f"파일 저장 경로: {excel_filename}")
            messagebox.showinfo("검색 완료", f"총 {len(summary_df)}개 단지의 매물 정보가 저장되었습니다.\n\n파일 저장 경로: {excel_filename}")
            
        except Exception as e:
            self.append_progress_text(f"결과 저장 중 오류: {str(e)}")
            messagebox.showerror("오류", f"결과 저장 중 오류가 발생했습니다.\n{str(e)}")
    
    def append_progress_text(self, text):
        """진행 상황 텍스트 추가 (스레드 안전)"""
        def _update():
            self.progress_text.config(state='normal')
            self.progress_text.insert(tk.END, text + "\n")
            self.progress_text.see(tk.END)
            self.progress_text.config(state='disabled')
        
        if threading.current_thread() is threading.main_thread():
            _update()
        else:
            self.root.after(0, _update)
    
    def clear_progress_text(self):
        """진행 상황 텍스트 초기화"""
        self.progress_text.config(state='normal')
        self.progress_text.delete('1.0', tk.END)
        self.progress_text.config(state='disabled')
    
    def update_progress(self, count):
        """프로그레스 상태 업데이트"""
        self.log(f"현재까지 {count}개 매물 수집됨")
    
    def log(self, message):
        """스레드 안전한 로깅"""
        self.log_queue.put(message)
    
    def process_log_queue(self):
        """로그 큐의 메시지를 처리"""
        try:
            while True:
                message = self.log_queue.get_nowait()
                
                print(message)
                
                if "단지 검색을 시작합니다" in message:
                    self.status_label.config(text="검색 중...")
                elif "매물 정보 수집을 시작합니다" in message:
                    self.status_label.config(text="매물 수집 중...")
                elif "총 " in message and "개의 매물 정보가 저장되었습니다" in message:
                    match = re.search(r'총 (\d+)개', message)
                    if match:
                        count = match.group(1)
                        self.status_label.config(text=f"{count}건 매물 검색 완료")
                elif "엑셀 파일" in message and "생성 완료" in message:
                    self.status_label.config(text="파일 저장 완료")
                
                self.log_queue.task_done()
        except queue.Empty:
            pass
        
        self.root.after(100, self.process_log_queue)
    
    def load_config(self):
        """설정 파일 로드"""
        try:
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if os.path.exists(config_path):
                self.config.read(config_path)
                if 'Settings' in self.config and 'save_path' in self.config['Settings']:
                    path = self.config['Settings']['save_path']
                    if os.path.exists(path):
                        self.save_path = path
        except Exception as e:
            print(f"설정 로드 오류: {str(e)}")
    
    def save_config(self):
        """설정 파일 저장"""
        try:
            config_path = os.path.join(os.path.expanduser("~"), ".naver_realestate_config.ini")
            
            if 'Settings' not in self.config:
                self.config['Settings'] = {}
            
            self.config['Settings']['save_path'] = self.save_path
            
            with open(config_path, 'w') as f:
                self.config.write(f)
        except Exception as e:
            print(f"설정 저장 오류: {str(e)}")
    
    def open_settings(self):
        """설정 창 열기"""
        path = filedialog.askdirectory(initialdir=self.save_path, title="저장할 폴더 선택")
        
        if path:
            self.save_path = path
            self.save_config()
            self.log(f"저장 경로가 변경되었습니다: {path}")

# 메인 함수
def main():
    root = tk.Tk()
    app = NaverRealEstateApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()